In [1]:
from bs4 import BeautifulSoup
import requests
# ! pip install wikipedia
import wikipedia
from googlesearch import search 
from urllib.parse import urlparse

In [2]:
#initialize Dataframe
import pandas as pd
import numpy as np

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [3]:
#Utility
def tostr(arr):
  res=''
  if res is not None:
    for a in arr:
      res=res+a.strip()+','
    return res[:-1]
  return res

In [4]:
#Getting music director info
# ! pip install wptools
# ! pip install wikipedia
import wptools
import wikipedia
def getmusicdir(movie,year):
  #print(wikipedia.search(movie))
  try:
    #title=wikipedia.page(movie).url[30:]
    #print(title+'|'+movie+" "+year+" tamil")
    so = wptools.page(movie).get_parse()
    infobox = so.data['infobox']
    if infobox is not None:
      if 'music' in infobox.keys():
        #print(infobox['music'])
        alphanumeric = ""
        for char in infobox['music']:
          if char.isalnum() or char is '.':
            alphanumeric += char
          elif char is ' ':
            alphanumeric += char
          else:
            alphanumeric += '-'
            
        music_dir=[]
        for part in alphanumeric.split('-'):
          if part is not '' and part is not ' ':
            music_dir.append(part)
        
        music_list=[]
        for dist in music_dir:
          if dist.lower().strip() not in ['br','\n','editor','nbsp','music director','small','songs','score','original songs','singer','background score','artist','artists','composer'] and len(dist.lower().strip())>1:
            music_list.append(dist.strip())
        print(music_list)
        return music_list
      return []
    return []
  except:
    print("Exception");
    return []

In [5]:
#Get IMDB rating
def getrating(movie,year):
  for j in search(str(movie).strip()+' '+ str(year).strip() +' tamil imdb', tld="co.in", num=5, stop=5, pause=2): 
    parsed_uri = urlparse(j)
    result = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
    if(result.strip()=='https://www.imdb.com/'):
      print(j)
      webdata=requests.get(j).text
      soup=BeautifulSoup(webdata,'lxml')
      if(len(soup.find_all('span',itemprop='ratingValue'))>0):
        rating=soup.find_all('span',itemprop='ratingValue')[0].text
        return str(rating)
      break
  return ''

In [6]:
xtest=getmusicdir('2.0_(film)','2018')
xtest

en.wikipedia.org (parse) 2.0_(film)
en.wikipedia.org (imageinfo) File:2.0 film poster.jpg


['A. R. Rahman']


2.0 (film) (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:2.0 film ...
  infobox: <dict(20)> name, image, caption, director, producer, wr...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:2...
  pageid: 48116360
  parsetree: <str(95020)> <root><template><title>Short description...
  requests: <list(2)> parse, imageinfo
  title: 2.0 (film)
  wikibase: Q21095433
  wikidata_url: https://www.wikidata.org/wiki/Q21095433
  wikitext: <str(76481)> {{Short description|2018 Indian science f...
}


['A. R. Rahman']

In [7]:
def getAllMusic():
    df['cast_size']=''
    for index,row in df.iterrows():
        mlink=row['cast']
        if mlink is not None:
            arr=str(mlink).split(',')
            df.at[index,'cast_size']=len(arr)

    for index,row in df.iterrows():
      mlink=row['link']
      if mlink is not None:
        print(str(mlink).strip()[6:])
        dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
        strdir=tostr(dirs)
        print(str(index)+' | '+strdir)
        df.at[index,'music']=strdir


In [8]:
def getAllrating():
    for index,row in df.iterrows():
      mname=row['title']
      myear=row['year']
      if mname is not None:
        rating=getrating(mname,myear)
        print(mname+' | '+rating)
        df.at[index,'imdb']=rating.strip()

In [33]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_2018'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_2018=soup.find_all('table')[2:5]

In [34]:
standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

In [35]:
for table in tables_2018:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='2018'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')-1

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')-1

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    
    for datarow in rows:
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Opening', 'Title', 'Director', 'Cast', 'Studio', 'Ref']
1!Mercury|Karthik Subbaraj|Prabhu Deva, Remya Nambeesan, Sananth, Indhuja|null|Stone Bench Creations|/wiki/Mercury_(2018_film)
2!Munthal|Stunt Jayanth|Appu Krishna, Muksha, Rajendran, Nizhalgal Ravi|null|Harvest Moon Pictures|
3!Diya|A. L. Vijay|Sai Pallavi, Naga Shaurya, Veronika Arora|null|Lyca Productions|/wiki/Diya_(film)
4!Paadam|Rajashekar|Karthik, Vijith, Mona, Yashika Aannand|null|Rollon Movies|/wiki/Paadam
5!Pakka|S. S. Surya|Vikram Prabhu, Nikki Galrani, Bindu Madhavi|null|Benn Consortium Studios|/wiki/Pakka_(film)
6!Sila Samayangalil|Priyadarshan|Prakash Raj, Sriya Reddy, Ashok Selvan, Varun|null|Prabhu Deva Studios,Think Big Studios|/wiki/Sometimes_(film)
7!Alai Pesi|Murali Bharathi|Akhil, Anu Krishna, Singampuli|null|Vijayalakshmi Creations|
8!Iruttu Araiyil Murattu Kuththu|Santhosh P. Jayakumar|Gautham Karthik, Vaibhavi Shandilya, Sha Ra, Yashika Aannand, Chandrika Ravi|null|Blue Ghost Pictures|/wiki/Iruttu_Araiyil

109!Ezhumin|V. P. Viji|Vivek, Devayani, Azhagam Perumal|null|Vaiyam Mediyas|/wiki/Ezhumin
110!Sandakozhi 2|N. Linguswamy|Vishal, Keerthy Suresh, Varalaxmi Sarathkumar, Rajkiran|null|Vishal Film Factory|/wiki/Sandakozhi_2
111!Genius|Suseenthiran|Roshan, Priyaa Lal, Singampuli, Aadukalam Naren|null|Sakthi Film Factory|/wiki/Genius_(2018_Tamil_film)
112!Jarugandi|A. N. Pitchumani|Jai, Reba Monica John, Amit Tiwari, Daniel Annie Pope|null|Shvedh Group|/wiki/Jarugandi
113!Raaga Thaalangal|R. Thirupathi Rajaji|Vijayan, Silk Smitha|null|Sri Dandayudhapani Movies|
114!Santhoshathil Kalavaram|Kranthi Prasad|Ravi Mariya, Rudra Aura, Niranth, Kalyan|null|Sree Guru Cinemas|/wiki/Santhoshathil_Kalavaram
115!Vanmurai Paguthi|Naga|Manikandan, Manogara, Raja, Rafia Jaffer|null|Aaruthra Cine Productions|
116!Billa Pandi|Saravanasakthi|R. K. Suresh, Indhuja, Chandini Tamilarasan|null|J. K. Film Productions|/wiki/Billa_Pandi
117!Kalavani Mappillai|Gandhi Manivasagam|Dinesh, Adhiti Menon, Devayani|null|Ra

In [36]:
df=df.drop(0,axis=0)
df

year                                   title                director  \
1    2018                                 Mercury        Karthik Subbaraj   
2    2018                                 Munthal           Stunt Jayanth   
3    2018                                    Diya             A. L. Vijay   
4    2018                                  Paadam              Rajashekar   
5    2018                                   Pakka             S. S. Surya   
6    2018                       Sila Samayangalil            Priyadarshan   
7    2018                               Alai Pesi         Murali Bharathi   
8    2018          Iruttu Araiyil Murattu Kuththu   Santhosh P. Jayakumar   
9    2018                   Kaathiruppor Pattiyal   Balayya D. Rajasekhar   
10   2018                 Iravukku Aayiram Kangal               Mu. Maran   
11   2018            Aaru Mudhal Aaru (Oru Iravu)             M. G. Reddy   
12   2018                           Irumbu Thirai           P. S. Mithran   
13   2018                     Nadigaiyar Thilagam              Nag Ashwin   
14   2018                      Bhaskar Oru Rascal                Siddique   
15   2018                             18 May 2009         Kuppan Ganeshan   
16   2018                                   Kaali   Kiruthiga Udhayanidhi   
17   2018              Kadhalargal Valibar Sangam       Kasi Vishwanathan   
18   2018                               Paalkaari  Om Pulli Jeevarathinam   
19   2018                                   Seyal             Ravi Abbulu   
20   2018                          Abhiyum Anuvum     B. R. Vijayalakshmi   
21   2018                       Oru Kuppai Kathai         Kaali Rangasamy   
22   2018                            Kaala Koothu            M. Nagarajan   
23   2018                       Pei Irukka Illaya        Pa. Ranjithkumar   
24   2018                       Puthiya Bruce Lee       Mulaiyur A. Sonai   
25   2018                                   Semma              Vallikanth   
26   2018                                  Antony             Kutti Kumar   
27   2018                                  Mohana               Ra. Anand   
28   2018                             Panjumittai             S. P. Mohan   
29   2018                     Vayakattu Mappillai           Vimal Murugan   
30   2018                                X Videos             Sajo Sundar   
..    ...                                     ...                     ...   
115  2018                        Vanmurai Paguthi                    Naga   
116  2018                             Billa Pandi          Saravanasakthi   
117  2018                      Kalavani Mappillai      Gandhi Manivasagam   
118  2018                                  Sarkar           AR Murugadoss   
119  2018                           Kaatrin Mozhi             Radha Mohan   
120  2018                      Thimiru Pudichavan                Ganeshaa   
121  2018                       Utharavu Maharaja           Asif Kuraishi   
122  2018                               Karimugan        Chella Thangaiah   
123  2018  Karthikeyanum Kaanamal Pona Kadhaliyum              M. A. Bala   
124  2018                           Pattinapakkam                 Jayadev   
125  2018                              Sagavaasam        S. Thomas Newton   
126  2018                                     Sei                Raj Babu   
127  2018                            Semmari Aadu     Sathish Subramanyam   
128  2018                                   Vandi            Rajeesh Bala   
129  2018                                     2.0                 Shankar   
130  2018                      Dhoni Kabadi Kuzhu              P. Iyappan   
131  2018          Evanukku Engeyo Matcham Irukku            A. R. Mukesh   
132  2018                             Seemathurai   Santhosh Thiyagarajan   
133  2018                            Vinai Ariyar           K. T. Murugan   
134  2018                      Bayangaramana Aalu             Arasar Raja   
1

In [38]:
df['cast_size']=''
for index,row in df.iterrows():
    mlink=row['cast']
    if mlink is not None:
        arr=str(mlink).split(',')
        df.at[index,'cast_size']=len(arr)

In [53]:
for index,row in df.iterrows():
  mlink=row['link']
  if mlink is not None:
    print(str(mlink).strip()[6:])
    dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
    strdir=tostr(dirs)
    print(str(index)+' | '+strdir)
    df.at[index,'music']=strdir


Mercury_(2018_film)


en.wikipedia.org (parse) Mercury_(2018_film)
en.wikipedia.org (imageinfo) File:Mercury film.jpg
Mercury (film) (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Mercury f...
  infobox: <dict(15)> name, image, caption, director, producer, wr...
  pageid: 54375444
  parsetree: <str(10607)> <root><template><title>short description...
  requests: <list(2)> parse, imageinfo
  title: Mercury (film)
  wikibase: Q30612742
  wikidata_url: https://www.wikidata.org/wiki/Q30612742
  wikitext: <str(6167)> {{short description|2018 film by Karthik S...
}
en.wikipedia.org (parse) Munthal


['Santhosh Narayanan']
1 | Santhosh Narayanan



API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Diya_(film)


Exception
2 | 
Diya_(film)


...

en.wikipedia.org (parse) Maari_2
en.wikipedia.org (imageinfo) File:Maari 2.jpg
Maari 2 (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Maari 2.j...
  infobox: <dict(16)> name, image, caption, director, writer, prod...
  pageid: 56403621
  parsetree: <str(21805)> <root><template><title>short description...
  requests: <list(2)> parse, imageinfo
  title: Maari 2
  wikibase: Q50819295
  wikidata_url: https://www.wikidata.org/wiki/Q50819295
  wikitext: <str(13676)> {{short description|2018 Indian Tamil-lan...
}
en.wikipedia.org (parse) Silukkuvarupatti_Singam


['Yuvan Shankar Raja']
143 | Yuvan Shankar Raja
Silukkuvarupatti_Singam


en.wikipedia.org (imageinfo) File:Silukkuvarupatti Singam.jpg


['Leon James', 'composer', 'Leon James']
144 | Leon James,composer,Leon James


Silukkuvarupatti Singam (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Silukkuva...
  infobox: <dict(16)> name, image, caption, director, producer, wr...
  pageid: 53753141
  parsetree: <str(17779)> <root><template><title>short description...
  requests: <list(2)> parse, imageinfo
  title: Silukkuvarupatti Singam
  wikibase: Q29467870
  wikidata_url: https://www.wikidata.org/wiki/Q29467870
  wikitext: <str(12829)> {{short description|2018 film directed by...
}


In [61]:
for index,row in df.iterrows():
  mname=row['title']
  myear=row['year']
  if mname is not None:
    rating=getrating(mname,myear)
    print(mname+' | '+rating)
    df.at[index,'imdb']=rating.strip()

https://www.imdb.com/title/tt7881558/
Mercury | 6.2
https://www.imdb.com/search/title/?title_type=feature&year=2018-01-01,2018-12-31&languages=ta&sort=user_rating,desc&explore=languages
Munthal | 
https://www.imdb.com/title/tt7695522/
Diya | 5.7
https://www.imdb.com/title/tt8333786/
Paadam | 4.9
https://www.imdb.com/title/tt7853278/
Pakka | 1.5
https://www.imdb.com/title/tt5877882/
Sila Samayangalil | 7.2
https://www.imdb.com/title/tt8364398/
Alai Pesi | 
https://www.imdb.com/title/tt7510220/
Iruttu Araiyil Murattu Kuththu | 4.2
https://www.imdb.com/title/tt8176308/
Kaathiruppor Pattiyal | 5.3
https://www.imdb.com/title/tt7179714/
Iravukku Aayiram Kangal | 7.3
https://www.imdb.com/title/tt8458764/
Aaru Mudhal Aaru (Oru Iravu) | 
https://www.imdb.com/title/tt07179784/
Irumbu Thirai | 7.7
https://www.imdb.com/title/tt7465992/
Nadigaiyar Thilagam | 8.5
https://www.imdb.com/title/tt7088032/
Bhaskar Oru Rascal | 4.7
https://www.imdb.com/title/tt9074156/
18 May 2009 | 
https://www.imdb.com/t

Johnny | 5.4
https://www.imdb.com/title/tt11318622/
Prabha | 
https://www.imdb.com/title/tt12916108/
Thiru | 
https://www.imdb.com/title/tt9466378/
Thulam | 
https://www.imdb.com/title/tt7852918/
Thuppakki Munai | 6.4
https://www.imdb.com/title/tt8011324/
Seethakaathi | 7.0
https://www.imdb.com/title/tt08364418/
Adanga Maru | 7.2
https://www.imdb.com/title/tt8458718/
Kanaa | 7.8
https://www.imdb.com/title/tt7510798/
Maari 2 | 5.6
https://www.imdb.com/title/tt7853260/
Silukkuvarupatti Singam | 5.4


In [63]:
df.to_csv('datav3_1_extended.csv')

In [85]:
df.head()

year    title          director  \
1  2018  Mercury  Karthik Subbaraj   
2  2018  Munthal     Stunt Jayanth   
3  2018     Diya       A. L. Vijay   
4  2018   Paadam        Rajashekar   
5  2018    Pakka       S. S. Surya   

                                              cast genre  \
1   Prabhu Deva, Remya Nambeesan, Sananth, Indhuja  null   
2  Appu Krishna, Muksha, Rajendran, Nizhalgal Ravi  null   
3        Sai Pallavi, Naga Shaurya, Veronika Arora  null   
4           Karthik, Vijith, Mona, Yashika Aannand  null   
5      Vikram Prabhu, Nikki Galrani, Bindu Madhavi  null   

                  producer                       link cast_size  \
1    Stone Bench Creations  /wiki/Mercury_(2018_film)         4   
2    Harvest Moon Pictures                                    4   
3         Lyca Productions          /wiki/Diya_(film)         3   
4            Rollon Movies               /wiki/Paadam         4   
5  Benn Consortium Studios         /wiki/Pakka_(film)         3   

                music imdb  
1  Santhosh Narayanan  6.2  
2                           
3              Sam CS  5.7  
4  Ganesh Raghavendra  4.9  
5           C. Sathya  1.5

# 2009

In [78]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_2009'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_2009=soup.find_all('table')[0:1]

In [79]:
standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

In [88]:
df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [89]:
for table in tables_2009:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='2009'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')-1

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')-1

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    if 'Production' in column_names:
      producer_index=column_names.index('Production')-1
      producer_member=True
    
    for datarow in rows:
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Opening', 'Title', 'Director', 'Cast', 'Production', 'Music Director', 'Genre']
1!A Aa E Ee|D. Sabapathy|Navdeep, Monica, Aravind Akash, Saranya Mohan|Comedy||/wiki/A_Aa_E_Ee_(2009_Tamil_film)
2!Villu|Prabhu Deva|Vijay, Nayantara|Action||/wiki/Villu_(film)
3!Kadhalna Summa Illai|Ilankannan|Ravi Krishna, Sharwanand, Kamalinee Mukherjee|Drama||/wiki/Kadhalna_Summa_Illai
4!Padikathavan|Suraj|Dhanush, Tamannaah|Action||/wiki/Padikkadavan_(2009_film)
5!Vennila Kabadi Kuzhu|Suseenthiran|Vishnu Vishal, Saranya Mohan, Kishore|Drama||/wiki/Vennila_Kabadi_Kuzhu
6!Gaja|Ganapathy|V. N. R, Akshaya|Drama||
7!Satrumun Kidaitha Thagaval|Bhuvanai Kannan|Kanal Kannan, Bharathi, Kushboo|Thriller||/wiki/Satrumun_Kidaitha_Thagaval
8!Naan Kadavul|Bala|Arya, Pooja|Drama||/wiki/Naan_Kadavul
9!Pettarasu|Saravana|Saravana, Saranya|Drama||
10!Kudiyarasu|Sabir Hussain|Vignesh, Sabarna Anand, Neepa, K. Suresh Kannan|Action Thriller||/wiki/Kudiyarasu
11!Perumal|Vincent Selva|Sundar C., Meenakshi, Namitha|Action||

97!Vedappan|Aanaivari A. Sridhar|Hemanth Kumar, Apsara, Nikitha|Drama||/wiki/Vedappan
98!Jaganmohini|N. K. Viswanathan|Raja, Namitha, Nila|Horror||/wiki/Jaganmohini_(2009_film)
99!Peranmai|S. P. Jananathan|Jayam Ravi, Vasundhara, Kadhal Saranya, Dhansika, Varsha, Liyashree|Action Thriller||/wiki/Peranmai
100!Aadhavan|K. S. Ravikumar|Suriya, Nayantara|Action Thriller||/wiki/Aadhavan
101!Kanden Kadhalai|R. Kannan|Bharath, Tamannaah|Romantic Comedy||/wiki/Kanden_Kadhalai
102!Adhe Neram Adhe Idam|M. Prabhu|Jai, Vijayalakshmi|Romance||/wiki/Adhe_Neram_Adhe_Idam
103!Kanna Nee Enakku Thaanada|Olimaaran|Udhay, Payal|Drama||
104!Karagam|Lakshmanraj|Santhosh Pandian, Aswardh|Drama||
105!Palaivana Solai|K. S. Dayalan|Nithin Sathya, Karthika|Drama||/wiki/Palaivana_Solai_(2009_film)
106!Saa Boo Thiri|Arshad Khan|Akshay, Prajin, Arshad Khan|Romantic Comedy||/wiki/Saa_Boo_Thiri
107!Thambivudayaan|Rajamahesh|Adhithya Anbu, Manisha Chatterji|Action||
108!Vaidehi|Gemini Raghava|Prithvi Rajan, Madhu Sant

In [92]:
df=df.drop(0)

In [93]:
df

year                        title                director  \
1    2009                    A Aa E Ee            D. Sabapathy   
2    2009                        Villu             Prabhu Deva   
3    2009         Kadhalna Summa Illai              Ilankannan   
4    2009                 Padikathavan                   Suraj   
5    2009         Vennila Kabadi Kuzhu            Suseenthiran   
6    2009                         Gaja               Ganapathy   
7    2009   Satrumun Kidaitha Thagaval         Bhuvanai Kannan   
8    2009                 Naan Kadavul                    Bala   
9    2009                    Pettarasu                Saravana   
10   2009                   Kudiyarasu           Sabir Hussain   
11   2009                      Perumal           Vincent Selva   
12   2009         Siva Manasula Sakthi               M. Rajesh   
13   2009                       Laadam           Prabu Solomon   
14   2009                TN 07 AL 4777       A. Lakshmikanthan   
15   2009                 Iru Nadhigal               Chanakyan   
16   2009                         Thee                G. Kicha   
17   2009                         1977      G. N. Dinesh Kumar   
18   2009                    Aarupadai  K. P. Sarvatheerthavan   
19   2009                    Innoruvan       S. T. Gunasekaran   
20   2009                Yavarum Nalam            Vikram Kumar   
21   2009                  Kanchivaram            Priyadarshan   
22   2009            Adada Enna Azhagu       T. M. Jayamurugan   
23   2009          Aadatha Aattamellam           A. B. Azhagar   
24   2009                   Nesikkiren             Nesamanavan   
25   2009                     Pattalam           Rohan Krishna   
26   2009                         Ayan             K. V. Anand   
27   2009              Ananda Tandavam          Gandhi Krishna   
28   2009               Naalai Namadhe                 Vinayan   
29   2009               Karthik Anitha                 Srihari   
30   2009            Naal Natchathiram         C. N. P. Sakthi   
..    ...                          ...                     ...   
99   2009                     Peranmai        S. P. Jananathan   
100  2009                     Aadhavan         K. S. Ravikumar   
101  2009              Kanden Kadhalai               R. Kannan   
102  2009         Adhe Neram Adhe Idam               M. Prabhu   
103  2009    Kanna Nee Enakku Thaanada               Olimaaran   
104  2009                      Karagam             Lakshmanraj   
105  2009              Palaivana Solai           K. S. Dayalan   
106  2009                Saa Boo Thiri             Arshad Khan   
107  2009               Thambivudayaan              Rajamahesh   
108  2009                      Vaidehi          Gemini Raghava   
109  2009  Swetha 5/10 Wellington Road                  Sanjay   
110  2009                  Thamizhagam         K. Suresh Kumar   
111  2009        Vizhiyil Malarnthathu              R. Sathish   
112  2009              Mathiya Chennai             Vivekanandh   
113  2009             Naan Avanillai 2                   Selva   
114  2009                         Yogi       Subramaniya Shiva   
115  2009             Edhuvum Nadakkum     Rozario, Maheswaran   
116  2009          Meendum Meendum Nee                           
117  2009                 Pinju Manasu               D. Jairam   
118  2009                    Renigunta        R. Panneerselvam   
119  2009                    Vettattam            D. S. Kannan   
120  2009                 Pachiyapuram              V. Agustin   
121  2009                       Thozhi         Elango Lakshman   
122  2009                       Vairam       S. P. Selva Durai   
123  2009                 Vettaikkaran            B. Babusivan   
124  2009                 Kandhakottai            S. Sakthivel   
125  2009                        Balam          Murali Krishna   
126  2009           Enakkul Oru Kadhal               M. Booman   
127  2009                    Odipolama       

In [95]:
df['cast_size']=''
for index,row in df.iterrows():
    mlink=row['cast']
    if mlink is not None:
        arr=str(mlink).split(',')
        df.at[index,'cast_size']=len(arr)

for index,row in df.iterrows():
  mlink=row['link']
  if mlink is not None:
    print(str(mlink).strip()[6:])
    dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
    strdir=tostr(dirs)
    print(str(index)+' | '+strdir)
    df.at[index,'music']=strdir


A_Aa_E_Ee_(2009_Tamil_film)


en.wikipedia.org (parse) A_Aa_E_Ee_(2009_Tamil_film)
A Aa E Ee (2009 Tamil film) (en) data
{
  infobox: <dict(13)> name, director, producer, writer, story, sta...
  pageid: 19609298
  parsetree: <str(8638)> <root><template><title>short description<...
  requests: <list(1)> parse
  title: A Aa E Ee (2009 Tamil film)
  wikibase: Q4655291
  wikidata_url: https://www.wikidata.org/wiki/Q4655291
  wikitext: <str(5165)> {{short description|2009 film by Sabapathy...
}
en.wikipedia.org (parse) Villu_(film)


['Vijay Antony']
1 | Vijay Antony
Villu_(film)


en.wikipedia.org (imageinfo) File:Villu-2.jpg
Villu (film) (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Villu-2.j...
  infobox: <dict(18)> name, image, caption, director, story, scree...
  pageid: 12911103
  parsetree: <str(25001)> <root><template><title>Use Indian Englis...
  requests: <list(2)> parse, imageinfo
  title: Villu (film)
  wikibase: Q7931129
  wikidata_url: https://www.wikidata.org/wiki/Q7931129
  wikitext: <str(17650)> {{Use Indian English|date=November 2015}}...
}
en.wikipedia.org (parse) Kadhalna_Summa_Illai


['Devi Sri Prasad']
2 | Devi Sri Prasad
Kadhalna_Summa_Illai
Exception
3 | 
Padikkadavan_(2009_film)


...

en.wikipedia.org (parse) Odipolama
Odipolama (en) data
{
  infobox: <dict(9)> name, director, producer, starring, music, ci...
  pageid: 25493478
  parsetree: <str(7303)> <root><template><title>Infobox film</titl...
  requests: <list(1)> parse
  title: Odipolama
  wikibase: Q7077837
  wikidata_url: https://www.wikidata.org/wiki/Q7077837
  wikitext: <str(3948)> {{Infobox film| name           = Odipolama...
}
en.wikipedia.org (parse) Naai_Kutty


['D. Imman']
127 | D. Imman
Naai_Kutty


en.wikipedia.org (imageinfo) File:Naai Kutty poster.jpg


['Vijayabharathi']
128 | Vijayabharathi


Naai Kutty (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Naai Kutt...
  infobox: <dict(15)> name, image, caption, director, producer, wr...
  pageid: 62269279
  parsetree: <str(18331)> <root><template><title>Use Indian Englis...
  requests: <list(2)> parse, imageinfo
  title: Naai Kutty
  wikibase: Q77895011
  wikidata_url: https://www.wikidata.org/wiki/Q77895011
  wikitext: <str(12609)> {{Use Indian English|date=November 2019}}...
}


In [96]:
for index,row in df.iterrows():
  mname=row['title']
  myear=row['year']
  if mname is not None:
    rating=getrating(mname,myear)
    print(mname+' | '+rating)
    df.at[index,'imdb']=rating.strip()

https://www.imdb.com/title/tt2009407/
A Aa E Ee | 5.7
https://www.imdb.com/title/tt1451763/
Villu | 3.3
Kadhalna Summa Illai | 
https://www.imdb.com/title/tt1956589/
Padikathavan | 4.9
https://www.imdb.com/title/tt2196091/
Vennila Kabadi Kuzhu | 7.3
https://www.imdb.com/name/nm1953187/
Gaja | 
https://www.imdb.com/title/tt8718580/
Satrumun Kidaitha Thagaval | 9.2
https://www.imdb.com/title/tt1379228/
Naan Kadavul | 7.7
https://www.imdb.com/title/tt0960886/
Pettarasu | 5.7
https://www.imdb.com/title/tt0250482/
Kudiyarasu | 
https://www.imdb.com/title/tt1846694/
Perumal | 2.8
https://www.imdb.com/title/tt1496872/
Siva Manasula Sakthi | 7.1
https://www.imdb.com/title/tt1435529/
Laadam | 5.9
https://www.imdb.com/title/tt6446698/
TN 07 AL 4777 | 7.4
https://www.imdb.com/title/tt0005372466/
Iru Nadhigal | 6.3
https://www.imdb.com/title/tt0364698/
Thee | 6.7
https://www.imdb.com/search/title/?explore=languages&title_type=feature&year=1977,1977&sort=user_rating,desc&languages=ta&ref_=adv_explo

In [98]:
df.to_csv('2009_films.csv')

In [99]:
df

year                        title                director  \
1    2009                    A Aa E Ee            D. Sabapathy   
2    2009                        Villu             Prabhu Deva   
3    2009         Kadhalna Summa Illai              Ilankannan   
4    2009                 Padikathavan                   Suraj   
5    2009         Vennila Kabadi Kuzhu            Suseenthiran   
6    2009                         Gaja               Ganapathy   
7    2009   Satrumun Kidaitha Thagaval         Bhuvanai Kannan   
8    2009                 Naan Kadavul                    Bala   
9    2009                    Pettarasu                Saravana   
10   2009                   Kudiyarasu           Sabir Hussain   
11   2009                      Perumal           Vincent Selva   
12   2009         Siva Manasula Sakthi               M. Rajesh   
13   2009                       Laadam           Prabu Solomon   
14   2009                TN 07 AL 4777       A. Lakshmikanthan   
15   2009                 Iru Nadhigal               Chanakyan   
16   2009                         Thee                G. Kicha   
17   2009                         1977      G. N. Dinesh Kumar   
18   2009                    Aarupadai  K. P. Sarvatheerthavan   
19   2009                    Innoruvan       S. T. Gunasekaran   
20   2009                Yavarum Nalam            Vikram Kumar   
21   2009                  Kanchivaram            Priyadarshan   
22   2009            Adada Enna Azhagu       T. M. Jayamurugan   
23   2009          Aadatha Aattamellam           A. B. Azhagar   
24   2009                   Nesikkiren             Nesamanavan   
25   2009                     Pattalam           Rohan Krishna   
26   2009                         Ayan             K. V. Anand   
27   2009              Ananda Tandavam          Gandhi Krishna   
28   2009               Naalai Namadhe                 Vinayan   
29   2009               Karthik Anitha                 Srihari   
30   2009            Naal Natchathiram         C. N. P. Sakthi   
..    ...                          ...                     ...   
99   2009                     Peranmai        S. P. Jananathan   
100  2009                     Aadhavan         K. S. Ravikumar   
101  2009              Kanden Kadhalai               R. Kannan   
102  2009         Adhe Neram Adhe Idam               M. Prabhu   
103  2009    Kanna Nee Enakku Thaanada               Olimaaran   
104  2009                      Karagam             Lakshmanraj   
105  2009              Palaivana Solai           K. S. Dayalan   
106  2009                Saa Boo Thiri             Arshad Khan   
107  2009               Thambivudayaan              Rajamahesh   
108  2009                      Vaidehi          Gemini Raghava   
109  2009  Swetha 5/10 Wellington Road                  Sanjay   
110  2009                  Thamizhagam         K. Suresh Kumar   
111  2009        Vizhiyil Malarnthathu              R. Sathish   
112  2009              Mathiya Chennai             Vivekanandh   
113  2009             Naan Avanillai 2                   Selva   
114  2009                         Yogi       Subramaniya Shiva   
115  2009             Edhuvum Nadakkum     Rozario, Maheswaran   
116  2009          Meendum Meendum Nee                           
117  2009                 Pinju Manasu               D. Jairam   
118  2009                    Renigunta        R. Panneerselvam   
119  2009                    Vettattam            D. S. Kannan   
120  2009                 Pachiyapuram              V. Agustin   
121  2009                       Thozhi         Elango Lakshman   
122  2009                       Vairam       S. P. Selva Durai   
123  2009                 Vettaikkaran            B. Babusivan   
124  2009                 Kandhakottai            S. Sakthivel   
125  2009                        Balam          Murali Krishna   
126  2009           Enakkul Oru Kadhal               M. Booman   
127  2009                    Odipolama       

# 2008

In [109]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_2008'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_2008=soup.find_all('table')[1:2]

In [110]:
standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

In [111]:
df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [113]:
for table in tables_2008:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='2008'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')-1

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')-1

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    if 'Production' in column_names:
      producer_index=column_names.index('Production')-1
      producer_member=True
    
    for datarow in rows:
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Opening', 'Title', 'Director', 'Cast', 'Production', 'Music Director', 'Genre']
1!Bheema|N. Linguswamy|Vikram, Trisha Krishnan|Action||/wiki/Bheema
2!Kaalai|Tarun Gopi|Silambarasan Rajendar, Vedhika Kumar|Action||/wiki/Kaalai
3!Pazhani|Perarasu|Bharath, Kajal Aggarwal, Kushboo|Action||/wiki/Pazhani_(2008_film)
4!Pirivom Santhippom|Karu Pazhaniappan|Cheran, Sneha|Family Drama||/wiki/Pirivom_Santhippom
5!Vaazhthugal|Seeman|R. Madhavan, Bhavana|Romance||/wiki/Vaazhthugal
6!Pidichirukku|Kanagu|Ashok, Vishakha Singh|Romance||/wiki/Pidichirukku
7!Pattaya Kelappu|Ponnambalam|Sriman, Payal, Ponnambalam|||/wiki/Pattaya_Kelappu
8!Ashoka|Prem Menon|Prem Menon, Anushree, Pooja Bharathi|Action||/wiki/Ashoka_(2008_film)
9!Indiralohathil Na Azhagappan|Thambi Ramaiah|Vadivelu, Yashimi Sharma, Suja Varunee|Comedy||/wiki/Indiralohathil_Na_Azhagappan
10!Thangam|G. Kicha|Sathyaraj, Meghna Nair, Goundamani|Action||/wiki/Thangam
11!Thodakkam|Muthukumar|Rishi, Abhinay, Raghu Vannan, Monica, Meghna Nair|Thr

In [114]:
df=df.drop(0)
df

year                         title               director  \
1    2008                        Bheema          N. Linguswamy   
2    2008                        Kaalai             Tarun Gopi   
3    2008                       Pazhani               Perarasu   
4    2008            Pirivom Santhippom      Karu Pazhaniappan   
5    2008                   Vaazhthugal                 Seeman   
6    2008                  Pidichirukku                 Kanagu   
7    2008               Pattaya Kelappu            Ponnambalam   
8    2008                        Ashoka             Prem Menon   
9    2008  Indiralohathil Na Azhagappan         Thambi Ramaiah   
10   2008                       Thangam               G. Kicha   
11   2008                     Thodakkam             Muthukumar   
12   2008                 Sadhu Miranda               Siddique   
13   2008                       Anjathe                Mysskin   
14   2008               Kadhal Kaditham                 Mukesh   
15   2008            Nenjathai Killadhe              Agathiyan   
16   2008                 Pachai Nirame                 Karkki   
17   2008                          Akku                 Mamani   
18   2008                      Thoondil        K. S. Adhiyaman   
19   2008               Valluvan Vasuki  Marumalarchi Bharathi   
20   2008               Sila Nerangalil                Jayaraj   
21   2008                        Thotta                  Selva   
22   2008                  Vambu Sandai             Raj Kapoor   
23   2008              Ezhuthiyatharadi  Sri Sathyaragaventher   
24   2008              Ponmagal Vandhal             Ramamurthy   
25   2008                  Velli Thirai                   Viji   
26   2008                     Theekuchi              Yuvarajan   
27   2008                      Unakkaga              Manoharan   
28   2008                 Vaitheeswaran      R. K. Vidhyadaran   
29   2008                          Inba                Vendhan   
30   2008                 Kannum Kannum           G. Marimuthu   
..    ...                           ...                    ...   
91   2008           Kadhalil Vizhunthen           P. V. Prasad   
92   2008                  Sakkarakatti            Kala Prabhu   
93   2008                         Durai           A. Venkatesh   
94   2008               Jai Vigneshwara         James Clifford   
95   2008             Kasimedu Govindan          Ramesh Sanjay   
96   2008                  Kathi Kappal        Dinesh Selvaraj   
97   2008            Netru Indru Naalai    Lakshmikanth Chenna   
98   2008                         Seval                   Hari   
99   2008                         Megam          Jeevarathinam   
100  2008                         Aegan          Raju Sundaram   
101  2008                        Poochi                   Velu   
102  2008                     Theeyavan              B. Kathir   
103  2008                         Velvi                  Jeans   
104  2008              Vaaranam Aayiram          Gautham Menon   
105  2008                    Thenavattu           V. V. Kathir   
106  2008              Ellam Avan Seyal           Shaji Kailas   
107  2008                   Kodaikkanal             T. K. Bose   
108  2008  Mahesh, Saranya Matrum Palar             P. V. Ravi   
109  2008                       Nadigai            Babu Ganesh   
110  2008                           Poo                   Sasi   
111  2008        Utharavindri Ullay Vaa                Vijayan   
112  2008         Pudhusu Kanna Pudhusu        A. R. Venkatesh   
113  2008                       Saamida           Vadivudayaan   
114  2008                        Suryaa         Jaguar Thangam   
115  2008                   Bommalattam           Bharathiraja   
116  2008                   Silambattam              Saravanan   
117  2008              Dindigul Sarathy     Subramaniam Pillai   
118  2008               Thiruvannamalai               Perarasu   
119  2008                Abhiyum Naanum      

In [115]:
df['cast_size']=''
for index,row in df.iterrows():
    mlink=row['cast']
    if mlink is not None:
        arr=str(mlink).split(',')
        df.at[index,'cast_size']=len(arr)

for index,row in df.iterrows():
  mlink=row['link']
  if mlink is not None:
    print(str(mlink).strip()[6:])
    dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
    strdir=tostr(dirs)
    print(str(index)+' | '+strdir)
    df.at[index,'music']=strdir


Bheema


en.wikipedia.org (parse) Bheema
en.wikipedia.org (imageinfo) File:Vintage Bhim oleograph litho Ra...
Bhima (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Vintage B...
  infobox: <dict(11)> name, series, image, alt, caption, spouse, c...
  iwlinks: <list(1)> https://commons.wikimedia.org/wiki/Category:Bhima
  pageid: 100588
  parsetree: <str(32291)> <root><template><title>about</title><par...
  requests: <list(2)> parse, imageinfo
  title: Bhima
  wikibase: Q3651886
  wikidata_url: https://www.wikidata.org/wiki/Q3651886
  wikitext: <str(27795)> {{about|a mighty warrior in the Mahabhara...
}
en.wikipedia.org (parse) Kaalai


1 | 
Kaalai


en.wikipedia.org (imageinfo) File:Kaalai Simbu poster.jpg
Kaalai (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Kaalai Si...
  infobox: <dict(17)> name, image, director, writer, screenplay, s...
  pageid: 7440211
  parsetree: <str(14266)> <root><template><title>Use dmy dates</ti...
  requests: <list(2)> parse, imageinfo
  title: Kaalai
  wikibase: Q6343330
  wikidata_url: https://www.wikidata.org/wiki/Q6343330
  wikitext: <str(10238)> {{Use dmy dates|date=December 2015}}{{Use...
}
en.wikipedia.org (parse) Pazhani_(2008_film)


['G. V. Prakash Kumar']
2 | G. V. Prakash Kumar
Pazhani_(2008_film)


...

en.wikipedia.org (imageinfo) File:Thiruvannamalai film poster.jpg
Thiruvannamalai (film) (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Thiruvann...
  infobox: <dict(15)> name, image, caption, director, writer, star...
  pageid: 18935404
  parsetree: <str(13727)> <root><template><title>Use dmy dates</ti...
  requests: <list(2)> parse, imageinfo
  title: Thiruvannamalai (film)
  wikibase: Q22315057
  wikidata_url: https://www.wikidata.org/wiki/Q22315057
  wikitext: <str(8149)> {{Use dmy dates|date=October 2015}}{{Use I...
}
en.wikipedia.org (parse) Abhiyum_Naanum


['Srikanth Deva']
118 | Srikanth Deva
Abhiyum_Naanum


en.wikipedia.org (imageinfo) File:Abhiyum naanum landscape.jpg
Abhiyum Naanum (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Abhiyum n...
  infobox: <dict(16)> name, image, caption, director, producer, wr...
  pageid: 11964824
  parsetree: <str(16082)> <root><template><title>About</title><par...
  requests: <list(2)> parse, imageinfo
  title: Abhiyum Naanum
  wikibase: Q4667488
  wikidata_url: https://www.wikidata.org/wiki/Q4667488
  wikitext: <str(11510)> {{About||the 2013 Malayalam film|Abhiyum ...
}
en.wikipedia.org (parse) Panchamirtham


['Vidyasagar', 'Vidyasagar']
119 | Vidyasagar,Vidyasagar
Panchamirtham
['Sundar C Babu']
120 | Sundar C Babu


Panchamirtham (en) data
{
  infobox: <dict(11)> name, director, producer, writer, starring, ...
  pageid: 20837828
  parsetree: <str(11867)> <root><template><title>about</title><par...
  requests: <list(1)> parse
  title: Panchamirtham
  wikibase: Q7130268
  wikidata_url: https://www.wikidata.org/wiki/Q7130268
  wikitext: <str(7828)> {{about|a film|the food mix used in puja|P...
}


In [116]:
for index,row in df.iterrows():
  mname=row['title']
  myear=row['year']
  if mname is not None:
    rating=getrating(mname,myear)
    print(mname+' | '+rating)
    df.at[index,'imdb']=rating.strip()

https://www.imdb.com/title/tt1034431/
Bheema | 5.7
https://www.imdb.com/title/tt3522722/
Kaalai | 4.3
https://www.imdb.com/title/tt2063749/
Pazhani | 3.4
https://www.imdb.com/title/tt2084920/
Pirivom Santhippom | 6.2
https://www.imdb.com/title/tt1509832/
Vaazhthugal | 4.5
https://www.imdb.com/list/ls022348096/
Pidichirukku | 
https://www.imdb.com/name/nm4776159/
Pattaya Kelappu | 
https://www.imdb.com/title/tt1273218/characters/nm0707425
Ashoka | 
https://www.imdb.com/title/tt6449478/
Indiralohathil Na Azhagappan | 7.0
https://www.imdb.com/title/tt1303809/
Thangam | 
https://www.imdb.com/title/tt8209234/
Thodakkam | 4.4
https://www.imdb.com/title/tt6450174/
Sadhu Miranda | 6.8
https://www.imdb.com/title/tt1922545/
Anjathe | 8.1
https://www.imdb.com/title/tt0453740/
Kadhal Kaditham | 7.5
https://www.imdb.com/title/tt0235617/
Nenjathai Killadhe | 7.3
https://www.imdb.com/title/tt0242256/reviews
Pachai Nirame | 
https://www.imdb.com/title/tt3194988/
Akku | 
https://www.imdb.com/title/tt16

In [117]:
df.to_csv('2008_films.csv')

In [118]:
df

year                         title               director  \
1    2008                        Bheema          N. Linguswamy   
2    2008                        Kaalai             Tarun Gopi   
3    2008                       Pazhani               Perarasu   
4    2008            Pirivom Santhippom      Karu Pazhaniappan   
5    2008                   Vaazhthugal                 Seeman   
6    2008                  Pidichirukku                 Kanagu   
7    2008               Pattaya Kelappu            Ponnambalam   
8    2008                        Ashoka             Prem Menon   
9    2008  Indiralohathil Na Azhagappan         Thambi Ramaiah   
10   2008                       Thangam               G. Kicha   
11   2008                     Thodakkam             Muthukumar   
12   2008                 Sadhu Miranda               Siddique   
13   2008                       Anjathe                Mysskin   
14   2008               Kadhal Kaditham                 Mukesh   
15   2008            Nenjathai Killadhe              Agathiyan   
16   2008                 Pachai Nirame                 Karkki   
17   2008                          Akku                 Mamani   
18   2008                      Thoondil        K. S. Adhiyaman   
19   2008               Valluvan Vasuki  Marumalarchi Bharathi   
20   2008               Sila Nerangalil                Jayaraj   
21   2008                        Thotta                  Selva   
22   2008                  Vambu Sandai             Raj Kapoor   
23   2008              Ezhuthiyatharadi  Sri Sathyaragaventher   
24   2008              Ponmagal Vandhal             Ramamurthy   
25   2008                  Velli Thirai                   Viji   
26   2008                     Theekuchi              Yuvarajan   
27   2008                      Unakkaga              Manoharan   
28   2008                 Vaitheeswaran      R. K. Vidhyadaran   
29   2008                          Inba                Vendhan   
30   2008                 Kannum Kannum           G. Marimuthu   
..    ...                           ...                    ...   
91   2008           Kadhalil Vizhunthen           P. V. Prasad   
92   2008                  Sakkarakatti            Kala Prabhu   
93   2008                         Durai           A. Venkatesh   
94   2008               Jai Vigneshwara         James Clifford   
95   2008             Kasimedu Govindan          Ramesh Sanjay   
96   2008                  Kathi Kappal        Dinesh Selvaraj   
97   2008            Netru Indru Naalai    Lakshmikanth Chenna   
98   2008                         Seval                   Hari   
99   2008                         Megam          Jeevarathinam   
100  2008                         Aegan          Raju Sundaram   
101  2008                        Poochi                   Velu   
102  2008                     Theeyavan              B. Kathir   
103  2008                         Velvi                  Jeans   
104  2008              Vaaranam Aayiram          Gautham Menon   
105  2008                    Thenavattu           V. V. Kathir   
106  2008              Ellam Avan Seyal           Shaji Kailas   
107  2008                   Kodaikkanal             T. K. Bose   
108  2008  Mahesh, Saranya Matrum Palar             P. V. Ravi   
109  2008                       Nadigai            Babu Ganesh   
110  2008                           Poo                   Sasi   
111  2008        Utharavindri Ullay Vaa                Vijayan   
112  2008         Pudhusu Kanna Pudhusu        A. R. Venkatesh   
113  2008                       Saamida           Vadivudayaan   
114  2008                        Suryaa         Jaguar Thangam   
115  2008                   Bommalattam           Bharathiraja   
116  2008                   Silambattam              Saravanan   
117  2008              Dindigul Sarathy     Subramaniam Pillai   
118  2008               Thiruvannamalai               Perarasu   
119  2008                Abhiyum Naanum      

# 2007

In [127]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_2007'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_2007=soup.find_all('table')[0:1]

In [128]:
standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

In [129]:
df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [131]:
for table in tables_2007:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='2007'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')-1

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')-1

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    if 'Production' in column_names:
      producer_index=column_names.index('Production')-1
      producer_member=True
    
    for datarow in rows:
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Opening', 'Title', 'Director', 'Cast', 'Production', 'Music director', 'Genre']
1!Kalakkura Chandru|Ravi-Raja|Karthik, Bhuvana, Pandiarajan, Urvashi|Comedy||/wiki/Kalakkura_Chandru
2!Aalwar|Chella|Ajith Kumar, Asin|Action||/wiki/Aalwar
3!Pokkiri|Prabhu Deva|Vijay, Asin, Prakash Raj, Vadivelu|Action||/wiki/Pokkiri
4!Thaamirabharani|Hari|Vishal, Bhanu, Nadhiya, Prabhu|Action||/wiki/Thaamirabharani
5!Veerasamy|T. Rajender|T. Rajender, Mumtaj, Meghna Naidu|Drama||/wiki/Veerasamy
6!Deepavali|Ezhil|Jayam Ravi, Bhavana|Romance||/wiki/Deepavali_(2007_film)
7!Pori|Subramaniya Siva|Jiiva, Pooja|Action||/wiki/Pori_(film)
8!Agaram|T. Nagarajan|Nandha, Archana|Drama||/wiki/Agaram_(film)
9!Lee|Prabu Solomon|Sibiraj, Nila, Prakash Raj|Action Thriller||/wiki/Lee_(2007_film)
10!Pachaikili Muthucharam|Gautham Menon|Sarath Kumar, Jyothika, Andrea Jeremiah|Thriller||/wiki/Pachaikili_Muthucharam
11!Mozhi|Radha Mohan|Prithviraj, Jyothika, Prakash Raj, Swarnamalya|Romance, Comedy||/wiki/Mozhi_(film)
12!Par

In [135]:
df=df.drop(0)
df

year                         title             director  \
1    2007             Kalakkura Chandru            Ravi-Raja   
2    2007                        Aalwar               Chella   
3    2007                       Pokkiri          Prabhu Deva   
4    2007               Thaamirabharani                 Hari   
5    2007                     Veerasamy          T. Rajender   
6    2007                     Deepavali                Ezhil   
7    2007                          Pori     Subramaniya Siva   
8    2007                        Agaram         T. Nagarajan   
9    2007                           Lee        Prabu Solomon   
10   2007        Pachaikili Muthucharam        Gautham Menon   
11   2007                         Mozhi          Radha Mohan   
12   2007                 Paruthiveeran                Ameer   
13   2007                       Murugaa          R. T. Nesan   
14   2007                          Muni     Raghava Lawrence   
15   2007                    Thirumagan       M. Rathnakumar   
16   2007              Nanbanin Kadhali             G. Kicha   
17   2007                        Sabari               Suresh   
18   2007              Kuttrapathirikai      R. K. Selvamani   
19   2007                     Manikanda                Selva   
20   2007                       Adavadi  V. S. Bharath Hanna   
21   2007                  Koodal Nagar       Seenu Ramasamy   
22   2007                         Kuppi      A. M. R. Ramesh   
23   2007                   Kaanal Neer       Chinni Jayanth   
24   2007                  Maya Kannadi               Cheran   
25   2007                 Unnale Unnale                Jeeva   
26   2007                Madurai Veeran        Vincent Selva   
27   2007                 Mudhal Kanave       C. Balamurugan   
28   2007                Naan Avanillai                Selva   
29   2007                Chennai 600028        Venkat Prabhu   
30   2007               Dhandayuthapani      Saravana Sakthi   
..    ...                           ...                  ...   
75   2007          Meendum Chandramathi             Kamalraj   
76   2007               Mudhal Mudhalai             Benjamin   
77   2007                         Vegam  K. R. Udhayashankar   
78   2007             Kattradhu Thamizh                  Ram   
79   2007  Pasupathi c/o Rasakkapalayam    K. Selva Bharathy   
80   2007                     Sringaram   Saradha Ramanathan   
81   2007                        Thavam      Sakthi Paramesh   
82   2007              Veeramum Eeramum           Sanjay Ram   
83   2007     Naalaiya Pozhuthum Unnodu    K. Moorthy Kannan   
84   2007             Niyabagam Varuthe            K. S. Raj   
85   2007           Ippadikku En Kadhal           B. Kishore   
86   2007                 Chandiramathi       V. S. Kamalraj   
87   2007                          Agra      Chithrai Selvan   
88   2007          Azhagiya Tamil Magan            Bharathan   
89   2007            Kannamoochi Yenada             V. Priya   
90   2007                   Machakaaran          Tamilvannan   
91   2007                   Polladhavan          Vetrimaaran   
92   2007                           Vel                 Hari   
93   2007                     Kelvikuri              Jailani   
94   2007          Onbadhu Roobai Nottu       Thangar Bachan   
95   2007                       Oram Po     Pushkar-Gayathri   
96   2007                    Rameswaram            S. Selvam   
97   2007                  Evano Oruvan      Nishikant Kamat   
98   2007                       Kallori    Balaji Shakthivel   
99   2007                         Billa        Vishnuvardhan   
100  2007                       Mirugam                 Samy   
101  2007                         Kanna                Anand   
102  2007                  Puli Varudhu          G. V. Kumar   
103  2007          Pazhaniappa Kalloori             R. Pavan   
104  2007                 Yaaruku Yaaro          Joe Stanley   

                                    

In [136]:
df['cast_size']=''
for index,row in df.iterrows():
    mlink=row['cast']
    if mlink is not None:
        arr=str(mlink).split(',')
        df.at[index,'cast_size']=len(arr)

for index,row in df.iterrows():
  mlink=row['link']
  if mlink is not None:
    print(str(mlink).strip()[6:])
    dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
    strdir=tostr(dirs)
    print(str(index)+' | '+strdir)
    df.at[index,'music']=strdir


Kalakkura_Chandru


en.wikipedia.org (parse) Kalakkura_Chandru
Kalakkura Chandru (en) data
{
  infobox: <dict(12)> name, director, writer, starring, producer, ...
  pageid: 15677743
  parsetree: <str(4043)> <root><template><title>Use dmy dates</tit...
  requests: <list(1)> parse
  title: Kalakkura Chandru
  wikibase: Q6350263
  wikidata_url: https://www.wikidata.org/wiki/Q6350263
  wikitext: <str(2252)> {{Use dmy dates|date=December 2015}}{{Use ...
}
en.wikipedia.org (parse) Aalwar


['Sunil Xavier']
1 | Sunil Xavier
Aalwar


en.wikipedia.org (imageinfo) File:Aalwar3.jpg
Aalwar (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Aalwar3.j...
  infobox: <dict(14)> name, image, director, writer, starring, pro...
  pageid: 5377233
  parsetree: <str(16285)> <root><template><title>for</title><part>...
  requests: <list(2)> parse, imageinfo
  title: Aalwar
  wikibase: Q3524853
  wikidata_url: https://www.wikidata.org/wiki/Q3524853
  wikitext: <str(9204)> {{for|the Hindu saints|Alwars}}{{short des...
}
en.wikipedia.org (parse) Pokkiri


['Srikanth Deva']
2 | Srikanth Deva
Pokkiri


...

Puli Varudhu (en) data
{
  infobox: <dict(11)> name, director, starring, producer, studio, ...
  pageid: 15662770
  parsetree: <str(8971)> <root><template><title>Use Indian English...
  requests: <list(1)> parse
  title: Puli Varudhu
  wikibase: Q22315103
  wikidata_url: https://www.wikidata.org/wiki/Q22315103
  wikitext: <str(6747)> {{Use Indian English|date=February 2016}}{...
}
en.wikipedia.org (parse) Pazhaniappa Kalloori


['Srikanth Deva']
102 | Srikanth Deva



API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Yaaruku_Yaaro


Exception
103 | 
Yaaruku_Yaaro
['Joe Stanley']
104 | Joe Stanley


Yaaruku Yaaro (en) data
{
  infobox: <dict(11)> name, director, writer, cinematography, edit...
  pageid: 23991832
  parsetree: <str(11272)> <root><template><title>Multiple issues</...
  requests: <list(1)> parse
  title: Yaaruku Yaaro
  wikibase: Q12988263
  wikidata_url: https://www.wikidata.org/wiki/Q12988263
  wikitext: <str(9425)> {{Multiple issues|{{tone|date=November 201...
}


In [137]:
for index,row in df.iterrows():
  mname=row['title']
  myear=row['year']
  if mname is not None:
    rating=getrating(mname,myear)
    print(mname+' | '+rating)
    df.at[index,'imdb']=rating.strip()

https://www.imdb.com/title/tt12139242/
Kalakkura Chandru | 
https://www.imdb.com/title/tt1810522/
Aalwar | 3.1
https://www.imdb.com/title/tt1451409/
Pokkiri | 7.2
https://www.imdb.com/title/tt1503167/
Thaamirabharani | 5.8
https://www.imdb.com/title/tt10168650/
Veerasamy | 7.4
https://www.imdb.com/title/tt1459034/
Deepavali | 5.7
https://www.imdb.com/title/tt5721586/
Pori | 3.6
https://www.imdb.com/title/tt8479614/
Agaram | 
https://www.imdb.com/title/tt8263552/
Lee | 6.4
https://www.imdb.com/title/tt0955313/
Pachaikili Muthucharam | 6.2
https://www.imdb.com/title/tt1034053/
Mozhi | 8.0
https://www.imdb.com/title/tt0458050/
Paruthiveeran | 8.2
Murugaa | 
https://www.imdb.com/title/tt4102346/
Muni | 6.3
https://www.imdb.com/title/tt10327074/
Thirumagan | 3.3
https://www.imdb.com/title/tt8263552/
Nanbanin Kadhali | 6.4
https://www.imdb.com/title/tt0861733/
Sabari | 4.3
https://www.imdb.com/title/tt8479614/
Kuttrapathirikai | 
https://www.imdb.com/title/tt1821536/
Manikanda | 4.0
https://

In [139]:
df.to_csv('2007_films.csv')

# 2006

In [17]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_2006'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_2006=soup.find_all('table')[0:4]

In [18]:
standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

In [19]:
df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [20]:
for table in tables_2006:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='2006'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')-1

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')-1

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    if 'Production' in column_names:
      producer_index=column_names.index('Production')-1
      producer_member=True
    
    for datarow in rows:
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Opening', 'Title', 'Director', 'Cast', 'Production', 'Ref', 'Genre']
1!Paramasivan|P. Vasu|Ajith Kumar, Laila, Jayaram|Action||/wiki/Paramasivan
2!Saravana|K. S. Ravikumar|Silambarasan, Jyothika, Prakash Raj|Action||/wiki/Saravana
3!Aathi|Ramana|Vijay, Trisha|Action||/wiki/Aathi
4!Pasa Kiligal|Amirtham|Prabhu, Murali, Vineeth, Navya Nair, Malavika, Roja|Drama||/wiki/Pasa_Kiligal
5!Dishyum|Sasi|Jiiva, Sandhya|Romance||/wiki/Dishyum
6!Idhaya Thirudan|Saran|Jayam Ravi, Kamna Jethmalani, Prakash Raj|Romance||/wiki/Idhaya_Thirudan
7!Amirtham|K. Kannan|Ganesh, Navya Nair|Drama||/wiki/Amirtham
8!Chithiram Pesuthadi|Mysskin|Narain, Bhavana|Action||/wiki/Chithiram_Pesuthadi
9!June R|Revathy S. Varma|Jyothika, Kushboo, Saritha|Drama||/wiki/June_R
10!Kodambakkam|K. P. Jagan|Nandha, Diya, Tejashree|Drama||/wiki/Kodambakkam_(film)
11!Kalabha Kadhalan|Igore|Arya, Renuka Menon, Akshaya|Romance||/wiki/Kalabha_Kadhalan
12!Kalvanin Kadhali|Thamilvaanan|S. J. Surya, Nayantara|Romance||/wiki/Kalvanin_Ka

92!Kasu|Gowri Manohar|Rahman, Sangeetha, Rajashree|null|Palavodai Amman Creations|/wiki/Kasu_(film)


In [21]:
df=df.drop(0)
df

year                                  title             director  \
1   2006                            Paramasivan              P. Vasu   
2   2006                               Saravana      K. S. Ravikumar   
3   2006                                  Aathi               Ramana   
4   2006                           Pasa Kiligal             Amirtham   
5   2006                                Dishyum                 Sasi   
6   2006                        Idhaya Thirudan                Saran   
7   2006                               Amirtham            K. Kannan   
8   2006                    Chithiram Pesuthadi              Mysskin   
9   2006                                 June R     Revathy S. Varma   
10  2006                            Kodambakkam          K. P. Jagan   
11  2006                       Kalabha Kadhalan                Igore   
12  2006                       Kalvanin Kadhali         Thamilvaanan   
13  2006                               Madarasi                Arjun   
14  2006                       Nagareega Komali       Ramji S. Balan   
15  2006                                 Thambi               Seeman   
16  2006                         Kovai Brothers   Sakthi Chidambaram   
17  2006                     Oru Kadhal Seiveer       Ravi Bhargavan   
18  2006                                 Sudesi          A. Y. Jeppy   
19  2006                        Mercury Pookkal        S. S. Stanley   
20  2006                               Pattiyal        Vishnuvardhan   
21  2006                        Theenda Theenda          A. P. Muhan   
22  2006  Azhagai Irukkirai Bayamai Irukkirathu         Vijay Milton   
23  2006                       Pachchak Kuthira            Parthiban   
24  2006                             Thirupathi             Perarasu   
25  2006                        Thiruttu Payale         Susi Ganesan   
26  2006                          Suyetchai MLA        Guru Dhanapal   
27  2006                                  Kokki        Prabu Solomon   
28  2006                                  Madhu        K. Thennarasu   
29  2006                                  Jerry       S. B. Khanthan   
30  2006                         Thalai Nagaram                Suraj   
..   ...                                    ...                  ...   
63  2006                               Sengathu         Indhu Nathan   
64  2006                 Maranthen Meimaranthen            Sivaraman   
65  2006                             Dharmapuri             Perarasu   
66  2006                                      E     S. P. Jananathan   
67  2006                               Varalaru      K. S. Ravikumar   
68  2006                            Thalaimagan         Sarath Kumar   
69  2006                               Vallavan         Silambarasan   
70  2006                               Vattaram                Saran   
71  2006              Kizhakku Kadalkarai Salai        S. S. Stanley   
72  2006                               Vathiyar         A. Venkatesh   
73  2006                         Aavani Thingal          Harikrishna   
74  2006           Prathi Gnayiru 9.30 to 10.00              M. Anbu   
75  2006                                  Rendu            Sundar C.   
76  2006                        Sivappathigaram    Karu Pazhaniappan   
77  2006                          Iruvar Mattum     Dwaraki Raghavan   
78  2006              Idhu Kadhal Varum Paruvam        Kasthuri Raja   
79  2006                         Nenjil Jil Jil                Selva   
80  2006                         Chennai Kadhal             Vikraman   
81  2006                        Aadu Puli Aatam           Sanjay Ram   
82  2006                    Ganapathy Vanthachi           P. C. Guna   
83  2006                      Nenjirukkum Varai  S. A. Chandrasekhar   
84  2006              Thiruvilaiyaadal Aarambam     Boopathy Pandian   
85  2006                         Azhagiya Asura                 Nagu   
86  2006                              Ilakka

In [22]:
df['cast_size']=''
for index,row in df.iterrows():
    mlink=row['cast']
    if mlink is not None:
        arr=str(mlink).split(',')
        df.at[index,'cast_size']=len(arr)

for index,row in df.iterrows():
  mlink=row['link']
  if mlink is not None:
    print(str(mlink).strip()[6:])
    dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
    strdir=tostr(dirs)
    print(str(index)+' | '+strdir)
    df.at[index,'music']=strdir


Paramasivan


en.wikipedia.org (parse) Paramasivan
en.wikipedia.org (imageinfo) File:Paramasivan.JPG
Paramasivan (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Paramasiv...
  infobox: <dict(15)> name, image, director, producer, writer, sto...
  pageid: 4662218
  parsetree: <str(12849)> <root><template><title>Use dmy dates</ti...
  requests: <list(2)> parse, imageinfo
  title: Paramasivan
  wikibase: Q7135149
  wikidata_url: https://www.wikidata.org/wiki/Q7135149
  wikitext: <str(8784)> {{Use dmy dates|date=June 2014}}{{Infobox ...
}
en.wikipedia.org (parse) Saravana


['Vidyasagar', 'Vidyasagar']
1 | Vidyasagar,Vidyasagar
Saravana


en.wikipedia.org (imageinfo) File:Saravana poster.jpg
Saravana (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Saravana ...
  infobox: <dict(13)> name, image, director, writer, story, starri...
  pageid: 4103051
  parsetree: <str(11594)> <root><template><title>Use dmy dates</ti...
  requests: <list(2)> parse, imageinfo
  title: Saravana
  wikibase: Q7423406
  wikidata_url: https://www.wikidata.org/wiki/Q7423406
  wikitext: <str(8917)> {{Use dmy dates|date=November 2015}}{{Use ...
}
en.wikipedia.org (parse) Aathi


['Srikanth Deva']
2 | Srikanth Deva
Aathi


...

Thagapansamy (en) data
{
  image: <list(0)> 
  infobox: <dict(13)> name, image, director, writer, starring, pro...
  pageid: 5562511
  parsetree: <str(12478)> <root><template><title>Use dmy dates</ti...
  requests: <list(1)> parse
  title: Thagapansamy
  wikibase: Q7709065
  wikidata_url: https://www.wikidata.org/wiki/Q7709065
  wikitext: <str(8585)> {{Use dmy dates|date=November 2015}}{{Use ...
}
en.wikipedia.org (parse) Adaikalam


['Srikanth Deva']
90 | Srikanth Deva
Adaikalam


Adaikalam (en) data
{
  image: <list(0)> 
  infobox: <dict(15)> name, image, caption, writer, starring, dire...
  pageid: 7806593
  parsetree: <str(11533)> <root><template><title>Use dmy dates</ti...
  requests: <list(1)> parse
  title: Adaikalam
  wikibase: Q4678406
  wikidata_url: https://www.wikidata.org/wiki/Q4678406
  wikitext: <str(7997)> {{Use dmy dates|date=February 2017}}{{Use ...
}
en.wikipedia.org (parse) Kasu_(film)


['Sabesh', 'Murali']
91 | Sabesh,Murali
Kasu_(film)
['V. S. Udhaya']
92 | V. S. Udhaya


Kasu (film) (en) data
{
  infobox: <dict(13)> name, director, producer, writer, starring, ...
  pageid: 62435334
  parsetree: <str(13636)> <root><template><title>Use Indian Englis...
  requests: <list(1)> parse
  title: Kasu (film)
  wikitext: <str(8114)> {{Use Indian English|date=July 2020}}{{Use...
}


In [23]:
for index,row in df.iterrows():
  mname=row['title']
  myear=row['year']
  if mname is not None:
    rating=getrating(mname,myear)
    print(mname+' | '+rating)
    df.at[index,'imdb']=rating.strip()

https://www.imdb.com/title/tt0816605/
Paramasivan | 4.3
https://www.imdb.com/title/tt0495044/
Saravana | 5.0
https://www.imdb.com/title/tt0769499/
Aathi | 3.8
https://www.imdb.com/title/tt1054636/
Pasa Kiligal | 4.6
https://www.imdb.com/title/tt1365448/
Dishyum | 5.3
https://www.imdb.com/title/tt0455545/
Idhaya Thirudan | 3.3
https://www.imdb.com/name/nm2692507/
Amirtham | 
https://www.imdb.com/title/tt2325589/
Chithiram Pesuthadi | 7.1
https://www.imdb.com/title/tt0475632/
June R | 5.0
https://www.imdb.com/title/tt6765436/
Kodambakkam | 
https://www.imdb.com/title/tt2325785/
Kalabha Kadhalan | 5.0
https://www.imdb.com/title/tt0459424/
Kalvanin Kadhali | 4.3
https://www.imdb.com/title/tt4483434/
Madarasi | 4.6
https://www.imdb.com/title/tt10711648/
Nagareega Komali | 6.9
https://www.imdb.com/title/tt0458176/
Thambi | 6.0
https://www.imdb.com/title/tt2325799/
Kovai Brothers | 3.4
https://www.imdb.com/title/tt0464071/
Oru Kadhal Seiveer | 6.9
https://www.imdb.com/title/tt0824704/
Sudesi 

In [24]:
df.to_csv('2006_movies.csv')

# 2005

In [27]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_2005'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_2005=soup.find_all('table')[1:2]

In [23]:
standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

In [24]:
df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [28]:
for table in tables_2005:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='2005'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')-1

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')-1

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    if 'Production' in column_names:
      producer_index=column_names.index('Production')-1
      producer_member=True
    
    for datarow in rows:
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Title', 'Director', 'Cast', 'Music director']
90!Ambuttu Imbuttu Embuttu|Ashok Kashyap|Ashok Kashyap, Ashwini, Mohana, Tejaswini Prakash, Crane Manohar|null|null|/wiki/Ambuttu_Imbuttu_Embuttu
91!Unnai Enakku Pidichuruku|Anbu Saravanan|Aswin, Aswini, Naveen, Munna|null|null|/w/index.php?title=Unnai_Enakku_Pidichuruku&action=edit&redlink=1
92!Iyyappa Saamy|Ravi-Raja|Pandiarajan, Ishaq Hussaini, Preethi Varma, Lavanya|null|null|/w/index.php?title=Iyyappa_Saamy&action=edit&redlink=1
93!Sendhalam Poove|A. C. Deepa Raj|Livingston, Devayani|null|null|/w/index.php?title=Sendhalam_Poove&action=edit&redlink=1
94!Thiru Thiru||Pandiarajan, Nanditha Jennifer|null|null|/w/index.php?title=Thiru_Thiru&action=edit&redlink=1
95!Plus Koottani|Babu Ganesh|Babu Ganesh, Manthra|null|null|/w/index.php?title=Plus_Koottani&action=edit&redlink=1
96!Pillayar Pettaiyum Kandhu Vattiyum|||null|null|/w/index.php?title=Pillayar_Pettaiyum_Kandhu_Vattiyum&action=edit&redlink=1
97!Pen Nila|||null|null|/w/index.php?tit

In [30]:
df=df.drop(0)
df

year                               title                   director  \
1    2005                              Dancer                      Keyar   
2    2005                            Aayudham             M. A. Murugesh   
3    2005                                Ayya                       Hari   
4    2005                  Devathaiyai Kanden           Boopathy Pandian   
5    2005                         Thirupaachi                   Perarasu   
6    2005                            Iyer IPS                 Hari Rajan   
7    2005                      Kannadi Pookal               K. Shahjahan   
8    2005                             Ayodhya             R. Jayaprakash   
9    2005                            Kannamma          S. S. Baba Vikram   
10   2005                                  Ji              N. Linguswamy   
11   2005                       Ulla Kadathal      Pugazhendhi Thangaraj   
12   2005                              Sukran        S. A. Chandrasekhar   
13   2005                Kadhal Seiya Virumbu             Ravi Bhargavan   
14   2005                     Mannin Maindhan             Rama Narayanan   
15   2005                                Raam                      Ameer   
16   2005                             Maayavi                 Singampuli   
17   2005                              London                  Sundar C.   
18   2005                      Karagattakkari            Bharathi Kannan   
19   2005                     Kicha Vayasu 16            A. N. Rajagopal   
20   2005                     Rightaa Thappaa                 R. Bhuvana   
21   2005                            Gurudeva                     Jaffer   
22   2005                     Thaka Thimi Tha                  Sundar C.   
23   2005                              Sevvel                    Primose   
24   2005                        Chandramukhi                    P. Vasu   
25   2005                          Priyasakhi            K. S. Adhiyaman   
26   2005                             Sachein             John Mahendran   
27   2005                      Mumbai Express  Singeetham Sreenivasa Rao   
28   2005                              Jithan              Vincent Selva   
29   2005                      Karka Kasadara           R. V. Udayakumar   
30   2005                                6'2"           V. Senthil Kumar   
..    ...                                 ...                        ...   
74   2005                               Majaa                      Shafi   
75   2005                         Saadhuriyan                D. J. Kumar   
76   2005                            Anbe Vaa          K. Selva Bharathy   
77   2005                    Bambara Kannaley              Parthy Baskar   
78   2005                      Uyir Ullavarai             S. S. Bharathy   
79   2005                   Kanda Naal Mudhal                   V. Priya   
80   2005                               Aanai                      Selva   
81   2005              Thavamai Thavamirundhu                     Cheran   
82   2005                                Aaru                       Hari   
83   2005                  Vetrivel Sakthivel             Lakshmi Priyan   
84   2005                          Sandakozhi               N. Lingusamy   
85   2005                            Veeranna                  Kalanidhi   
86   2005                   Vanakkam Thalaiva            Sakthi Paramesh   
87   2005                          Sollattuma               Selvaraj.K.S   
88   2005        Sorry Enaku Kalyanamayidichu                      Sagar   
89   2005                          Unarchigal                   K. Rajan   
90   2005             Ambuttu Imbuttu Embuttu              Ashok Kashyap   
91   2005            Unnai Enakku Pidichuruku             Anbu Saravanan   
92   2005                       Iyyappa Saamy                  Ravi-Raja   
93   2005                     Sendhalam Poove            A. C. Deepa Raj   
94   2005                         Thiru Thiru        

In [31]:
df['cast_size']=''
for index,row in df.iterrows():
    mlink=row['cast']
    if mlink is not None:
        arr=str(mlink).split(',')
        df.at[index,'cast_size']=len(arr)

for index,row in df.iterrows():
  mlink=row['link']
  if mlink is not None:
    print(str(mlink).strip()[6:])
    dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
    strdir=tostr(dirs)
    print(str(index)+' | '+strdir)
    df.at[index,'music']=strdir


Dancer_(2005_film)


en.wikipedia.org (parse) Dancer_(2005_film)
Dancer (2005 film) (en) data
{
  infobox: <dict(13)> name, director, writer, starring, editing, c...
  pageid: 16179543
  parsetree: <str(5761)> <root><template><title>Use dmy dates</tit...
  requests: <list(1)> parse
  title: Dancer (2005 film)
  wikibase: Q5215391
  wikidata_url: https://www.wikidata.org/wiki/Q5215391
  wikitext: <str(2970)> {{Use dmy dates|date=October 2015}}{{Use I...
}
en.wikipedia.org (parse) Aayudham_(2005_film)


['Pravin Mani']
1 | Pravin Mani
Aayudham_(2005_film)


Aayudham (2005 film) (en) data
{
  image: <list(0)> 
  infobox: <dict(13)> name, image, director, producer, writer, sta...
  pageid: 9668032
  parsetree: <str(11763)> <root><template><title>Use dmy dates</ti...
  requests: <list(1)> parse
  title: Aayudham (2005 film)
  wikibase: Q4662834
  wikidata_url: https://www.wikidata.org/wiki/Q4662834
  wikitext: <str(7903)> {{Use dmy dates|date=November 2015}}{{Use ...
}
en.wikipedia.org (parse) Ayya_(2005_Tamil_film)


['Dhina', 'Dhina']
2 | Dhina,Dhina
Ayya_(2005_Tamil_film)


...

API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Jathi_(film)


Exception
101 | 
Jathi_(film)


Jathi (film) (en) data
{
  infobox: <dict(13)> name, director, producer, writer, starring, ...
  pageid: 63702185
  parsetree: <str(13317)> <root><template><title>Use Indian Englis...
  requests: <list(1)> parse
  title: Jathi (film)
  wikitext: <str(7878)> {{Use Indian English|date=July 2020}}{{Use...
}
en.wikipedia.org (parse) ex.php?title=Vairavan&action=edit&redlink=1


['Agni Kalaivani', 'Sathya']
102 | Agni Kalaivani,Sathya
ex.php?title=Vairavan&action=edit&redlink=1
Exception
103 | 


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}


In [32]:
for index,row in df.iterrows():
  mname=row['title']
  myear=row['year']
  if mname is not None:
    rating=getrating(mname,myear)
    print(mname+' | '+rating)
    df.at[index,'imdb']=rating.strip()

https://www.imdb.com/title/tt0453201/
Dancer | 6.9
https://www.imdb.com/title/tt11338476/
Aayudham | 
https://www.imdb.com/title/tt0449828/
Ayya | 5.7
https://www.imdb.com/title/tt2396443/
Devathaiyai Kanden | 4.4
https://www.imdb.com/title/tt0475756/
Thirupaachi | 5.8
https://www.imdb.com/title/tt3961528/
Iyer IPS | 
https://www.imdb.com/title/tt6539142/
Kannadi Pookal | 6.7
https://www.imdb.com/title/tt1579740/
Ayodhya | 
https://www.imdb.com/title/tt0317238/
Kannamma | 6.6
https://www.imdb.com/title/tt0460836/
Ji | 4.5
https://www.imdb.com/name/nm1417314/
Ulla Kadathal | 
https://www.imdb.com/title/tt0953605/
Sukran | 3.1
Kadhal Seiya Virumbu | 
Mannin Maindhan | 
https://www.imdb.com/title/tt0453190/
Raam | 7.8
https://www.imdb.com/title/tt1496797/
Maayavi | 4.8
https://www.imdb.com/title/tt1499272/
London | 4.4
https://www.imdb.com/title/tt6812038/
Karagattakkari | 
https://www.imdb.com/title/tt6812038/
Kicha Vayasu 16 | 
https://www.imdb.com/title/tt8903454/
Rightaa Thappaa | 
ht

In [33]:
df

year                               title                   director  \
1    2005                              Dancer                      Keyar   
2    2005                            Aayudham             M. A. Murugesh   
3    2005                                Ayya                       Hari   
4    2005                  Devathaiyai Kanden           Boopathy Pandian   
5    2005                         Thirupaachi                   Perarasu   
6    2005                            Iyer IPS                 Hari Rajan   
7    2005                      Kannadi Pookal               K. Shahjahan   
8    2005                             Ayodhya             R. Jayaprakash   
9    2005                            Kannamma          S. S. Baba Vikram   
10   2005                                  Ji              N. Linguswamy   
11   2005                       Ulla Kadathal      Pugazhendhi Thangaraj   
12   2005                              Sukran        S. A. Chandrasekhar   
13   2005                Kadhal Seiya Virumbu             Ravi Bhargavan   
14   2005                     Mannin Maindhan             Rama Narayanan   
15   2005                                Raam                      Ameer   
16   2005                             Maayavi                 Singampuli   
17   2005                              London                  Sundar C.   
18   2005                      Karagattakkari            Bharathi Kannan   
19   2005                     Kicha Vayasu 16            A. N. Rajagopal   
20   2005                     Rightaa Thappaa                 R. Bhuvana   
21   2005                            Gurudeva                     Jaffer   
22   2005                     Thaka Thimi Tha                  Sundar C.   
23   2005                              Sevvel                    Primose   
24   2005                        Chandramukhi                    P. Vasu   
25   2005                          Priyasakhi            K. S. Adhiyaman   
26   2005                             Sachein             John Mahendran   
27   2005                      Mumbai Express  Singeetham Sreenivasa Rao   
28   2005                              Jithan              Vincent Selva   
29   2005                      Karka Kasadara           R. V. Udayakumar   
30   2005                                6'2"           V. Senthil Kumar   
..    ...                                 ...                        ...   
74   2005                               Majaa                      Shafi   
75   2005                         Saadhuriyan                D. J. Kumar   
76   2005                            Anbe Vaa          K. Selva Bharathy   
77   2005                    Bambara Kannaley              Parthy Baskar   
78   2005                      Uyir Ullavarai             S. S. Bharathy   
79   2005                   Kanda Naal Mudhal                   V. Priya   
80   2005                               Aanai                      Selva   
81   2005              Thavamai Thavamirundhu                     Cheran   
82   2005                                Aaru                       Hari   
83   2005                  Vetrivel Sakthivel             Lakshmi Priyan   
84   2005                          Sandakozhi               N. Lingusamy   
85   2005                            Veeranna                  Kalanidhi   
86   2005                   Vanakkam Thalaiva            Sakthi Paramesh   
87   2005                          Sollattuma               Selvaraj.K.S   
88   2005        Sorry Enaku Kalyanamayidichu                      Sagar   
89   2005                          Unarchigal                   K. Rajan   
90   2005             Ambuttu Imbuttu Embuttu              Ashok Kashyap   
91   2005            Unnai Enakku Pidichuruku             Anbu Saravanan   
92   2005                       Iyyappa Saamy                  Ravi-Raja   
93   2005                     Sendhalam Poove            A. C. Deepa Raj   
94   2005                         Thiru Thiru        

In [34]:
df.to_csv('2005_movies.csv')

# 2004

In [72]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_2004'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_2004=soup.find_all('table')[2:3]

In [63]:
standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

In [64]:
df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [75]:
for table in tables_2004:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='2004'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Title', 'Director', 'Cast', 'Music director']
82!Baabi|||null|null|/w/index.php?title=Baabi_(2004_film)&action=edit&redlink=1
83!Image|||null|null|/w/index.php?title=Image_(2004_film)&action=edit&redlink=1
84!Kizhakku Boomi|||null|null|/w/index.php?title=Kizhakku_Boomi&action=edit&redlink=1
85!Pinju Nenjangal Puthiyathor Ulagam|||null|null|/w/index.php?title=Pinju_Nenjangal_Puthiyathor_Ulagam&action=edit&redlink=1
86!Rush|S. Rajkumar|Rajkumar, Mangai|null|null|/w/index.php?title=Rush_(2004_film)&action=edit&redlink=1
87!Pithan|N.Manikandan|Ashwathaman, Devan, Priya|null|null|/w/index.php?title=Pithan&action=edit&redlink=1


In [77]:
df=df.drop(0)

In [78]:
df

year                               title             director  \
1   2004                              Campus               Sharvi   
2   2004                     Kadhal Saregama             Ameerjan   
3   2004                   Ullathai Killathe         Vijay Kannan   
4   2004                          Engal Anna             Siddique   
5   2004                                 Jai           S. Narayan   
6   2004                               Kovil                 Hari   
7   2004     Pudhukottaiyilirundhu Saravanan        S. S. Stanley   
8   2004                          Virumaandi         Kamal Haasan   
9   2004                 Ennavo Pudichirukku   Meenakshi Sundaram   
10  2004                            Thendral       Thangar Bachan   
11  2004                      Vayasu Pasanga      Bharathi Kannan   
12  2004                             Kamaraj      A. Balakrishnan   
13  2004                          Varnajalam    Nakulan Ponnusamy   
14  2004                           Autograph               Cheran   
15  2004                 Kangalal Kaidhu Sei         Bharathiraja   
16  2004                           Adi Thadi              Shivraj   
17  2004                           Gambeeram               Suresh   
18  2004                          Nee Mattum         Mathi Natesh   
19  2004                       Aarumugasaamy             Raviraja   
20  2004              Pethi Solla Thattathea               Dharma   
21  2004                              Udhaya      Azhagam Perumal   
22  2004                      Kadhal Dot Com             Selvaraj   
23  2004              Kanavu Meippada Vendum  Janaki Vishwanathan   
24  2004                             Kuththu         A. Venkatesh   
25  2004                              Ghilli              Dharani   
26  2004                           Aethirree      K. S. Ravikumar   
27  2004                   Vaanam Vasappadum         P. C. Sriram   
28  2004                            Kavidhai             G. Kicha   
29  2004                                Arul                 Hari   
30  2004                                Jana         Shaji Kailas   
..   ...                                 ...                  ...   
58  2004                                Giri             Sundar C   
59  2004       M. Kumaran Son of Mahalakshmi              M. Raja   
60  2004                 Oru Murai Sollividu           A. Jawahar   
61  2004                                Bose        Senthil Kumar   
62  2004                                Adhu  Ramesh Balakrishnan   
63  2004                   7G Rainbow Colony        Selvaraghavan   
64  2004                      Vishwa Thulasi          Sumathy Ram   
65  2004       En Purushan Ethir Veetu Ponnu         Barani Kumar   
66  2004                      Neranja Manasu          Samudrakani   
67  2004                           Attahasam                Saran   
68  2004                         Chatrapathy            Srimahesh   
69  2004                              Dreams        Kasthuri Raja   
70  2004                           Manmadhan        A. J. Murugan   
71  2004                   Amma Appa Chellam              Suriyan   
72  2004                        Maha Nadigan   Sakthi Chidambaram   
73  2004                                 Aai         A. Venkatesh   
74  2004                            Jaisurya          Manoj Kumar   
75  2004                             Jananam        Ramesh Selvan   
76  2004                             Kaadhal     Balaji Sakthivel   
77  2004                     Meesai Madhavan              S. Ramu   
78  2004                     Gomathi Nayagam            Ponvannan   
79  2004        Kadhale Engal Desiya Geetham          Vijay Anand   
80  2004                         Ramakrishna            Agathiyan   
81  2004                              Remote            Karvannan   
82  2004                               Baabi                        
83  2004                               Image                        
84

In [79]:
df['cast_size']=''
for index,row in df.iterrows():
    mlink=row['cast']
    if mlink is not None:
        arr=str(mlink).split(',')
        df.at[index,'cast_size']=len(arr)

for index,row in df.iterrows():
  mlink=row['link']
  if mlink is not None:
    print(str(mlink).strip()[6:])
    dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
    strdir=tostr(dirs)
    print(str(index)+' | '+strdir)
    df.at[index,'music']=strdir


Campus_(film)


en.wikipedia.org (parse) Campus_(film)
Campus (film) (en) data
{
  infobox: <dict(13)> name, director, producer, writer, starring, ...
  pageid: 56089419
  parsetree: <str(9245)> <root><template><title>Use dmy dates</tit...
  requests: <list(1)> parse
  title: Campus (film)
  wikibase: Q48674020
  wikidata_url: https://www.wikidata.org/wiki/Q48674020
  wikitext: <str(5768)> {{Use dmy dates|date=December 2017}}{{Use ...
}
en.wikipedia.org (parse) Kadhal Saregama


['Rajneesh']
1 | Rajneesh



API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Ullathai_Killathe


Exception
2 | 
Ullathai_Killathe


...

API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) ex.php?title=Rush_(2004_film)&action=edi...


Exception
85 | 
ex.php?title=Rush_(2004_film)&action=edit&redlink=1


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) ex.php?title=Pithan&action=edit&redlink=1


Exception
86 | 
ex.php?title=Pithan&action=edit&redlink=1
Exception
87 | 


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}


In [80]:
for index,row in df.iterrows():
  mname=row['title']
  myear=row['year']
  if mname is not None:
    rating=getrating(mname,myear)
    print(mname+' | '+rating)
    df.at[index,'imdb']=rating.strip()

https://www.imdb.com/title/tt4944412/
Campus | 
https://www.imdb.com/title/tt0453740/
Kadhal Saregama | 7.5
https://www.imdb.com/list/ls066460534/?ref_=rls_1&page=2
Ullathai Killathe | 
https://www.imdb.com/title/tt0396582/
Engal Anna | 5.3
https://www.imdb.com/title/tt6811240/
Jai | 4.2
https://www.imdb.com/title/tt1483773/
Kovil | 5.7
https://www.imdb.com/title/tt1744880/
Pudhukottaiyilirundhu Saravanan | 3.8
https://www.imdb.com/title/tt0364647/
Virumaandi | 8.3
Ennavo Pudichirukku | 
https://www.imdb.com/title/tt6758130/
Thendral | 
https://www.imdb.com/title/tt6536178/
Vayasu Pasanga | 
https://www.imdb.com/title/tt0375887/
Kamaraj | 7.0
https://www.imdb.com/title/tt4946168/
Varnajalam | 5.3
https://www.imdb.com/title/tt0411131/
Autograph | 7.7
https://www.imdb.com/title/tt0375889/
Kangalal Kaidhu Sei | 5.5
https://www.imdb.com/title/tt2325509/
Adi Thadi | 4.7
https://www.imdb.com/title/tt1401121/
Gambeeram | 4.4
https://www.imdb.com/title/tt0376003/
Nee Mattum | 6.3
https://www.i

In [81]:
df

year                               title             director  \
1   2004                              Campus               Sharvi   
2   2004                     Kadhal Saregama             Ameerjan   
3   2004                   Ullathai Killathe         Vijay Kannan   
4   2004                          Engal Anna             Siddique   
5   2004                                 Jai           S. Narayan   
6   2004                               Kovil                 Hari   
7   2004     Pudhukottaiyilirundhu Saravanan        S. S. Stanley   
8   2004                          Virumaandi         Kamal Haasan   
9   2004                 Ennavo Pudichirukku   Meenakshi Sundaram   
10  2004                            Thendral       Thangar Bachan   
11  2004                      Vayasu Pasanga      Bharathi Kannan   
12  2004                             Kamaraj      A. Balakrishnan   
13  2004                          Varnajalam    Nakulan Ponnusamy   
14  2004                           Autograph               Cheran   
15  2004                 Kangalal Kaidhu Sei         Bharathiraja   
16  2004                           Adi Thadi              Shivraj   
17  2004                           Gambeeram               Suresh   
18  2004                          Nee Mattum         Mathi Natesh   
19  2004                       Aarumugasaamy             Raviraja   
20  2004              Pethi Solla Thattathea               Dharma   
21  2004                              Udhaya      Azhagam Perumal   
22  2004                      Kadhal Dot Com             Selvaraj   
23  2004              Kanavu Meippada Vendum  Janaki Vishwanathan   
24  2004                             Kuththu         A. Venkatesh   
25  2004                              Ghilli              Dharani   
26  2004                           Aethirree      K. S. Ravikumar   
27  2004                   Vaanam Vasappadum         P. C. Sriram   
28  2004                            Kavidhai             G. Kicha   
29  2004                                Arul                 Hari   
30  2004                                Jana         Shaji Kailas   
..   ...                                 ...                  ...   
58  2004                                Giri             Sundar C   
59  2004       M. Kumaran Son of Mahalakshmi              M. Raja   
60  2004                 Oru Murai Sollividu           A. Jawahar   
61  2004                                Bose        Senthil Kumar   
62  2004                                Adhu  Ramesh Balakrishnan   
63  2004                   7G Rainbow Colony        Selvaraghavan   
64  2004                      Vishwa Thulasi          Sumathy Ram   
65  2004       En Purushan Ethir Veetu Ponnu         Barani Kumar   
66  2004                      Neranja Manasu          Samudrakani   
67  2004                           Attahasam                Saran   
68  2004                         Chatrapathy            Srimahesh   
69  2004                              Dreams        Kasthuri Raja   
70  2004                           Manmadhan        A. J. Murugan   
71  2004                   Amma Appa Chellam              Suriyan   
72  2004                        Maha Nadigan   Sakthi Chidambaram   
73  2004                                 Aai         A. Venkatesh   
74  2004                            Jaisurya          Manoj Kumar   
75  2004                             Jananam        Ramesh Selvan   
76  2004                             Kaadhal     Balaji Sakthivel   
77  2004                     Meesai Madhavan              S. Ramu   
78  2004                     Gomathi Nayagam            Ponvannan   
79  2004        Kadhale Engal Desiya Geetham          Vijay Anand   
80  2004                         Ramakrishna            Agathiyan   
81  2004                              Remote            Karvannan   
82  2004                               Baabi                        
83  2004                               Image                        
84

In [83]:
df.to_csv('2004_movies.csv')

# 2003

In [23]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_2003'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_2003=soup.find_all('table')[1:2]

In [24]:
standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

In [19]:
df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [26]:
for table in tables_2003:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='2003'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Title', 'Director', 'Cast', 'Production', 'Music director']
93!Adhaanda Idhaanda|Antony|Pandiarajan, Sangeetha|null|null|
94!Beacham|P. Selvam|Aneesh Ravi, Sonia Mohandas|null|null|/w/index.php?title=Beacham_(2003_film)&action=edit&redlink=1
95!Bhayam|||null|null|
96!E. P. Ko 215|||null|null|
97!Oru Thadava Sonna|M. Jameenraj|Chinni Jayanth, Swarna, Lavanya|null|null|
98!Paththikichu|S. S. Arasan|Devaguru, Manjari, Shakeela, Abhinayashree|null|null|
99!Rekkai|B. Lenin|Narayana Rao, M. S. Vishalakshi, N. Madankumar|null|null|
100!Vaanam Vaazhthattum|Raveeshwar|Vimalahasan, Rashmi, Kulkarni|null|null|
101!Vani Mahal|Jayram|Vignesh, Suvalakshmi|null|null|


In [27]:
df=df.drop(0)

In [28]:
df

year                    title             director  \
1    2003                    Dhool              Dharani   
2    2003               Anbe Sivam             Sundar C   
3    2003         Annai Kaligambal       Rama Narayanan   
4    2003           Chokka Thangam         K. Bhagyaraj   
5    2003               Kalatpadai            J. Ramesh   
6    2003              Ramachandra           Raj Kapoor   
7    2003                Vaseegara    K. Selva Bharathy   
8    2003                 Pop Corn               Nassar   
9    2003           Aasai Aasaiyai          Ravi Mariya   
10   2003                     Anbu       Thalapathy Raj   
11   2003             Anbu Thollai               Hayath   
12   2003          Julie Ganapathi        Balu Mahendra   
13   2003               Kadhaludan          Rajakumaran   
14   2003         Student Number 1                Selva   
15   2003                Yes Madam               Rajaji   
16   2003                 Military        G. Sai Suresh   
17   2003                 Pallavan           Padmamagan   
18   2003               Manasellam             Santhosh   
19   2003    Nilavil Kalangamillai          V. Sabrivel   
20   2003              Pavalakkodi     Ram-CP Saravanan   
21   2003  Ennai Thalatta Varuvala      K. S. Ravindran   
22   2003           Magic Magic 3D                 Jose   
23   2003                      Dum         A. Venkatesh   
24   2003          Kadhal Sadugudu          V. Z. Durai   
25   2003                     Sena               Sujith   
26   2003                    Arasu               Suresh   
27   2003        Banda Paramasivam      T. P. Gajendran   
28   2003           Punnagai Poove      S. D. Sabapathy   
29   2003                Well Done            Ravindran   
30   2003                Anbe Anbe        Mani Bharathi   
..    ...                      ...                  ...   
72   2003           Vadakku Vaasal         B. M. Sundar   
73   2003                 Anjaneya         N. Maharajan   
74   2003                   Ottran           Ilankannan   
75   2003               Pithamagan                 Bala   
76   2003               Thirumalai               Ramana   
77   2003                  Jay Jay                Saran   
78   2003                  Iyarkai     S. P. Jananathan   
79   2003                  Kurumbu        Vishnuvardhan   
80   2003        Nadhi Karaiyinile            Ponvannan   
81   2003            Anbe Un Vasam              R. Balu   
82   2003              Mullil Roja     Kasthuri Pandian   
83   2003           Konji Pesalaam       R. Kaleeswaran   
84   2003          Ooruku Nooruper             B. Lenin   
85   2003      Enakku 20 Unakku 18       Jyothi Krishna   
86   2003                     Joot      Azhagam Perumal   
87   2003   Thathi Thavadhu Manasu  Azhagu Rajusundaram   
88   2003                    Soori              Shelvan   
89   2003                    Indru    Naveen Muthuraman   
90   2003     Sindhamal Sitharamal    K. A. Shanmugavel   
91   2003          Kadhal Kirukkan   Sakthi Chidambaram   
92   2003                 Oruththi         Amshan Kumar   
93   2003        Adhaanda Idhaanda               Antony   
94   2003                  Beacham            P. Selvam   
95   2003                   Bhayam                        
96   2003             E. P. Ko 215                        
97   2003        Oru Thadava Sonna         M. Jameenraj   
98   2003             Paththikichu         S. S. Arasan   
99   2003                   Rekkai             B. Lenin   
100  2003      Vaanam Vaazhthattum           Raveeshwar   
101  2003               Vani Mahal               Jayram   

                                                  cast genre producer  \
1                         Vikram, Jyothika, Reemma Sen  null     null   
2                        Kamal Haasan, Madhavan, Kiran  null     null   
3            Livingston, Ramya Krishnan, Anu Prabhakar  null     null   
4                  Vijayakanth, Soundarya, Prakash Raj

In [29]:
df['cast_size']=''
for index,row in df.iterrows():
    mlink=row['cast']
    if mlink is not None:
        arr=str(mlink).split(',')
        df.at[index,'cast_size']=len(arr)

for index,row in df.iterrows():
  mlink=row['link']
  if mlink is not None:
    print(str(mlink).strip()[6:])
    dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
    strdir=tostr(dirs)
    print(str(index)+' | '+strdir)
    df.at[index,'music']=strdir


Dhool


en.wikipedia.org (parse) Dhool
en.wikipedia.org (imageinfo) File:Dhool poster.jpg
Dhool (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Dhool pos...
  infobox: <dict(18)> name, image, caption, director, producer, wr...
  pageid: 5208783
  parsetree: <str(19582)> <root><template><title>About</title><par...
  requests: <list(2)> parse, imageinfo
  title: Dhool
  wikibase: Q3025733
  wikidata_url: https://www.wikidata.org/wiki/Q3025733
  wikitext: <str(13253)> {{About|the 2003 film|the 2011 film|Dhool...
}
en.wikipedia.org (parse) Anbe_Sivam


['Vidyasagar', 'Vidyasagar']
1 | Vidyasagar,Vidyasagar
Anbe_Sivam


en.wikipedia.org (imageinfo) File:Anbe Sivam.jpg
Anbe Sivam (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Anbe Siva...
  infobox: <dict(18)> name, image, alt, caption, director, produce...
  pageid: 2852763
  parsetree: <str(103967)> <root><template><title>short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Anbe Sivam
  wikibase: Q3418126
  wikidata_url: https://www.wikidata.org/wiki/Q3418126
  wikitext: <str(79343)> {{short description|2003 film by Sundar C...
}
en.wikipedia.org (parse) Annai_Kaligambal


['Vidyasagar', 'composer', 'Vidyasagar']
2 | Vidyasagar,composer,Vidyasagar
Annai_Kaligambal


...

API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Vaanam Vaazhthattum


Exception
99 | 



API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Vani Mahal


Exception
100 | 

Exception
101 | 


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}


In [30]:
for index,row in df.iterrows():
  mname=row['title']
  myear=row['year']
  if mname is not None:
    rating=getrating(mname,myear)
    print(mname+' | '+rating)
    df.at[index,'imdb']=rating.strip()

https://www.imdb.com/title/tt0362548/
Dhool | 7.1
https://www.imdb.com/title/tt0367495/
Anbe Sivam | 8.7
https://www.imdb.com/title/tt0233222/
Annai Kaligambal | 6.6
https://www.imdb.com/title/tt8938678/
Chokka Thangam | 6.2
https://www.imdb.com/india/top-rated-tamil-movies/
Kalatpadai | 
https://www.imdb.com/name/nm0045084/
Ramachandra | 
https://www.imdb.com/title/tt0476706/
Vaseegara | 6.0
https://www.imdb.com/title/tt0390376/
Pop Corn | 3.8
https://www.imdb.com/title/tt8669128/
Aasai Aasaiyai | 4.8
https://www.imdb.com/title/tt8914086/
Anbu | 
https://www.imdb.com/title/tt8914086/
Anbu Thollai | 
https://www.imdb.com/title/tt0353590/
Julie Ganapathi | 6.0
https://www.imdb.com/title/tt0375880/
Kadhaludan | 7.9
https://www.imdb.com/title/tt12364466/
Student Number 1 | 
https://www.imdb.com/title/tt0032148/
Yes Madam | 7.3
https://www.imdb.com/search/title/?languages=ta&sort=user_rating,asc
Military | 
https://www.imdb.com/title/tt5730414/
Pallavan | 
https://www.imdb.com/title/tt0782

In [31]:
df.to_csv("2003_movies.csv")

# 2002

In [16]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_2002'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_2002=soup.find_all('table')[2:4]

In [17]:
standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

In [18]:
df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [20]:
for table in tables_2002:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='2002'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')-1

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')-1

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Opening', 'Title', 'Director', 'Cast', 'Studio', 'Ref']
1!Alli Arjuna|Saran|Manoj Bharathiraja, Richa Pallod, Preetha|null|Janani Art Creations|/wiki/Alli_Arjuna
2!Azhagi|Thangar Bachan|Parthiban, Nandita Das, Devayani|null|Udayageetha Cine Creations|/wiki/Azhagi_(2002_film)
3!Pammal K. Sambandam|Mouli|Kamal Haasan, Simran, Abbas, Sneha|null|Sri Rajlakshmi Films|/wiki/Pammal_K._Sambandam
4!Punnagai Desam|K. Shajahan|Tarun, Kunal, Sneha, Hamsavardhan, Preetha|null|Super Good Films|/wiki/Punnagai_Desam
5!Red|Ram Sathya|Ajith Kumar, Priya Gill, Salim Ghouse|null|NIC Arts|/wiki/Red_(2002_film)
6!Vivaramana Aalu|K. Selva Bharathy|Sathyaraj, Devayani, Mumtaj|null|Evergreen Movies International|/wiki/Vivaramana_Aalu
7!Mangalyam|V. J. Rao-Asra|Thalaivasal Vijay, Thara|null||
8!Amaiyappan|Rama Murugapandian|Ponnambalam, Roshini|null||
9!Shakalaka Baby|Rama Narayanan|Ramki, Roja|null|Shivakami Production|/wiki/Shakalaka_Baby_(film)
10!Dhaya|Senthil Kumar|Prakash Raj, Meena, Raghuvaran|null|Due

In [21]:
df=df.drop(0)
df

year                              title             director  \
1   2002                        Alli Arjuna                Saran   
2   2002                             Azhagi       Thangar Bachan   
3   2002                Pammal K. Sambandam                Mouli   
4   2002                     Punnagai Desam          K. Shajahan   
5   2002                                Red           Ram Sathya   
6   2002                    Vivaramana Aalu    K. Selva Bharathy   
7   2002                          Mangalyam       V. J. Rao-Asra   
8   2002                         Amaiyappan   Rama Murugapandian   
9   2002                     Shakalaka Baby       Rama Narayanan   
10  2002                              Dhaya        Senthil Kumar   
11  2002              Kannathil Muthamittal          Mani Ratnam   
12  2002                    Charlie Chaplin   Sakthi Chidambaram   
13  2002                           Kamarasu     P. C. Anbazhagan   
14  2002                        Roja Kootam                 Sasi   
15  2002                            Saptham             Sivaguru   
16  2002                             Gemini                Saran   
17  2002                          Raajjiyam          Manoj Kumar   
18  2002                  Sri Bannari Amman      Bharathi Kannan   
19  2002                            Thamizh                 Hari   
20  2002                          Thamizhan               Majith   
21  2002                      Junior Senior            J. Suresh   
22  2002            Varushamellam Vasantham         Ravi Shankar   
23  2002                 Thulluvadho Ilamai        Kasthuri Raja   
24  2002                     Unnai Ninaithu             Vikraman   
25  2002              Pesadha Kannum Pesume       Murali Krishna   
26  2002  Nettru Varai Nee Yaaro Naan Yaaro           A. Sridhar   
27  2002               Enge Enadhu Kavithai              Aravind   
28  2002                                123            K. Subash   
29  2002                              Devan         Arun Pandian   
30  2002                          Ezhumalai                Arjun   
..   ...                                ...                  ...   
48  2002                     En Mana Vaanil              Vinayan   
49  2002                      Nilai Kannadi   C. T. Chandrasekar   
50  2002                             Maaran           A. Jawahar   
51  2002               Namma Veetu Kalyanam            V. Sekhar   
52  2002                        Samasthanam            Rajkapoor   
53  2002                          Jjunction     R. S. Ramanathan   
54  2002                              Album        Vasanthabalan   
55  2002                          Five Star         Susi Ganesan   
56  2002                         University          Pragahdeesh   
57  2002              Andipatti Arasampatti          Dhananjeyan   
58  2002                          Bagavathi         A. Venkatesh   
59  2002                               Game        John Amritraj   
60  2002               Kadhal Azhivathillai          T. Rajendar   
61  2002                  Padai Veetu Amman           Pugazhmani   
62  2002                            Ramanaa     A. R. Murugadoss   
63  2002              Solla Marandha Kadhai       Thangar Bachan   
64  2002                            Villain      K. S. Ravikumar   
65  2002                   April Maadhathil        S. S. Stanley   
66  2002                           Gummalam     Sughi S. Moorthy   
67  2002                      I Love You Da         C. Rajadurai   
68  2002                               Jaya            Rathnaraj   
69  2002                             Mutham  S. A. Chandrasekhar   
70  2002                               Bala               Deepak   
71  2002                   Mounam Pesiyadhe                Ameer   
72  2002                       Kadhal Virus               Kathir   
73  2002                              Style    Sibi Chakravarthy   
74  2002                       Virumbugiren         Susi Gan

In [22]:
df['cast_size']=''
for index,row in df.iterrows():
    mlink=row['cast']
    if mlink is not None:
        arr=str(mlink).split(',')
        df.at[index,'cast_size']=len(arr)

for index,row in df.iterrows():
  mlink=row['link']
  if mlink is not None:
    print(str(mlink).strip()[6:])
    dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
    strdir=tostr(dirs)
    print(str(index)+' | '+strdir)
    df.at[index,'music']=strdir


Alli_Arjuna


en.wikipedia.org (parse) Alli_Arjuna
Alli Arjuna (en) data
{
  infobox: <dict(14)> name, director, writer, starring, producer, ...
  pageid: 9272916
  parsetree: <str(9870)> <root><template><title>for</title><part><...
  requests: <list(1)> parse
  title: Alli Arjuna
  wikibase: Q3418172
  wikidata_url: https://www.wikidata.org/wiki/Q3418172
  wikitext: <str(6658)> {{for|the 1935 film|Alli Arjuna (1935 film...
}
en.wikipedia.org (parse) Azhagi_(2002_film)


['A. R. Rahman']
1 | A. R. Rahman
Azhagi_(2002_film)


en.wikipedia.org (imageinfo) File:Azhagi (2002).jpg
Azhagi (2002 film) (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Azhagi (2...
  infobox: <dict(15)> name, image, director, writer, starring, pro...
  pageid: 7237349
  parsetree: <str(13280)> <root><template><title>about </title><pa...
  requests: <list(2)> parse, imageinfo
  title: Azhagi (2002 film)
  wikibase: Q4832499
  wikidata_url: https://www.wikidata.org/wiki/Q4832499
  wikitext: <str(9436)> {{about || the 1953 film|Azhagi (1953 film...
}
en.wikipedia.org (parse) Pammal_K._Sambandam


['Ilaiyaraaja']
2 | Ilaiyaraaja
Pammal_K._Sambandam


...

API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Kalai Kalluri


Exception
75 | 



API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Nanba_Nanba


Exception
76 | 
Nanba_Nanba
['Chittey Prakash']
77 | Chittey Prakash


Nanba Nanba (en) data
{
  infobox: <dict(13)> name, director, producer, writer, starring, ...
  pageid: 53290934
  parsetree: <str(7748)> <root><template><title>Use dmy dates</tit...
  requests: <list(1)> parse
  title: Nanba Nanba
  wikibase: Q12984861
  wikidata_url: https://www.wikidata.org/wiki/Q12984861
  wikitext: <str(5333)> {{Use dmy dates|date=March 2018}}{{Use Ind...
}


In [23]:
for index,row in df.iterrows():
  mname=row['title']
  myear=row['year']
  if mname is not None:
    rating=getrating(mname,myear)
    print(mname+' | '+rating)
    df.at[index,'imdb']=rating.strip()

https://www.imdb.com/title/tt0485183/
Alli Arjuna | 3.6
https://www.imdb.com/title/tt0459186/
Azhagi | 7.1
https://www.imdb.com/title/tt0422908/
Pammal K. Sambandam | 7.1
https://www.imdb.com/title/tt0155067/
Punnagai Desam | 7.4
https://www.imdb.com/title/tt1039983/
Red | 4.3
Vivaramana Aalu | 
https://www.imdb.com/name/nm11660146/
Mangalyam | 
https://www.imdb.com/india/top-rated-tamil-movies/
Amaiyappan | 
https://www.imdb.com/title/tt00220656/
Shakalaka Baby | 8.3
https://www.imdb.com/title/tt6801812/
Dhaya | 5.1
https://www.imdb.com/title/tt0312859/
Kannathil Muthamittal | 8.4
https://www.imdb.com/title/tt7238390/
Charlie Chaplin | 6.1
https://www.imdb.com/title/tt9001478/
Kamarasu | 5.2
https://www.imdb.com/title/tt1365500/
Roja Kootam | 6.1
https://www.imdb.com/search/title/?title_type=feature&year=2002-01-01,2002-12-31&languages=ta&sort=release_date,desc&explore=languages&view=advanced
Saptham | 
https://www.imdb.com/title/tt0327761/
Gemini | 6.1
https://www.imdb.com/title/tt12

In [24]:
df

year                              title             director  \
1   2002                        Alli Arjuna                Saran   
2   2002                             Azhagi       Thangar Bachan   
3   2002                Pammal K. Sambandam                Mouli   
4   2002                     Punnagai Desam          K. Shajahan   
5   2002                                Red           Ram Sathya   
6   2002                    Vivaramana Aalu    K. Selva Bharathy   
7   2002                          Mangalyam       V. J. Rao-Asra   
8   2002                         Amaiyappan   Rama Murugapandian   
9   2002                     Shakalaka Baby       Rama Narayanan   
10  2002                              Dhaya        Senthil Kumar   
11  2002              Kannathil Muthamittal          Mani Ratnam   
12  2002                    Charlie Chaplin   Sakthi Chidambaram   
13  2002                           Kamarasu     P. C. Anbazhagan   
14  2002                        Roja Kootam                 Sasi   
15  2002                            Saptham             Sivaguru   
16  2002                             Gemini                Saran   
17  2002                          Raajjiyam          Manoj Kumar   
18  2002                  Sri Bannari Amman      Bharathi Kannan   
19  2002                            Thamizh                 Hari   
20  2002                          Thamizhan               Majith   
21  2002                      Junior Senior            J. Suresh   
22  2002            Varushamellam Vasantham         Ravi Shankar   
23  2002                 Thulluvadho Ilamai        Kasthuri Raja   
24  2002                     Unnai Ninaithu             Vikraman   
25  2002              Pesadha Kannum Pesume       Murali Krishna   
26  2002  Nettru Varai Nee Yaaro Naan Yaaro           A. Sridhar   
27  2002               Enge Enadhu Kavithai              Aravind   
28  2002                                123            K. Subash   
29  2002                              Devan         Arun Pandian   
30  2002                          Ezhumalai                Arjun   
..   ...                                ...                  ...   
48  2002                     En Mana Vaanil              Vinayan   
49  2002                      Nilai Kannadi   C. T. Chandrasekar   
50  2002                             Maaran           A. Jawahar   
51  2002               Namma Veetu Kalyanam            V. Sekhar   
52  2002                        Samasthanam            Rajkapoor   
53  2002                          Jjunction     R. S. Ramanathan   
54  2002                              Album        Vasanthabalan   
55  2002                          Five Star         Susi Ganesan   
56  2002                         University          Pragahdeesh   
57  2002              Andipatti Arasampatti          Dhananjeyan   
58  2002                          Bagavathi         A. Venkatesh   
59  2002                               Game        John Amritraj   
60  2002               Kadhal Azhivathillai          T. Rajendar   
61  2002                  Padai Veetu Amman           Pugazhmani   
62  2002                            Ramanaa     A. R. Murugadoss   
63  2002              Solla Marandha Kadhai       Thangar Bachan   
64  2002                            Villain      K. S. Ravikumar   
65  2002                   April Maadhathil        S. S. Stanley   
66  2002                           Gummalam     Sughi S. Moorthy   
67  2002                      I Love You Da         C. Rajadurai   
68  2002                               Jaya            Rathnaraj   
69  2002                             Mutham  S. A. Chandrasekhar   
70  2002                               Bala               Deepak   
71  2002                   Mounam Pesiyadhe                Ameer   
72  2002                       Kadhal Virus               Kathir   
73  2002                              Style    Sibi Chakravarthy   
74  2002                       Virumbugiren         Susi Gan

In [25]:
df.to_csv('2002_movies.csv')

# 2001

In [35]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_2001'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_2001=soup.find_all('table')[4:5]

In [36]:
standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

In [32]:
df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [40]:
for table in tables_2001:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='2001'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')-1

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')-1

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
#             df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
#             df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
#               df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
#             df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Title', 'Director', 'Cast', 'Studio', 'Ref']
75!Maname Mayangathe|Shapeer Kumar|Padmaja, Hema, Meenu|null|Padmaja, Hema, Meenu|
75!Theerpugal Maatrapadalaam|M. K. Arunthavaraj|Babu Ganesh, Vichithra|null|Babu Ganesh, Vichithra|


In [41]:
df=df.drop(0)

In [42]:
df

year                            title                  director  \
1   2001                           Dheena          A. R. Murugadoss   
2   2001                          Friends                  Siddique   
3   2001                            Looty               Parameshwar   
4   2001                        Nageswari            Rama Narayanan   
5   2001              Pullanalum Pondatti             Adade Manohar   
6   2001                    Vaanchinathan              Shaji Kailas   
7   2001                      Nila Kaalam            Gandhi Krishna   
8   2001                          Minnale             Gautham Menon   
9   2001          Engalukkum Kaalam Varum               Bala Rooban   
10  2001            Ullam Kollai Poguthae                 Sundar C.   
11  2001           Piriyadha Varam Vendum                     Kamal   
12  2001                            Rishi                 Sundar C.   
13  2001          Thaalikaatha Kaaliamman                Somasundar   
14  2001  En Purushan Kuzhandhai Maadhiri            S. P. Rajkumar   
15  2001            Ninaikkatha Naalillai                A. L. Raja   
16  2001            Paarvai Ondre Podhume            Murali Krishna   
17  2001                 Natchatra Kadhal                   Nayakam   
18  2001                Vinnukum Mannukum               Rajakumaran   
19  2001                            Badri         P. A. Arun Prasad   
20  2001                   Dumm Dumm Dumm           Azhagam Perumal   
21  2001                      Little John  Singeetham Srinivasa Rao   
22  2001                Seerivarum Kaalai                 Ramarajan   
23  2001              Sri Raja Rajeshwari           Bharathi Kannan   
24  2001             En Iniya Pon Nilavae             Balu Mahendra   
25  2001                          Asathal                   P. Vasu   
26  2001            Middle Class Madhavan           T. P. Gajendran   
27  2001                Sigamani Ramamani                      Visu   
28  2001                        Aanandham              N. Lingusamy   
29  2001            Sonnal Thaan Kaadhala               T. Rajendar   
30  2001                          Citizen          Saravana Subbiah   
..   ...                              ...                       ...   
45  2001               Poovellam Un Vasam                     Ezhil   
46  2001                           Vedham                     Arjun   
47  2001                       Samudhiram           K. S. Ravikumar   
48  2001                         Chocklet               A.Venkatesh   
49  2001              Alli Thandha Vaanam            Sridhar Prasad   
50  2001                           Maayan                    Nassar   
51  2001               Veettoda Mappillai                 V. Sekhar   
52  2001                  Pandavar Bhoomi                    Cheran   
53  2001                              12B                     Jeeva   
54  2001                    Kabadi Kabadi               Pandiarajan   
55  2001                    Mitta Miraasu                Kalanjiyam   
56  2001                       Asokavanam       Thakkali Srinivasan   
57  2001                      Aalavandhan            Suresh Krishna   
58  2001                    Aandan Adimai                Manivannan   
59  2001                Kaatrukkenna Veli     Pugazhendhi Thangaraj   
60  2001                             Kasi                   Vinayan   
61  2001                    Love Marriage                 K. Subash   
62  2001           Manadhai Thirudivittai      R. D. Narayanamurthy   
63  2001                           Nandha                      Bala   
64  2001              Paarthale Paravasam            K. Balachandar   
65  2001                    Ponnana Neram                   Raviraj   
66  2001                        Shahjahan                      Ravi   
67  2001                          Thavasi               Udayasankar   
68  2001                 Azhagana Naatkal                 Sundar C.   
69  2001                    

In [43]:
df['cast_size']=''
for index,row in df.iterrows():
    mlink=row['cast']
    if mlink is not None:
        arr=str(mlink).split(',')
        df.at[index,'cast_size']=len(arr)

for index,row in df.iterrows():
  mlink=row['link']
  if mlink is not None:
    print(str(mlink).strip()[6:])
    dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
    strdir=tostr(dirs)
    print(str(index)+' | '+strdir)
    df.at[index,'music']=strdir


Dheena


en.wikipedia.org (parse) Dheena
en.wikipedia.org (imageinfo) File:Dheena.jpg
Dheena (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Dheena.jp...
  infobox: <dict(16)> name, image, caption, native_name, director,...
  pageid: 6809623
  parsetree: <str(16319)> <root><template><title>other uses</title...
  requests: <list(2)> parse, imageinfo
  title: Dheena
  wikibase: Q3424187
  wikidata_url: https://www.wikidata.org/wiki/Q3424187
  wikitext: <str(10889)> {{other uses}}{{short description|2001 fi...
}
en.wikipedia.org (parse) Friends_(2001_film)


['Yuvan Shankar Raja']
1 | Yuvan Shankar Raja
Friends_(2001_film)


en.wikipedia.org (imageinfo) File:Friends 2001 poster.jpg
Friends (2001 film) (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Friends 2...
  infobox: <dict(17)> name, image, caption, director, writer, stor...
  pageid: 5671546
  parsetree: <str(14800)> <root><template><title>short description...
  requests: <list(2)> parse, imageinfo
  title: Friends (2001 film)
  wikibase: Q3429658
  wikidata_url: https://www.wikidata.org/wiki/Q3429658
  wikitext: <str(10019)> {{short description|2001 film by Siddique...
}
en.wikipedia.org (parse) Looty


['Ilaiyaraaja']
2 | Ilaiyaraaja
Looty


...

en.wikipedia.org (parse) Vadagupatti_Maapillai
Vadagupatti Maapillai (en) data
{
  infobox: <dict(12)> name, director, producer, writer, starring, ...
  pageid: 53220138
  parsetree: <str(4742)> <root><template><title>short description<...
  requests: <list(1)> parse
  title: Vadagupatti Maapillai
  wikibase: Q29025743
  wikidata_url: https://www.wikidata.org/wiki/Q29025743
  wikitext: <str(2560)> {{short description|2001 film}}{{Use dmy d...
}
en.wikipedia.org (parse) Maname Mayangathe


['Sirpy']
72 | Sirpy



API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Theerpugal Maatrapadalaam


Exception
73 | 

Exception
74 | 


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}


In [44]:
for index,row in df.iterrows():
  mname=row['title']
  myear=row['year']
  if mname is not None:
    rating=getrating(mname,myear)
    print(mname+' | '+rating)
    df.at[index,'imdb']=rating.strip()

https://www.imdb.com/title/tt0807944/
Dheena | 7.2
https://www.imdb.com/title/tt0308280/
Friends | 7.5
https://www.imdb.com/title/tt8032774/
Looty | 4.3
https://www.imdb.com/search/title/?year=2001,2001&title_type=feature&explore=languages&languages=ta&ref_=adv_explore_rhs
Nageswari | 
Pullanalum Pondatti | 
https://www.imdb.com/name/nm1115537/
Vaanchinathan | 
https://www.imdb.com/title/tt5082662/
Nila Kaalam | 6.8
https://www.imdb.com/title/tt0357905/
Minnale | 7.6
https://www.imdb.com/search/title/?languages=ta&sort=user_rating,asc
Engalukkum Kaalam Varum | 
https://www.imdb.com/title/tt10225032/
Ullam Kollai Poguthae | 8.0
https://www.imdb.com/title/tt0353884/
Piriyadha Varam Vendum | 6.1
https://www.imdb.com/title/tt1507347/
Rishi | 3.9
https://www.imdb.com/title/tt000294264/
Thaalikaatha Kaaliamman | 7.2
https://www.imdb.com/title/tt6536876/
En Purushan Kuzhandhai Maadhiri | 5.7
https://www.imdb.com/title/tt6758006/
Ninaikkatha Naalillai | 
Paarvai Ondre Podhume | 
https://www.im

In [45]:
df

year                            title                  director  \
1   2001                           Dheena          A. R. Murugadoss   
2   2001                          Friends                  Siddique   
3   2001                            Looty               Parameshwar   
4   2001                        Nageswari            Rama Narayanan   
5   2001              Pullanalum Pondatti             Adade Manohar   
6   2001                    Vaanchinathan              Shaji Kailas   
7   2001                      Nila Kaalam            Gandhi Krishna   
8   2001                          Minnale             Gautham Menon   
9   2001          Engalukkum Kaalam Varum               Bala Rooban   
10  2001            Ullam Kollai Poguthae                 Sundar C.   
11  2001           Piriyadha Varam Vendum                     Kamal   
12  2001                            Rishi                 Sundar C.   
13  2001          Thaalikaatha Kaaliamman                Somasundar   
14  2001  En Purushan Kuzhandhai Maadhiri            S. P. Rajkumar   
15  2001            Ninaikkatha Naalillai                A. L. Raja   
16  2001            Paarvai Ondre Podhume            Murali Krishna   
17  2001                 Natchatra Kadhal                   Nayakam   
18  2001                Vinnukum Mannukum               Rajakumaran   
19  2001                            Badri         P. A. Arun Prasad   
20  2001                   Dumm Dumm Dumm           Azhagam Perumal   
21  2001                      Little John  Singeetham Srinivasa Rao   
22  2001                Seerivarum Kaalai                 Ramarajan   
23  2001              Sri Raja Rajeshwari           Bharathi Kannan   
24  2001             En Iniya Pon Nilavae             Balu Mahendra   
25  2001                          Asathal                   P. Vasu   
26  2001            Middle Class Madhavan           T. P. Gajendran   
27  2001                Sigamani Ramamani                      Visu   
28  2001                        Aanandham              N. Lingusamy   
29  2001            Sonnal Thaan Kaadhala               T. Rajendar   
30  2001                          Citizen          Saravana Subbiah   
..   ...                              ...                       ...   
45  2001               Poovellam Un Vasam                     Ezhil   
46  2001                           Vedham                     Arjun   
47  2001                       Samudhiram           K. S. Ravikumar   
48  2001                         Chocklet               A.Venkatesh   
49  2001              Alli Thandha Vaanam            Sridhar Prasad   
50  2001                           Maayan                    Nassar   
51  2001               Veettoda Mappillai                 V. Sekhar   
52  2001                  Pandavar Bhoomi                    Cheran   
53  2001                              12B                     Jeeva   
54  2001                    Kabadi Kabadi               Pandiarajan   
55  2001                    Mitta Miraasu                Kalanjiyam   
56  2001                       Asokavanam       Thakkali Srinivasan   
57  2001                      Aalavandhan            Suresh Krishna   
58  2001                    Aandan Adimai                Manivannan   
59  2001                Kaatrukkenna Veli     Pugazhendhi Thangaraj   
60  2001                             Kasi                   Vinayan   
61  2001                    Love Marriage                 K. Subash   
62  2001           Manadhai Thirudivittai      R. D. Narayanamurthy   
63  2001                           Nandha                      Bala   
64  2001              Paarthale Paravasam            K. Balachandar   
65  2001                    Ponnana Neram                   Raviraj   
66  2001                        Shahjahan                      Ravi   
67  2001                          Thavasi               Udayasankar   
68  2001                 Azhagana Naatkal                 Sundar C.   
69  2001                    

In [46]:
df.to_csv('2001_movies.csv')

# 2000

In [53]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_2000'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_2000=soup.find_all('table')[0:4]

In [54]:
standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

In [55]:
df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [57]:
for table in tables_2000:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='2000'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')-1

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')-1

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Opening', 'Title', 'Director', 'Cast', 'Studio', 'Ref']
1!Kadhal Rojavae|K. R.|George Vishnu, Pooja Kumar, Sarath Babu|null|Oscar Movies|/wiki/Kadhal_Rojavae
2!Kannukkul Nilavu|Fazil|Vijay, Shalini, Kaveri, Raghuvaran|null|Chinthamani Cine Arts|/wiki/Kannukkul_Nilavu
3!Thirunelveli|Bharathi Kannan|Prabhu, Roja, Karan, Udhaya, Sithara, Vindhya|null|Super Good Films|/wiki/Thirunelveli_(film)
4!Vaanathaippola|Vikraman|Vijayakanth, Prabhu Deva, Livingston, Meena, Kausalya, Anju Aravind|null|Oscar Films|/wiki/Vaanathaippola
5!Eazhaiyin Sirippil|K. Subash|Prabhu Deva, Roja, Kausalya, Suvalakshmi|null|Sudhalakshmi Pictures|/wiki/Eazhaiyin_Sirippil
6!Good Luck|Manoj Bhatnagar|Prashanth, Riya Sen, Raghuvaran, Suhasini|null|Sameera Films|/wiki/Good_Luck_(2000_film)
7!Annai|Manobala|Bhanupriya, Nassar, Ramesh Khanna, Manobala|null|SG Telearts|/wiki/Annai_(2000_film)
8!Hey Ram|Kamal Haasan|Kamal Haasan, Shahrukh Khan, Rani Mukerji, Vasundhara Das|null|Raaj Kamal Films International|/wiki/Hey_Ram

In [58]:
df=df.drop(0)
df

year                       title              director  \
1   2000              Kadhal Rojavae                 K. R.   
2   2000            Kannukkul Nilavu                 Fazil   
3   2000                Thirunelveli       Bharathi Kannan   
4   2000              Vaanathaippola              Vikraman   
5   2000          Eazhaiyin Sirippil             K. Subash   
6   2000                   Good Luck       Manoj Bhatnagar   
7   2000                       Annai              Manobala   
8   2000                     Hey Ram          Kamal Haasan   
9   2000                Sudhandhiram            Raj Kapoor   
10  2000            Thai Poranthachu       R. K. Kalaimani   
11  2000                   Mugavaree           V. Z. Durai   
12  2000          Kakkai Siraginilae               P. Vasu   
13  2000                Alaipayuthey           Mani Ratnam   
14  2000              Rajakali Amman        Rama Narayanan   
15  2000             Sandhitha Velai          Ravichandran   
16  2000                   Vallarasu          N. Maharajan   
17  2000                 James Pandu                 Selva   
18  2000                  Veeranadai                Seeman   
19  2000              Athey Manithan           K. Rajeswar   
20  2000   Kandukondain Kandukondain           Rajiv Menon   
21  2000  Kandha Kadamba Kathir Vela        Rama Narayanan   
22  2000                       Kushi           S. J. Surya   
23  2000             Kannan Varuvaan              Sundar C   
24  2000   Unnai Kodu Ennai Tharuven          Kavi Kalidas   
25  2000                Magalirkkaga                Indian   
26  2000           Karisakattu Poove         Kasthuri Raja   
27  2000                        Appu               Vasanth   
28  2000               Ennamma Kannu    Sakthi Chidambaram   
29  2000                  Nagalingam           Babu Ganesh   
30  2000            Vetri Kodi Kattu                Cheran   
..   ...                         ...                   ...   
42  2000                       Maayi         Surya Prakash   
43  2000                    Bharathi     Gnana Rajasekaran   
44  2000       Chinna Chinna Kannile              Ameerjan   
45  2000                   Krodham 2            Prem Menon   
46  2000                     Sabhash             K. Subash   
47  2000        Unnai Kann Theduthey             Sundar C.   
48  2000          Budget Padmanabhan       T. P. Gajendran   
49  2000             Ninaivellam Nee    Jayaram V. Esthava   
50  2000                      Rhythm               Vasanth   
51  2000                   Ilaiyavan               T. Babu   
52  2000           Karuvelam Pookkal               Poomani   
53  2000             Puratchikkaaran      Velu Prabhakaran   
54  2000         Uyirile Kalanthathu            K. R. Jaya   
55  2000            Kannukku Kannaga            S. Dayalan   
56  2000             Palayathu Amman        Rama Narayanan   
57  2000              Priyamaanavale     K. Selva Bharathy   
58  2000                     Thenali       K. S. Ravikumar   
59  2000                    Vaanavil           Manoj Kumar   
60  2000                       Seenu               P. Vasu   
61  2000                     Anbudan           Indra Kumar   
62  2000         Vanna Thamizh Pattu               P. Vasu   
63  2000                  Snegithiye          Priyadarshan   
64  2000                  En Sakhiye             Ravi Raja   
65  2000                     Penngal           Raj Marudhu   
66  2000                 Pottu Amman       K. Rajarathinam   
67  2000                      Manasu          Abdul Rahman   
68  2000                 Aval Paavam  M. K. Arundhava Raja   
69  2000                    Ennavale             J. Suresh   
70  2000                 Manu Needhi        Thambi Ramaiah   
71  2000           Nee Enthan Vaanam          R. K. Suresh   

                                                 cast genre  \
1             George Vishnu, Pooja Kumar, Sarath Babu  null   
2                  Vijay, Shalini

In [59]:
df['cast_size']=''
for index,row in df.iterrows():
    mlink=row['cast']
    if mlink is not None:
        arr=str(mlink).split(',')
        df.at[index,'cast_size']=len(arr)

for index,row in df.iterrows():
  mlink=row['link']
  if mlink is not None:
    print(str(mlink).strip()[6:])
    dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
    strdir=tostr(dirs)
    print(str(index)+' | '+strdir)
    df.at[index,'music']=strdir


Kadhal_Rojavae


en.wikipedia.org (parse) Kadhal_Rojavae
en.wikipedia.org (imageinfo) File:Kadhal Rojavae.jpg
Kadhal Rojavae (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Kadhal Ro...
  infobox: <dict(14)> name, image, director, writer, story, starri...
  pageid: 36221524
  parsetree: <str(11328)> <root><template><title>Use dmy dates</ti...
  requests: <list(2)> parse, imageinfo
  title: Kadhal Rojavae
  wikibase: Q12980167
  wikidata_url: https://www.wikidata.org/wiki/Q12980167
  wikitext: <str(6235)> {{Use dmy dates|date=March 2019}}{{Use Ind...
}
en.wikipedia.org (parse) Kannukkul_Nilavu


['Ilaiyaraaja']
1 | Ilaiyaraaja
Kannukkul_Nilavu


en.wikipedia.org (imageinfo) File:Kannukkul Nilavu poster.jpg
Kannukkul Nilavu (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Kannukkul...
  infobox: <dict(18)> name, image, caption, director, producer, wr...
  pageid: 9649668
  parsetree: <str(18386)> <root><template><title>Use dmy dates</ti...
  requests: <list(2)> parse, imageinfo
  title: Kannukkul Nilavu
  wikibase: Q3536588
  wikidata_url: https://www.wikidata.org/wiki/Q3536588
  wikitext: <str(12562)> {{Use dmy dates|date=November 2015}}{{Use...
}
en.wikipedia.org (parse) Thirunelveli_(film)


['Ilaiyaraaja']
2 | Ilaiyaraaja
Thirunelveli_(film)


...

Ennavale (en) data
{
  infobox: <dict(13)> name, director, writer, starring, producer, ...
  pageid: 4983197
  parsetree: <str(10032)> <root><template><title>short description...
  requests: <list(1)> parse
  title: Ennavale
  wikibase: Q3644962
  wikidata_url: https://www.wikidata.org/wiki/Q3644962
  wikitext: <str(6304)> {{short description|2001 film by J. Suresh...
}
en.wikipedia.org (parse) Manu_Needhi


['S. A. Rajkumar']
69 | S. A. Rajkumar
Manu_Needhi


Manu Needhi (en) data
{
  infobox: <dict(12)> name, director, producer, writer, starring, ...
  pageid: 41853081
  parsetree: <str(7946)> <root><template><title>Use dmy dates</tit...
  requests: <list(1)> parse
  title: Manu Needhi
  wikibase: Q15711834
  wikidata_url: https://www.wikidata.org/wiki/Q15711834
  wikitext: <str(5466)> {{Use dmy dates|date=November 2015}}{{Use ...
}
en.wikipedia.org (parse) Nee_Enthan_Vaanam


['Deva', 'Deva']
70 | Deva,Deva
Nee_Enthan_Vaanam
['S. P. Venkatesh', 'Sangeetha Rajan']
71 | S. P. Venkatesh,Sangeetha Rajan


Nee Enthan Vaanam (en) data
{
  infobox: <dict(13)> name, director, producer, writer, starring, ...
  pageid: 56051539
  parsetree: <str(10146)> <root><template><title>Use dmy dates</ti...
  requests: <list(1)> parse
  title: Nee Enthan Vaanam
  wikibase: Q12930053
  wikidata_url: https://www.wikidata.org/wiki/Q12930053
  wikitext: <str(6719)> {{Use dmy dates|date=December 2017}}{{Use ...
}


In [60]:
for index,row in df.iterrows():
  mname=row['title']
  myear=row['year']
  if mname is not None:
    rating=getrating(mname,myear)
    print(mname+' | '+rating)
    df.at[index,'imdb']=rating.strip()

https://www.imdb.com/title/tt0274594/
Kadhal Rojavae | 5.2
https://www.imdb.com/title/tt0274604/
Kannukkul Nilavu | 5.9
https://www.imdb.com/title/tt8052464/
Thirunelveli | 
https://www.imdb.com/title/tt0810469/
Vaanathaippola | 7.0
https://www.imdb.com/title/tt8954404/
Eazhaiyin Sirippil | 5.5
https://www.imdb.com/title/tt8954964/
Good Luck | 4.1
https://www.imdb.com/title/tt8914606/
Annai | 
https://www.imdb.com/title/tt0222012/
Hey Ram | 7.9
https://www.imdb.com/title/tt1505127/
Sudhandhiram | 4.0
https://www.imdb.com/title/tt12160082/
Thai Poranthachu | 
https://www.imdb.com/title/tt1388441/
Mugavaree | 7.2
https://www.imdb.com/title/tt0274595/
Kakkai Siraginilae | 4.4
https://www.imdb.com/title/tt0242256/
Alaipayuthey | 
https://www.imdb.com/title/tt1604179/
Rajakali Amman | 4.0
https://www.imdb.com/title/tt4484900/
Sandhitha Velai | 5.3
https://www.imdb.com/title/tt4516596/
Vallarasu | 6.2
https://www.imdb.com/title/tt6757804/
James Pandu | 4.6
https://www.imdb.com/title/tt449861

In [61]:
df

year                       title              director  \
1   2000              Kadhal Rojavae                 K. R.   
2   2000            Kannukkul Nilavu                 Fazil   
3   2000                Thirunelveli       Bharathi Kannan   
4   2000              Vaanathaippola              Vikraman   
5   2000          Eazhaiyin Sirippil             K. Subash   
6   2000                   Good Luck       Manoj Bhatnagar   
7   2000                       Annai              Manobala   
8   2000                     Hey Ram          Kamal Haasan   
9   2000                Sudhandhiram            Raj Kapoor   
10  2000            Thai Poranthachu       R. K. Kalaimani   
11  2000                   Mugavaree           V. Z. Durai   
12  2000          Kakkai Siraginilae               P. Vasu   
13  2000                Alaipayuthey           Mani Ratnam   
14  2000              Rajakali Amman        Rama Narayanan   
15  2000             Sandhitha Velai          Ravichandran   
16  2000                   Vallarasu          N. Maharajan   
17  2000                 James Pandu                 Selva   
18  2000                  Veeranadai                Seeman   
19  2000              Athey Manithan           K. Rajeswar   
20  2000   Kandukondain Kandukondain           Rajiv Menon   
21  2000  Kandha Kadamba Kathir Vela        Rama Narayanan   
22  2000                       Kushi           S. J. Surya   
23  2000             Kannan Varuvaan              Sundar C   
24  2000   Unnai Kodu Ennai Tharuven          Kavi Kalidas   
25  2000                Magalirkkaga                Indian   
26  2000           Karisakattu Poove         Kasthuri Raja   
27  2000                        Appu               Vasanth   
28  2000               Ennamma Kannu    Sakthi Chidambaram   
29  2000                  Nagalingam           Babu Ganesh   
30  2000            Vetri Kodi Kattu                Cheran   
..   ...                         ...                   ...   
42  2000                       Maayi         Surya Prakash   
43  2000                    Bharathi     Gnana Rajasekaran   
44  2000       Chinna Chinna Kannile              Ameerjan   
45  2000                   Krodham 2            Prem Menon   
46  2000                     Sabhash             K. Subash   
47  2000        Unnai Kann Theduthey             Sundar C.   
48  2000          Budget Padmanabhan       T. P. Gajendran   
49  2000             Ninaivellam Nee    Jayaram V. Esthava   
50  2000                      Rhythm               Vasanth   
51  2000                   Ilaiyavan               T. Babu   
52  2000           Karuvelam Pookkal               Poomani   
53  2000             Puratchikkaaran      Velu Prabhakaran   
54  2000         Uyirile Kalanthathu            K. R. Jaya   
55  2000            Kannukku Kannaga            S. Dayalan   
56  2000             Palayathu Amman        Rama Narayanan   
57  2000              Priyamaanavale     K. Selva Bharathy   
58  2000                     Thenali       K. S. Ravikumar   
59  2000                    Vaanavil           Manoj Kumar   
60  2000                       Seenu               P. Vasu   
61  2000                     Anbudan           Indra Kumar   
62  2000         Vanna Thamizh Pattu               P. Vasu   
63  2000                  Snegithiye          Priyadarshan   
64  2000                  En Sakhiye             Ravi Raja   
65  2000                     Penngal           Raj Marudhu   
66  2000                 Pottu Amman       K. Rajarathinam   
67  2000                      Manasu          Abdul Rahman   
68  2000                 Aval Paavam  M. K. Arundhava Raja   
69  2000                    Ennavale             J. Suresh   
70  2000                 Manu Needhi        Thambi Ramaiah   
71  2000           Nee Enthan Vaanam          R. K. Suresh   

                                                 cast genre  \
1             George Vishnu, Pooja Kumar, Sarath Babu  null   
2                  Vijay, Shalini

# 1999

In [7]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1999'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1999=soup.find_all('table')[1:5]

In [8]:
standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

In [9]:
df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [10]:
for table in tables_1999:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='1999'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')-1

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')-1

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Opening', 'Title', 'Director', 'Cast', 'Studio']
1!The Terrorist|Santosh Sivan|Ayesha Dharker, Krishna|null|Indian Image Productions|/wiki/The_Terrorist_(1997_film)
2!Suriya Paarvai|Jagan|Arjun, Pooja|null|Modooril Creations|/wiki/Suriya_Paarvai
3!Housefull|Parthiban|Parthiban, Vikram, Suvalakshmi|null|Bioscope Film Framers|/wiki/Housefull_(1999_film)
4!Mannavaru Chinnavaru|P. N. Ramachandra Rao|Sivaji Ganesan, Arjun, Soundarya, Maheswari|null|Kalaipuli International|/wiki/Mannavaru_Chinnavaru
5!Maya|Rama Narayanan|Napoleon, Nagma|null|Sri Thenandal Films|/wiki/Maya_(1999_film)
6!Ponnu Veetukkaran|P. Vasu|Sathyaraj, Preetha|null|Ganga Gowri Productions|/wiki/Ponnu_Veetukkaran
7!Thodarum|Ramesh Khanna|Ajith Kumar, Devayani, Heera Rajagopal|null|Sree Devi Movie Makers|/wiki/Thodarum
8!Thulladha Manamum Thullum|Ezhil|Vijay, Simran|null|Super Good Films|/wiki/Thulladha_Manamum_Thullum
9!Kallazhagar|Bharathi|Vijayakanth, Laila|null|Pangaj Productions|/wiki/Kallazhagar
10!Ninaivirukkum Var

In [11]:
df=df.drop(0)
df

year                           title               director  \
1   1999                   The Terrorist          Santosh Sivan   
2   1999                  Suriya Paarvai                  Jagan   
3   1999                       Housefull              Parthiban   
4   1999            Mannavaru Chinnavaru  P. N. Ramachandra Rao   
5   1999                            Maya         Rama Narayanan   
6   1999               Ponnu Veetukkaran                P. Vasu   
7   1999                        Thodarum          Ramesh Khanna   
8   1999       Thulladha Manamum Thullum                  Ezhil   
9   1999                     Kallazhagar               Bharathi   
10  1999             Ninaivirukkum Varai              K. Subash   
11  1999                     Unnai Thedi              Sundar C.   
12  1999                     Chinna Raja     Chithra Lakshmanan   
13  1999                   Adutha Kattam                 Sarath   
14  1999                 En Swasa Kaatre             K. S. Ravi   
15  1999                Ethirum Pudhirum           V. C. Ramani   
16  1999               Endrendrum Kadhal        Manoj Bhatnagar   
17  1999               Ullathai Killathe           Vejey Kannan   
18  1999                    Chinna Durai             R. Chandra   
19  1999                      Poo Vaasam          Sree Bharathi   
20  1999                           Annan              Anu Mohan   
21  1999                       Padayappa        K. S. Ravikumar   
22  1999             Monisha En Monalisa            T. Rajendar   
23  1999                       Periyanna    S. A. Chandrasekhar   
24  1999                   Aasai Alaigal          S. Manikandan   
25  1999             Nilave Mugam Kaattu             Kalanjiyam   
26  1999               Poomagal Oorvalam        Rasu Madhuravan   
27  1999                   Poomaname Vaa          V. Azhagappan   
28  1999                           Vaali            S. J. Surya   
29  1999                       Rajasthan        R. K. Selvamani   
30  1999                 Annan Thangachi             Charan Raj   
..   ...                             ...                    ...   
54  1999         Unakkaga Ellam Unakkaga              Sundar C.   
55  1999           Manaivikku Mariyadhai       V. C. Guhanathan   
56  1999                           Mugam      Gnana Rajasekaran   
57  1999                   Nesam Pudhusu            Karthik Vel   
58  1999          Pombalainga Samacharam              Vinay Raj   
59  1999             Maravathe Kanmaniye         N. S. Madhavan   
60  1999                      Suryodayam         L. V. Aadhavan   
61  1999                           Jayam              Ravi Raja   
62  1999                           Hello      K. Selva Bharathy   
63  1999                Kanmani Unakkaga              Ravi Raja   
64  1999            Kannupada Poguthaiya        Bharathi Ganesh   
65  1999               Maanaseega Kadhal            B. S. Raman   
66  1999                       Mudhalvan                Shankar   
67  1999                            Ooty                  Anwar   
68  1999                Pudhu Kudithanam             Raghuvasan   
69  1999                       Taj Mahal           Bharathiraja   
70  1999                        Iraniyan          Vincent Selva   
71  1999  Sundari Neeyum Sundaran Naanum        A. N. Rajagopal   
72  1999                 Kudumba Sangili  P. N. Ramachandra Rao   
73  1999                           Sivan       Velu Prabhakaran   
74  1999  Thirupathi Ezhumalai Venkatesa         Rama Narayanan   
75  1999                            Time         Geetha Krishna   
76  1999          Unnaruge Naan Irundhal                  Selva   
77  1999                     Azhagarsamy              Sundar C.   
78  1999              Mudhal Etcharikkai                  Devar   
79  1999                           Sethu                   Bala   
80  1999           Aasaiyil Oru Kaditham                  Selva   
81  1999          Manam Virumbuthe Unnai        M. 

In [12]:
df['cast_size']=''
for index,row in df.iterrows():
    mlink=row['cast']
    if mlink is not None:
        arr=str(mlink).split(',')
        df.at[index,'cast_size']=len(arr)

for index,row in df.iterrows():
  mlink=row['link']
  if mlink is not None:
    print(str(mlink).strip()[6:])
    dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
    strdir=tostr(dirs)
    print(str(index)+' | '+strdir)
    df.at[index,'music']=strdir


The_Terrorist_(1997_film)


en.wikipedia.org (parse) The_Terrorist_(1997_film)
en.wikipedia.org (imageinfo) File:The Terrorist.jpg
The Terrorist (1997 film) (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:The Terro...
  infobox: <dict(14)> name, image, caption, director, writer, star...
  pageid: 30674
  parsetree: <str(11675)> <root><template><title>About</title><par...
  requests: <list(2)> parse, imageinfo
  title: The Terrorist (1997 film)
  wikibase: Q3182893
  wikidata_url: https://www.wikidata.org/wiki/Q3182893
  wikitext: <str(8531)> {{About|the 1998 Indian film|other films w...
}
en.wikipedia.org (parse) Suriya_Paarvai


['Rajamani', 'Sonu Sisupal']
1 | Rajamani,Sonu Sisupal
Suriya_Paarvai


Suriya Paarvai (en) data
{
  infobox: <dict(13)> name, director, producer, screenplay, starri...
  pageid: 39347176
  parsetree: <str(10363)> <root><template><title>Use dmy dates</ti...
  requests: <list(1)> parse
  title: Suriya Paarvai
  wikibase: Q16254542
  wikidata_url: https://www.wikidata.org/wiki/Q16254542
  wikitext: <str(5376)> {{Use dmy dates|date=November 2015}}{{Use ...
}
en.wikipedia.org (parse) Housefull_(1999_film)


['S. A. Rajkumar']
2 | S. A. Rajkumar
Housefull_(1999_film)


...

en.wikipedia.org (imageinfo) File:Manam Virumbuthe Unnai DVD cover.jpg
Manam Virumbuthe Unnai (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Manam Vir...
  infobox: <dict(15)> name, image, caption, director, producer, wr...
  pageid: 39002914
  parsetree: <str(11623)> <root><template><title>Use dmy dates</ti...
  requests: <list(2)> parse, imageinfo
  title: Manam Virumbuthe Unnai
  wikibase: Q16251234
  wikidata_url: https://www.wikidata.org/wiki/Q16251234
  wikitext: <str(6113)> {{Use dmy dates|date=November 2015}}{{Use ...
}
en.wikipedia.org (parse) Paattali


['Ilaiyaraaja']
81 | Ilaiyaraaja
Paattali


Paattali (en) data
{
  infobox: <dict(13)> name, director, screenplay, story, starring,...
  pageid: 25388193
  parsetree: <str(12692)> <root><template><title>Use Indian Englis...
  requests: <list(1)> parse
  title: Paattali
  wikibase: Q19808819
  wikidata_url: https://www.wikidata.org/wiki/Q19808819
  wikitext: <str(7677)> {{Use Indian English|date=February 2014}}{...
}
en.wikipedia.org (parse) Vaaliba Mayakkam


['S. A. Rajkumar']
82 | S. A. Rajkumar

Exception
83 | 


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}


In [13]:
for index,row in df.iterrows():
  mname=row['title']
  myear=row['year']
  if mname is not None:
    rating=getrating(mname,myear)
    print(mname+' | '+rating)
    df.at[index,'imdb']=rating.strip()

https://www.imdb.com/title/tt0169302/
The Terrorist | 7.0
https://www.imdb.com/title/tt9166956/
Suriya Paarvai | 5.5
https://www.imdb.com/title/tt0289226/
Housefull | 6.9
https://www.imdb.com/title/tt12458854/
Mannavaru Chinnavaru | 
https://www.imdb.com/title/tt4719168/
Maya | 7.3
https://www.imdb.com/title/tt0292192/
Ponnu Veetukkaran | 5.9
https://www.imdb.com/title/tt0274959/
Thodarum | 5.3
https://www.imdb.com/title/tt1517561/
Thulladha Manamum Thullum | 7.7
https://www.imdb.com/title/tt8994434/
Kallazhagar | 4.4
Ninaivirukkum Varai | 
https://www.imdb.com/title/tt2380272/
Unnai Thedi | 6.0
https://www.imdb.com/title/tt11603262/
Chinna Raja | 
https://www.imdb.com/title/tt2879080/
Adutha Kattam | 
https://www.imdb.com/title/tt0485205/
En Swasa Kaatre | 5.6
https://www.imdb.com/title/tt0251690/
Ethirum Pudhirum | 5.5
https://www.imdb.com/title/tt6829596/
Endrendrum Kadhal | 6.9
Ullathai Killathe | 
https://www.imdb.com/title/tt0280548/plotsummary
Chinna Durai | 
https://www.imdb.co

In [14]:
df

year                           title               director  \
1   1999                   The Terrorist          Santosh Sivan   
2   1999                  Suriya Paarvai                  Jagan   
3   1999                       Housefull              Parthiban   
4   1999            Mannavaru Chinnavaru  P. N. Ramachandra Rao   
5   1999                            Maya         Rama Narayanan   
6   1999               Ponnu Veetukkaran                P. Vasu   
7   1999                        Thodarum          Ramesh Khanna   
8   1999       Thulladha Manamum Thullum                  Ezhil   
9   1999                     Kallazhagar               Bharathi   
10  1999             Ninaivirukkum Varai              K. Subash   
11  1999                     Unnai Thedi              Sundar C.   
12  1999                     Chinna Raja     Chithra Lakshmanan   
13  1999                   Adutha Kattam                 Sarath   
14  1999                 En Swasa Kaatre             K. S. Ravi   
15  1999                Ethirum Pudhirum           V. C. Ramani   
16  1999               Endrendrum Kadhal        Manoj Bhatnagar   
17  1999               Ullathai Killathe           Vejey Kannan   
18  1999                    Chinna Durai             R. Chandra   
19  1999                      Poo Vaasam          Sree Bharathi   
20  1999                           Annan              Anu Mohan   
21  1999                       Padayappa        K. S. Ravikumar   
22  1999             Monisha En Monalisa            T. Rajendar   
23  1999                       Periyanna    S. A. Chandrasekhar   
24  1999                   Aasai Alaigal          S. Manikandan   
25  1999             Nilave Mugam Kaattu             Kalanjiyam   
26  1999               Poomagal Oorvalam        Rasu Madhuravan   
27  1999                   Poomaname Vaa          V. Azhagappan   
28  1999                           Vaali            S. J. Surya   
29  1999                       Rajasthan        R. K. Selvamani   
30  1999                 Annan Thangachi             Charan Raj   
..   ...                             ...                    ...   
54  1999         Unakkaga Ellam Unakkaga              Sundar C.   
55  1999           Manaivikku Mariyadhai       V. C. Guhanathan   
56  1999                           Mugam      Gnana Rajasekaran   
57  1999                   Nesam Pudhusu            Karthik Vel   
58  1999          Pombalainga Samacharam              Vinay Raj   
59  1999             Maravathe Kanmaniye         N. S. Madhavan   
60  1999                      Suryodayam         L. V. Aadhavan   
61  1999                           Jayam              Ravi Raja   
62  1999                           Hello      K. Selva Bharathy   
63  1999                Kanmani Unakkaga              Ravi Raja   
64  1999            Kannupada Poguthaiya        Bharathi Ganesh   
65  1999               Maanaseega Kadhal            B. S. Raman   
66  1999                       Mudhalvan                Shankar   
67  1999                            Ooty                  Anwar   
68  1999                Pudhu Kudithanam             Raghuvasan   
69  1999                       Taj Mahal           Bharathiraja   
70  1999                        Iraniyan          Vincent Selva   
71  1999  Sundari Neeyum Sundaran Naanum        A. N. Rajagopal   
72  1999                 Kudumba Sangili  P. N. Ramachandra Rao   
73  1999                           Sivan       Velu Prabhakaran   
74  1999  Thirupathi Ezhumalai Venkatesa         Rama Narayanan   
75  1999                            Time         Geetha Krishna   
76  1999          Unnaruge Naan Irundhal                  Selva   
77  1999                     Azhagarsamy              Sundar C.   
78  1999              Mudhal Etcharikkai                  Devar   
79  1999                           Sethu                   Bala   
80  1999           Aasaiyil Oru Kaditham                  Selva   
81  1999          Manam Virumbuthe Unnai        M. 

In [15]:
df.to_csv('1999_movies.csv')

# 1998 

In [7]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1998'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1998=soup.find_all('table')[0:4]

In [8]:
standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

In [9]:
df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [10]:
for table in tables_1998:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='1998'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')-1

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')-1

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Opening', 'Title', 'Director', 'Cast', 'Studio', 'Ref']
1!Moovendhar|Suraj|Sarath Kumar, Devayani|null|Ganga Gowri Production|/wiki/Moovendhar
2!Kaadhale Nimmadhi|Indiran|Murali, Suriya, Jeevitha Sharma, Sangeetha|null|Sivasakthi Movie Makers|/wiki/Kaadhale_Nimmadhi
3!Kizhakkum Merkkum|Kalanjiyam|Napoleon, Devayani|null|Deivanai Movie International|/wiki/Kizhakkum_Merkkum
4!Maru Malarchi|Bharathi|Mammooty, Devayani, Ranjith|null|Pangaj Productions|/wiki/Maru_Malarchi
5!Naam Iruvar Namakku Iruvar|Sundar C.|Prabhu Deva, Meena, Maheswari|null|BS Arts|/wiki/Naam_Iruvar_Namakku_Iruvar
6!Ponmanam|S. P. Rajkumar|Prabhu, Suvalakshmi, Priya Raman|null|Pyramid Films International|/wiki/Ponmanam
7!Ulavuthurai|Ramesh Selvan|Vijayakanth, Meena, Sanghavi|null|IV Cine Arts|/wiki/Ulavuthurai
8!Udhavikku Varalaamaa|Gokula Krishnan|Karthik, Devayani, Sangeetha, Anju Aravind|null|Taaj International|/wiki/Udhavikku_Varalaamaa
9!Kondattam|K. S. Ravikumar|Arjun, Gemini Ganesan, Simran, Mantra|null|Sai Lak

In [11]:
df=df.drop(0)
df

year                       title                 director  \
1   1998                  Moovendhar                    Suraj   
2   1998           Kaadhale Nimmadhi                  Indiran   
3   1998           Kizhakkum Merkkum               Kalanjiyam   
4   1998               Maru Malarchi                 Bharathi   
5   1998  Naam Iruvar Namakku Iruvar                Sundar C.   
6   1998                    Ponmanam           S. P. Rajkumar   
7   1998                 Ulavuthurai            Ramesh Selvan   
8   1998        Udhavikku Varalaamaa          Gokula Krishnan   
9   1998                   Kondattam          K. S. Ravikumar   
10  1998                Dhinamdhorum                Nagarajan   
11  1998             Sundara Pandian                 R. Raghu   
12  1998                 Swarnamukhi          K. S. Adhiyaman   
13  1998                       Velai                J. Suresh   
14  1998              Kaadhal Mannan                    Saran   
15  1998             Kalar Kanavugal                   Vishnu   
16  1998        Kangalin Vaarthaigal         Muktha S. Sundar   
17  1998      Vaettiya Madichu Kattu             K. Bhagyaraj   
18  1998     Thulli Thirintha Kaalam              Balasekaran   
19  1998            Ini Ellam Sugame             A. R. Ramesh   
20  1998                  Santhosham                  Karthik   
21  1998           Ninaithen Vandhai        K. Selva Bharathy   
22  1998           Kaathala Kaathala  Singeetam Srinivasa Rao   
23  1998       Ponnu Velayira Bhoomi              K. Krishnan   
24  1998              Veera Thalattu            Kasthuri Raja   
25  1998   Kavalai Padathe Sagodhara                   Keyaar   
26  1998                       Jeans                  Shankar   
27  1998     Vettu Onnu Thundu Rendu             C. Dinakaran   
28  1998                       Jolly               Dilipkumar   
29  1998                      Rathna             Ilanchezhian   
30  1998               Aval Varuvala               Raj Kapoor   
..   ...                         ...                      ...   
39  1998             Kalyana Galatta               Raj Kapoor   
40  1998                   Sollamale                     Sasi   
41  1998                 Nilaave Vaa             A. Venkatesh   
42  1998   Unnidathil Ennai Koduthen                 Vikraman   
43  1998             En Aasai Rasave            Kasthuri Raja   
44  1998            Thaayin Manikodi                    Arjun   
45  1998     Ellame En Pondattithaan                V. Sekhar   
46  1998      Kannedhirey Thondrinal             Ravichandran   
47  1998                  Senthooram                 Sangaman   
48  1998                Aasai Thambi            Senthilnathan   
49  1998                     Unnudan                  R. Balu   
50  1998              Desiya Geetham                   Cheran   
51  1998          En Uyir Nee Thaane           S. P. Rajkumar   
52  1998                   Simmarasi            Erode Soundar   
53  1998                  Urimai Por           Raja Rajendran   
54  1998        Veeram Vilanja Mannu            Kasthuri Raja   
55  1998             Pudhumai Pithan                    Jeeva   
56  1998             Ponmaanai Thedi                     Pari   
57  1998                 Gurupaarvai              Manoj Kumar   
58  1998             Uyirodu Uyiraga             Sushma Ahuja   
59  1998           Kumbakonam Gopalu                   Keyaar   
60  1998                   Kannathal          Bharathi Kannan   
61  1998                     Pooveli                    Selva   
62  1998                 Thalaimurai         Saravana Pandian   
63  1998          Ilamanasa Killathe         P. N. Seenivasan   
64  1998                Thanga Magal                 Ilayavan   
65  1998      Cheran Chozhan Pandian             Senthamizhan   
66  1998            Kaadhal Kavithai                Agathiyan   
67  1998                Sivappu Nila         J. George Prasad   
68  1998             Bhagavath Singh   

In [12]:
df['cast_size']=''
for index,row in df.iterrows():
    mlink=row['cast']
    if mlink is not None:
        arr=str(mlink).split(',')
        df.at[index,'cast_size']=len(arr)

for index,row in df.iterrows():
  mlink=row['link']
  if mlink is not None:
    print(str(mlink).strip()[6:])
    dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
    strdir=tostr(dirs)
    print(str(index)+' | '+strdir)
    df.at[index,'music']=strdir


Moovendhar


en.wikipedia.org (parse) Moovendhar
Moovendhar (en) data
{
  infobox: <dict(15)> name, director, producer, writer, screenplay...
  pageid: 36057875
  parsetree: <str(9355)> <root><template><title>Not to be confused...
  requests: <list(1)> parse
  title: Moovendhar
  wikibase: Q15711859
  wikidata_url: https://www.wikidata.org/wiki/Q15711859
  wikitext: <str(5958)> {{Not to be confused with|Three Crowned Ki...
}
en.wikipedia.org (parse) Kaadhale_Nimmadhi


['Sirpy']
1 | Sirpy
Kaadhale_Nimmadhi


en.wikipedia.org (imageinfo) File:Kaadhale Nimmadhi poster.jpg
Kaadhale Nimmadhi (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Kaadhale ...
  infobox: <dict(13)> name, image, starring, director, producer, w...
  pageid: 6809433
  parsetree: <str(8446)> <root><template><title>short description<...
  requests: <list(2)> parse, imageinfo
  title: Kaadhale Nimmadhi
  wikibase: Q6343243
  wikidata_url: https://www.wikidata.org/wiki/Q6343243
  wikitext: <str(6033)> {{short description|1998 film by Indhran}}...
}
en.wikipedia.org (parse) Kizhakkum_Merkkum


['Deva', 'Deva']
2 | Deva,Deva
Kizhakkum_Merkkum


...

en.wikipedia.org (parse) Kaadhal_Kavithai
en.wikipedia.org (imageinfo) File:Kaadhal Kavithai poster.jpg
Kaadhal Kavithai (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Kaadhal K...
  infobox: <dict(14)> name, image, caption, director, producer, st...
  pageid: 4555159
  parsetree: <str(14315)> <root><template><title>Use dmy dates</ti...
  requests: <list(2)> parse, imageinfo
  title: Kaadhal Kavithai
  wikibase: Q6343229
  wikidata_url: https://www.wikidata.org/wiki/Q6343229
  wikitext: <str(10046)> {{Use dmy dates|date=December 2015}}{{Use...
}
en.wikipedia.org (parse) Sivappu_Nila_(1998_film)


['Ilaiyaraaja']
66 | Ilaiyaraaja
Sivappu_Nila_(1998_film)


Sivappu Nila (en) data
{
  infobox: <dict(13)> name, director, producer, writer, starring, ...
  pageid: 45672176
  parsetree: <str(8795)> <root><template><title>Use dmy dates</tit...
  requests: <list(1)> parse
  title: Sivappu Nila
  wikibase: Q19895897
  wikidata_url: https://www.wikidata.org/wiki/Q19895897
  wikitext: <str(5801)> {{Use dmy dates|date=November 2015}}{{Use ...
}
en.wikipedia.org (parse) Bhagavath_Singh_(1998_film)


['Deva', 'Deva']
67 | Deva,Deva
Bhagavath_Singh_(1998_film)
['Deva', 'Deva']
68 | Deva,Deva


Bhagavath Singh (film) (en) data
{
  infobox: <dict(12)> name, writer, starring, director, producer, ...
  pageid: 48426779
  parsetree: <str(6459)> <root><template><title>Use Indian English...
  requests: <list(1)> parse
  title: Bhagavath Singh (film)
  wikibase: Q21646165
  wikidata_url: https://www.wikidata.org/wiki/Q21646165
  wikitext: <str(4658)> {{Use Indian English|date=November 2015}}{...
}


In [13]:
for index,row in df.iterrows():
  mname=row['title']
  myear=row['year']
  if mname is not None:
    rating=getrating(mname,myear)
    print(mname+' | '+rating)
    df.at[index,'imdb']=rating.strip()

Moovendhar | 
https://www.imdb.com/title/tt1725993/
Kaadhale Nimmadhi | 5.8
https://www.imdb.com/title/tt0316114/
Kizhakkum Merkkum | 4.2
https://www.imdb.com/title/tt0246774/
Maru Malarchi | 6.8
https://www.imdb.com/title/tt10163202/
Naam Iruvar Namakku Iruvar | 6.8
Ponmanam | 
https://www.imdb.com/title/tt10127922/
Ulavuthurai | 6.6
https://www.imdb.com/title/tt10763466/
Udhavikku Varalaamaa | 
https://www.imdb.com/title/tt11898492/
Kondattam | 
https://www.imdb.com/title/tt8718580/
Dhinamdhorum | 9.2
https://www.imdb.com/title/tt2391732/
Sundara Pandian | 6.9
https://www.imdb.com/title/tt6754698/
Swarnamukhi | 5.6
https://www.imdb.com/title/tt1509834/
Velai | 
https://www.imdb.com/title/tt2378298/
Kaadhal Mannan | 7.1
https://www.imdb.com/title/tt10160574/
Kalar Kanavugal | 
https://www.imdb.com/title/tt0292044/
Kangalin Vaarthaigal | 
https://www.imdb.com/title/tt6758346/
Vaettiya Madichu Kattu | 5.4
https://www.imdb.com/title/tt10295714/
Thulli Thirintha Kaalam | 7.2
Ini Ellam Sug

In [14]:
df

year                       title                 director  \
1   1998                  Moovendhar                    Suraj   
2   1998           Kaadhale Nimmadhi                  Indiran   
3   1998           Kizhakkum Merkkum               Kalanjiyam   
4   1998               Maru Malarchi                 Bharathi   
5   1998  Naam Iruvar Namakku Iruvar                Sundar C.   
6   1998                    Ponmanam           S. P. Rajkumar   
7   1998                 Ulavuthurai            Ramesh Selvan   
8   1998        Udhavikku Varalaamaa          Gokula Krishnan   
9   1998                   Kondattam          K. S. Ravikumar   
10  1998                Dhinamdhorum                Nagarajan   
11  1998             Sundara Pandian                 R. Raghu   
12  1998                 Swarnamukhi          K. S. Adhiyaman   
13  1998                       Velai                J. Suresh   
14  1998              Kaadhal Mannan                    Saran   
15  1998             Kalar Kanavugal                   Vishnu   
16  1998        Kangalin Vaarthaigal         Muktha S. Sundar   
17  1998      Vaettiya Madichu Kattu             K. Bhagyaraj   
18  1998     Thulli Thirintha Kaalam              Balasekaran   
19  1998            Ini Ellam Sugame             A. R. Ramesh   
20  1998                  Santhosham                  Karthik   
21  1998           Ninaithen Vandhai        K. Selva Bharathy   
22  1998           Kaathala Kaathala  Singeetam Srinivasa Rao   
23  1998       Ponnu Velayira Bhoomi              K. Krishnan   
24  1998              Veera Thalattu            Kasthuri Raja   
25  1998   Kavalai Padathe Sagodhara                   Keyaar   
26  1998                       Jeans                  Shankar   
27  1998     Vettu Onnu Thundu Rendu             C. Dinakaran   
28  1998                       Jolly               Dilipkumar   
29  1998                      Rathna             Ilanchezhian   
30  1998               Aval Varuvala               Raj Kapoor   
..   ...                         ...                      ...   
39  1998             Kalyana Galatta               Raj Kapoor   
40  1998                   Sollamale                     Sasi   
41  1998                 Nilaave Vaa             A. Venkatesh   
42  1998   Unnidathil Ennai Koduthen                 Vikraman   
43  1998             En Aasai Rasave            Kasthuri Raja   
44  1998            Thaayin Manikodi                    Arjun   
45  1998     Ellame En Pondattithaan                V. Sekhar   
46  1998      Kannedhirey Thondrinal             Ravichandran   
47  1998                  Senthooram                 Sangaman   
48  1998                Aasai Thambi            Senthilnathan   
49  1998                     Unnudan                  R. Balu   
50  1998              Desiya Geetham                   Cheran   
51  1998          En Uyir Nee Thaane           S. P. Rajkumar   
52  1998                   Simmarasi            Erode Soundar   
53  1998                  Urimai Por           Raja Rajendran   
54  1998        Veeram Vilanja Mannu            Kasthuri Raja   
55  1998             Pudhumai Pithan                    Jeeva   
56  1998             Ponmaanai Thedi                     Pari   
57  1998                 Gurupaarvai              Manoj Kumar   
58  1998             Uyirodu Uyiraga             Sushma Ahuja   
59  1998           Kumbakonam Gopalu                   Keyaar   
60  1998                   Kannathal          Bharathi Kannan   
61  1998                     Pooveli                    Selva   
62  1998                 Thalaimurai         Saravana Pandian   
63  1998          Ilamanasa Killathe         P. N. Seenivasan   
64  1998                Thanga Magal                 Ilayavan   
65  1998      Cheran Chozhan Pandian             Senthamizhan   
66  1998            Kaadhal Kavithai                Agathiyan   
67  1998                Sivappu Nila         J. George Prasad   
68  1998             Bhagavath Singh   

In [15]:
df.to_csv('1998_films.csv')

# 1997

In [19]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1997'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1997=soup.find_all('table')[0:1]

In [20]:
standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

In [23]:
df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [24]:
for table in tables_1997:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='1997'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')-1

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')-1

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')-1
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Opening', 'Title', 'Director', 'Cast', 'Production', 'Music Director']
1!Sakthi|R. Raghu Raj|Vineeth, Yuvarani|null|Gentleman Film International|/wiki/Sakthi_(1997_film)
2!Dharma Chakkaram|K. S. Ravikumar|Vijayakanth, Deepti Bhatnagar, Rambha|null|Lakshmi Movie Makers|/wiki/Dharma_Chakkaram
3!Iruvar|Mani Ratnam|Mohanlal, Aishwarya Rai, Prakash Raj, Gouthami, Revathi, Tabu|null|Madras Talkies|/wiki/Iruvar
4!Kaalamellam Kaathiruppen|R. Sundarrajan|Vijay, Dimple|null|M. G. Pictures|/wiki/Kaalamellam_Kaathiruppen
5!Kathirunda Kadhal|Vijeshwaran|Arun Vijay, Suvalakshmi|null|Ganga Gowri Productions|/wiki/Kathirunda_Kadhal
6!Minsara Kanavu|Rajiv Menon|Prabhu Deva, Arvind Swamy, Kajol|null|AVM Productions|/wiki/Minsara_Kanavu
7!Nalla Manusukkaran|Jayarajendran|Pandiarajan, Jayaragini|null|Annai Sree Baalaambikai Creations|/wiki/Nalla_Manusukkaran
8!Om Saravana Bhava|N. Krishnaswamy|Sowmyan, Vidyasri|null||
9!Periya Thambi|Chitra Lakshmanan|Prabhu, Nagma|null|Gayathri Films|/wiki/Periya_Thamb

In [25]:
df=df.drop(0)

In [26]:
df

year                       title             director  \
1   1997                      Sakthi         R. Raghu Raj   
2   1997            Dharma Chakkaram      K. S. Ravikumar   
3   1997                      Iruvar          Mani Ratnam   
4   1997    Kaalamellam Kaathiruppen       R. Sundarrajan   
5   1997           Kathirunda Kadhal          Vijeshwaran   
6   1997              Minsara Kanavu          Rajiv Menon   
7   1997          Nalla Manusukkaran        Jayarajendran   
8   1997           Om Saravana Bhava      N. Krishnaswamy   
9   1997               Periya Thambi    Chitra Lakshmanan   
10  1997           Bharathi Kannamma               Cheran   
11  1997               Gopura Deepam            Ramarajan   
12  1997                       Nesam            K. Subash   
13  1997                    Pudhayal                Selva   
14  1997  Kaalamellam Kadhal Vaazhga              R. Balu   
15  1997              Ettupatti Rasa        Kasthuri Raja   
16  1997               Parama Pithaa         K. Alexander   
17  1997              Vaimaye Vellum              P. Vasu   
18  1997                     Mannava         L. M. Balaji   
19  1997               Maradha Uravu        Thiruvarangan   
20  1997                   Adhipathi             Balaraja   
21  1997                  Aravindhan         T. Nagarajan   
22  1997           Mappillai Gounder          Manivasagam   
23  1997             Vivasaayi Magan            Ramarajan   
24  1997                 Arunachalam            Sundar C.   
25  1997              Thaali Pudhusu                Keyar   
26  1997       Dhinamum Ennai Gavani         A. R. Ramesh   
27  1997                    My India      K. Alex Pandian   
28  1997                       Raasi          Muraliappas   
29  1997                      Vallal            Rajkapoor   
30  1997                      Sishya                Selva   
..   ...                         ...                  ...   
48  1997              Raman Abdullah        Balu Mahendra   
49  1997                   Abhimanyu            K. Subash   
50  1997             Adimai Changili      R. K. Selvamani   
51  1997           Kalyana Vaibhogam           N. Rathnam   
52  1997                Nerrukku Ner              Vasanth   
53  1997       Adrasakkai Adrasakkai     V. C. Guhanathan   
54  1997                 Ganga Gowri          Madheswaran   
55  1997             Naalaya Udhayam        K. R. Ranjith   
56  1997         Aahaa Enna Porutham       C. Ranganathan   
57  1997                   Paththini              P. Vasu   
58  1997                      Samrat         C. Dinakaran   
59  1997                    Aahaa..!       Suresh Krishna   
60  1997             Periya Manushan        Guru Dhanapal   
61  1997                  Porkkaalam               Cheran   
62  1997                   Ratchagan        Praveen Kanth   
63  1997          Thedinen Vanthathu           Ravi Varma   
64  1997      Themmangu Paattukaaran        Gangai Amaren   
65  1997                      Vasuki        Kasthuri Raja   
66  1997                  Vidukadhai            Agathiyan   
67  1997                 Janakiraman            Sundar C.   
68  1997                    Thadayam  Ramesh Balakrishnan   
69  1997       Veerapandi Kottayiley          Chandernath   
70  1997                     Kadavul     Velu Prabhakaran   
71  1997                 Roja Malare          Jayamurugan   
72  1997                Thambi Durai        Senthilnathan   
73  1997                    Arasiyal      R. K. Selvamani   
74  1997                  Poochudava          Udayasankar   
75  1997         Rettai Jadai Vayasu         C. Sivakumar   
76  1997       Kadhalukku Mariyadhai                Fazil   
77  1997                   Pudhalvan      Selva Vinayagam   

                                                 cast genre  \
1                                   Vineeth, Yuvarani  null   
2               Vijayakanth, Deepti Bhatnagar, Rambha  null   
3   Mohanlal, Aishwarya Rai, Pra

In [27]:
df['cast_size']=''
for index,row in df.iterrows():
    mlink=row['cast']
    if mlink is not None:
        arr=str(mlink).split(',')
        df.at[index,'cast_size']=len(arr)

for index,row in df.iterrows():
  mlink=row['link']
  if mlink is not None:
    print(str(mlink).strip()[6:])
    dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
    strdir=tostr(dirs)
    print(str(index)+' | '+strdir)
    df.at[index,'music']=strdir


Sakthi_(1997_film)


en.wikipedia.org (parse) Sakthi_(1997_film)
en.wikipedia.org (imageinfo) File:Sakthi DVD cover.jpg
Sakthi (1997 film) (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Sakthi DV...
  infobox: <dict(16)> name, image, caption, director, producer, st...
  pageid: 45638323
  parsetree: <str(8787)> <root><template><title>Use dmy dates</tit...
  requests: <list(2)> parse, imageinfo
  title: Sakthi (1997 film)
  wikibase: Q19896053
  wikidata_url: https://www.wikidata.org/wiki/Q19896053
  wikitext: <str(5330)> {{Use dmy dates|date=October 2015}}{{Use I...
}
en.wikipedia.org (parse) Dharma_Chakkaram


['R. Anandh']
1 | R. Anandh
Dharma_Chakkaram


en.wikipedia.org (imageinfo) File:Dharma Chakkaram poster.jpg
Dharma Chakkaram (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Dharma Ch...
  infobox: <dict(17)> name, image, caption, director, producer, wr...
  pageid: 36156432
  parsetree: <str(7743)> <root><template><title>Use dmy dates</tit...
  requests: <list(2)> parse, imageinfo
  title: Dharma Chakkaram
  wikibase: Q5269221
  wikidata_url: https://www.wikidata.org/wiki/Q5269221
  wikitext: <str(4494)> {{Use dmy dates|date=October 2015}}{{Use I...
}
en.wikipedia.org (parse) Iruvar


['Deva', 'Deva']
2 | Deva,Deva
Iruvar


...

Rettai Jadai Vayasu (en) data
{
  infobox: <dict(12)> name, director, producer, starring, music, c...
  pageid: 6795113
  parsetree: <str(9550)> <root><template><title>Use Indian English...
  requests: <list(1)> parse
  title: Rettai Jadai Vayasu
  wikibase: Q3429439
  wikidata_url: https://www.wikidata.org/wiki/Q3429439
  wikitext: <str(4671)> {{Use Indian English|date=November 2015}}{...
}
en.wikipedia.org (parse) Kadhalukku_Mariyadhai


['Deva', 'Deva']
75 | Deva,Deva
Kadhalukku_Mariyadhai


en.wikipedia.org (imageinfo) File:Kadhalukku Mariyadhai poster.jpg
Kadhalukku Mariyadhai (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Kadhalukk...
  infobox: <dict(19)> name, image, caption, director, producer, wr...
  pageid: 7903143
  parsetree: <str(15135)> <root><template><title>short description...
  requests: <list(2)> parse, imageinfo
  title: Kadhalukku Mariyadhai
  wikibase: Q3465274
  wikidata_url: https://www.wikidata.org/wiki/Q3465274
  wikitext: <str(10533)> {{short description|1997 film directed by...
}
en.wikipedia.org (parse) Pudhalvan


['Ilayaraja']
76 | Ilayaraja
Pudhalvan
['Deva', 'Deva']
77 | Deva,Deva


Pudhalvan (en) data
{
  infobox: <dict(15)> name, director, producer, writer, story, scr...
  pageid: 45449833
  parsetree: <str(7449)> <root><template><title>Use dmy dates</tit...
  requests: <list(1)> parse
  title: Pudhalvan
  wikibase: Q19895306
  wikidata_url: https://www.wikidata.org/wiki/Q19895306
  wikitext: <str(4481)> {{Use dmy dates|date=December 2015}}{{Use ...
}


In [28]:
for index,row in df.iterrows():
  mname=row['title']
  myear=row['year']
  if mname is not None:
    rating=getrating(mname,myear)
    print(mname+' | '+rating)
    df.at[index,'imdb']=rating.strip()

https://www.imdb.com/title/tt7471040/
Sakthi | 
https://www.imdb.com/title/tt4488906/
Dharma Chakkaram | 
https://www.imdb.com/title/tt0119385/
Iruvar | 8.5
https://www.imdb.com/title/tt6827136/
Kaalamellam Kaathiruppen | 6.5
https://www.imdb.com/title/tt8976960/
Kathirunda Kadhal | 
https://www.imdb.com/title/tt0274789/
Minsara Kanavu | 7.0
https://www.imdb.com/title/tt10157778/
Nalla Manusukkaran | 
https://www.imdb.com/name/nm0793851/
Om Saravana Bhava | 
https://www.imdb.com/name/nm0694893/
Periya Thambi | 
https://www.imdb.com/title/tt6442354/
Bharathi Kannamma | 7.2
https://www.imdb.com/title/tt8954992/
Gopura Deepam | 
https://www.imdb.com/title/tt2373644/
Nesam | 6.5
https://www.imdb.com/title/tt0246877/
Pudhayal | 6.0
https://www.imdb.com/title/tt8976818/
Kaalamellam Kadhal Vaazhga | 6.3
https://www.imdb.com/title/tt8948586/
Ettupatti Rasa | 5.8
https://www.imdb.com/title/tt11209926/
Parama Pithaa | 
https://www.imdb.com/title/tt10219296/
Vaimaye Vellum | 
https://www.imdb.com

In [29]:
df

year                       title             director  \
1   1997                      Sakthi         R. Raghu Raj   
2   1997            Dharma Chakkaram      K. S. Ravikumar   
3   1997                      Iruvar          Mani Ratnam   
4   1997    Kaalamellam Kaathiruppen       R. Sundarrajan   
5   1997           Kathirunda Kadhal          Vijeshwaran   
6   1997              Minsara Kanavu          Rajiv Menon   
7   1997          Nalla Manusukkaran        Jayarajendran   
8   1997           Om Saravana Bhava      N. Krishnaswamy   
9   1997               Periya Thambi    Chitra Lakshmanan   
10  1997           Bharathi Kannamma               Cheran   
11  1997               Gopura Deepam            Ramarajan   
12  1997                       Nesam            K. Subash   
13  1997                    Pudhayal                Selva   
14  1997  Kaalamellam Kadhal Vaazhga              R. Balu   
15  1997              Ettupatti Rasa        Kasthuri Raja   
16  1997               Parama Pithaa         K. Alexander   
17  1997              Vaimaye Vellum              P. Vasu   
18  1997                     Mannava         L. M. Balaji   
19  1997               Maradha Uravu        Thiruvarangan   
20  1997                   Adhipathi             Balaraja   
21  1997                  Aravindhan         T. Nagarajan   
22  1997           Mappillai Gounder          Manivasagam   
23  1997             Vivasaayi Magan            Ramarajan   
24  1997                 Arunachalam            Sundar C.   
25  1997              Thaali Pudhusu                Keyar   
26  1997       Dhinamum Ennai Gavani         A. R. Ramesh   
27  1997                    My India      K. Alex Pandian   
28  1997                       Raasi          Muraliappas   
29  1997                      Vallal            Rajkapoor   
30  1997                      Sishya                Selva   
..   ...                         ...                  ...   
48  1997              Raman Abdullah        Balu Mahendra   
49  1997                   Abhimanyu            K. Subash   
50  1997             Adimai Changili      R. K. Selvamani   
51  1997           Kalyana Vaibhogam           N. Rathnam   
52  1997                Nerrukku Ner              Vasanth   
53  1997       Adrasakkai Adrasakkai     V. C. Guhanathan   
54  1997                 Ganga Gowri          Madheswaran   
55  1997             Naalaya Udhayam        K. R. Ranjith   
56  1997         Aahaa Enna Porutham       C. Ranganathan   
57  1997                   Paththini              P. Vasu   
58  1997                      Samrat         C. Dinakaran   
59  1997                    Aahaa..!       Suresh Krishna   
60  1997             Periya Manushan        Guru Dhanapal   
61  1997                  Porkkaalam               Cheran   
62  1997                   Ratchagan        Praveen Kanth   
63  1997          Thedinen Vanthathu           Ravi Varma   
64  1997      Themmangu Paattukaaran        Gangai Amaren   
65  1997                      Vasuki        Kasthuri Raja   
66  1997                  Vidukadhai            Agathiyan   
67  1997                 Janakiraman            Sundar C.   
68  1997                    Thadayam  Ramesh Balakrishnan   
69  1997       Veerapandi Kottayiley          Chandernath   
70  1997                     Kadavul     Velu Prabhakaran   
71  1997                 Roja Malare          Jayamurugan   
72  1997                Thambi Durai        Senthilnathan   
73  1997                    Arasiyal      R. K. Selvamani   
74  1997                  Poochudava          Udayasankar   
75  1997         Rettai Jadai Vayasu         C. Sivakumar   
76  1997       Kadhalukku Mariyadhai                Fazil   
77  1997                   Pudhalvan      Selva Vinayagam   

                                                 cast genre  \
1                                   Vineeth, Yuvarani  null   
2               Vijayakanth, Deepti Bhatnagar, Rambha  null   
3   Mohanlal, Aishwarya Rai, Pra

In [30]:
df.to_csv('1997_movies.csv')

# 1996

In [7]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1996'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1996=soup.find_all('table')[0:1]

In [8]:
standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

In [9]:
df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [11]:
for table in tables_1996:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='1996'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')-1

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')-1

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')-1
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Opening', 'Title', 'Director', 'Cast', 'Production', 'Music Director']
1!Aruva Velu|P. S. Bharathi Kannan|Nassar, Urvashi|null|Goodwill Movies|/wiki/Aruva_Velu
2!Coimbatore Mappillai|C. Ranganathan|Vijay, Sanghavi|null|Shree Vijayalakshmi Movieland|/wiki/Coimbatore_Mappillai
3!Kizhakku Mugam|M. Anna Thurai|Karthik, Reshma|null|Thai Films International|/wiki/Kizhakku_Mugam
4!Love Birds|P. Vasu|Prabhu Deva, Nagma|null|Pyramid Films|/wiki/Love_Birds_(1996_film)
5!Parambarai|K. S. Ravikumar|Prabhu, Roja|null|Malar Films|/wiki/Parambarai
6!Thayagam|A. R. Ramesh|Vijayakanth, Ranjitha|null|Seranaadu Movie Creations|/wiki/Thayagam
7!Thirumbi Paar|Rama Narayanan|Saravanan, Yuvarani|null|Sri Thenandal Films|/wiki/Thirumbi_Paar_(1996_film)
8!Ullathai Allitha|Sundar C.|Karthik, Rambha, Goundamani|null|Ganga Gowri Productions|/wiki/Ullathai_Allitha
9!Vaanmathi|Agathiyan|Ajith Kumar, Swathi|null|Sivashakthi Movie Makers|/wiki/Vaanmathi
10!Mahaprabhu|A. Venkatesh|Sarath Kumar, Sukanya, Vineetha|nul

In [12]:
df=df.drop(0)
df

year                     title                   director  \
1   1996                Aruva Velu      P. S. Bharathi Kannan   
2   1996      Coimbatore Mappillai             C. Ranganathan   
3   1996            Kizhakku Mugam             M. Anna Thurai   
4   1996                Love Birds                    P. Vasu   
5   1996                Parambarai            K. S. Ravikumar   
6   1996                  Thayagam               A. R. Ramesh   
7   1996             Thirumbi Paar             Rama Narayanan   
8   1996          Ullathai Allitha                  Sundar C.   
9   1996                 Vaanmathi                  Agathiyan   
10  1996                Mahaprabhu               A. Venkatesh   
11  1996          Vaikarai Pookkal          E. Mu. Vetrivalan   
12  1996     Amman Kovil Vaasalile                  Ramarajan   
13  1996          Meendum Savithri                       Visu   
14  1996           Nattupura Pattu              Kasthuri Raja   
15  1996            Poove Unakkaga                   Vikraman   
16  1996                Musthaffaa              R. Aravindraj   
17  1996           Kalloori Vaasal                  Pavithran   
18  1996       Vaazhga Jananayagam                 E. Ramdoss   
19  1996       Summa Irunga Machan               S. N. Prasad   
20  1996               Suriya Raja          Senthamizh Selvan   
21  1996        Puthiya Parasakthi                Gowri Rajan   
22  1996           Vasantha Vaasal                       M. R   
23  1996              Irattai Roja                      K. R.   
24  1996  Mappillai Manasu Poopola  Kuruvikkarambai Shanmugam   
25  1996       Maanbumigu Maanavan        S. A. Chandrasekhar   
26  1996        Kaalam Maari Pochu                  V. Sekhar   
27  1996              Karuppu Roja                  J. Paneer   
28  1996                    Rajali           Velu Prabhakaran   
29  1996                 Sengottai            C. V. Sasikumar   
30  1996                    Indian                    Shankar   
..   ...                       ...                        ...   
48  1996                Vishwanath                 K. Goutham   
49  1996              Kadhal Desam                     Kathir   
50  1996                 Mettukudi                  Sundar C.   
51  1996                  Manikkam              K. V. Pandian   
52  1996                Sivasakthi             Suresh Krissna   
53  1996           Vetri Vinayagar                 K. Shankar   
54  1996          Namma Ooru Raasa                  Ramarajan   
55  1996           Ilamai Rojakkal              S. Manikandan   
56  1996                    Subash           R. V. Udayakumar   
57  1996                Parivattam            T. K. Rajendran   
58  1996     Kodugal Illatha Kolam                    L. Balu   
59  1996                Tata Birla             C. Ranganathan   
60  1996                Andha Naal              Vishnu Priyan   
61  1996               Thuraimugam               K. Rajeshwar   
62  1996                 Alexander                      K. R.   
63  1996           Avvai Shanmughi            K. S. Ravikumar   
64  1996        Gokulathil Seethai                  Agathiyan   
65  1996                     Kalki             K. Balachander   
66  1996                 Mr. Romeo                 K. S. Ravi   
67  1996                   Nethaji                   G. Kicha   
68  1996          Panchalankurichi                     Seeman   
69  1996              Senathipathi             M. Rathnakumar   
70  1996                 Vasantham           T. S. Ravivarman   
71  1996               Gnanapazham              R. P. Vishwam   
72  1996             Gopala Gopala                Pandiarajan   
73  1996                   Poomani             Mu. Kalanjiyam   
74  1996         Purushan Pondatti          N. K. Viswanathan   
75  1996                     Selva               A. Venkatesh   
76  1996            Pudhiya Ulagam                   Ilayavan   
77  1996      Take It Easy Urvashi     

In [13]:
df['cast_size']=''
for index,row in df.iterrows():
    mlink=row['cast']
    if mlink is not None:
        arr=str(mlink).split(',')
        df.at[index,'cast_size']=len(arr)

for index,row in df.iterrows():
  mlink=row['link']
  if mlink is not None:
    print(str(mlink).strip()[6:])
    dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
    strdir=tostr(dirs)
    print(str(index)+' | '+strdir)
    df.at[index,'music']=strdir


Aruva_Velu


en.wikipedia.org (parse) Aruva_Velu
Aruva Velu (en) data
{
  infobox: <dict(16)> name, director, producer, story, screenplay,...
  pageid: 37888853
  parsetree: <str(8339)> <root><template><title>short description<...
  requests: <list(1)> parse
  title: Aruva Velu
  wikibase: Q4802326
  wikidata_url: https://www.wikidata.org/wiki/Q4802326
  wikitext: <str(4955)> {{short description|1996 film by Bharathi ...
}
en.wikipedia.org (parse) Coimbatore_Mappillai


['Adithyan']
1 | Adithyan
Coimbatore_Mappillai


en.wikipedia.org (imageinfo) File:Coimbatore Mappillai.jpg


Exception
2 | 
Kizhakku_Mugam


...

Selva (film) (en) data
{
  infobox: <dict(17)> name, director, writer, screenplay, story, s...
  pageid: 9678000
  parsetree: <str(6474)> <root><template><title>Use dmy dates</tit...
  requests: <list(1)> parse
  title: Selva (film)
  wikibase: Q7448847
  wikidata_url: https://www.wikidata.org/wiki/Q7448847
  wikitext: <str(4196)> {{Use dmy dates|date=October 2015}}{{Use I...
}
en.wikipedia.org (parse) Pudhiya Ulagam


['Sirpy']
75 | Sirpy



API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Take_It_Easy_Urvashi


Exception
76 | 
Take_It_Easy_Urvashi
['Soundaryan']
77 | Soundaryan


Take It Easy Urvashi (en) data
{
  infobox: <dict(13)> name, director, producer, writer, starring, ...
  pageid: 62304620
  parsetree: <str(11588)> <root><template><title>Use Indian Englis...
  requests: <list(1)> parse
  title: Take It Easy Urvashi
  wikibase: Q77895255
  wikidata_url: https://www.wikidata.org/wiki/Q77895255
  wikitext: <str(7052)> {{Use Indian English|date=November 2019}}{...
}


In [14]:
for index,row in df.iterrows():
  mname=row['title']
  myear=row['year']
  if mname is not None:
    rating=getrating(mname,myear)
    print(mname+' | '+rating)
    df.at[index,'imdb']=rating.strip()

Aruva Velu | 
https://www.imdb.com/title/tt6823220/
Coimbatore Mappillai | 6.4
https://www.imdb.com/title/tt10681962/
Kizhakku Mugam | 
https://www.imdb.com/title/tt0353675/
Love Birds | 5.1
https://www.imdb.com/title/tt10163264/
Parambarai | 
https://www.imdb.com/title/tt6945658/
Thayagam | 5.3
https://www.imdb.com/title/tt1445778/
Thirumbi Paar | 6.3
https://www.imdb.com/title/tt2326059/
Ullathai Allitha | 7.7
https://www.imdb.com/title/tt0984243/
Vaanmathi | 5.8
https://www.imdb.com/search/title/?year=1996,1996&title_type=feature&explore=languages&view=advanced&languages=ta&ref_=adv_explore_rhs
Mahaprabhu | 
https://www.imdb.com/title/tt10846126/
Vaikarai Pookkal | 
https://www.imdb.com/title/tt8913842/
Amman Kovil Vaasalile | 
https://www.imdb.com/title/tt6713424/
Meendum Savithri | 6.0
https://www.imdb.com/title/tt0305881/
Nattupura Pattu | 5.0
https://www.imdb.com/title/tt2325915/
Poove Unakkaga | 7.8
https://www.imdb.com/title/tt6749820/
Musthaffaa | 
https://www.imdb.com/title/

In [15]:
df

year                     title                   director  \
1   1996                Aruva Velu      P. S. Bharathi Kannan   
2   1996      Coimbatore Mappillai             C. Ranganathan   
3   1996            Kizhakku Mugam             M. Anna Thurai   
4   1996                Love Birds                    P. Vasu   
5   1996                Parambarai            K. S. Ravikumar   
6   1996                  Thayagam               A. R. Ramesh   
7   1996             Thirumbi Paar             Rama Narayanan   
8   1996          Ullathai Allitha                  Sundar C.   
9   1996                 Vaanmathi                  Agathiyan   
10  1996                Mahaprabhu               A. Venkatesh   
11  1996          Vaikarai Pookkal          E. Mu. Vetrivalan   
12  1996     Amman Kovil Vaasalile                  Ramarajan   
13  1996          Meendum Savithri                       Visu   
14  1996           Nattupura Pattu              Kasthuri Raja   
15  1996            Poove Unakkaga                   Vikraman   
16  1996                Musthaffaa              R. Aravindraj   
17  1996           Kalloori Vaasal                  Pavithran   
18  1996       Vaazhga Jananayagam                 E. Ramdoss   
19  1996       Summa Irunga Machan               S. N. Prasad   
20  1996               Suriya Raja          Senthamizh Selvan   
21  1996        Puthiya Parasakthi                Gowri Rajan   
22  1996           Vasantha Vaasal                       M. R   
23  1996              Irattai Roja                      K. R.   
24  1996  Mappillai Manasu Poopola  Kuruvikkarambai Shanmugam   
25  1996       Maanbumigu Maanavan        S. A. Chandrasekhar   
26  1996        Kaalam Maari Pochu                  V. Sekhar   
27  1996              Karuppu Roja                  J. Paneer   
28  1996                    Rajali           Velu Prabhakaran   
29  1996                 Sengottai            C. V. Sasikumar   
30  1996                    Indian                    Shankar   
..   ...                       ...                        ...   
48  1996                Vishwanath                 K. Goutham   
49  1996              Kadhal Desam                     Kathir   
50  1996                 Mettukudi                  Sundar C.   
51  1996                  Manikkam              K. V. Pandian   
52  1996                Sivasakthi             Suresh Krissna   
53  1996           Vetri Vinayagar                 K. Shankar   
54  1996          Namma Ooru Raasa                  Ramarajan   
55  1996           Ilamai Rojakkal              S. Manikandan   
56  1996                    Subash           R. V. Udayakumar   
57  1996                Parivattam            T. K. Rajendran   
58  1996     Kodugal Illatha Kolam                    L. Balu   
59  1996                Tata Birla             C. Ranganathan   
60  1996                Andha Naal              Vishnu Priyan   
61  1996               Thuraimugam               K. Rajeshwar   
62  1996                 Alexander                      K. R.   
63  1996           Avvai Shanmughi            K. S. Ravikumar   
64  1996        Gokulathil Seethai                  Agathiyan   
65  1996                     Kalki             K. Balachander   
66  1996                 Mr. Romeo                 K. S. Ravi   
67  1996                   Nethaji                   G. Kicha   
68  1996          Panchalankurichi                     Seeman   
69  1996              Senathipathi             M. Rathnakumar   
70  1996                 Vasantham           T. S. Ravivarman   
71  1996               Gnanapazham              R. P. Vishwam   
72  1996             Gopala Gopala                Pandiarajan   
73  1996                   Poomani             Mu. Kalanjiyam   
74  1996         Purushan Pondatti          N. K. Viswanathan   
75  1996                     Selva               A. Venkatesh   
76  1996            Pudhiya Ulagam                   Ilayavan   
77  1996      Take It Easy Urvashi     

In [16]:
df.to_csv("1996_movies.csv")

# 1995

In [21]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1995'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1995=soup.find_all('table')[1:2]

In [22]:
standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

In [23]:
df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [24]:
for table in tables_1995:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='1995'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')-1

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')-1

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')-1
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Opening', 'Title', 'Director', 'Cast', 'Production', 'Music Director']
1!Karuppu Nila|R. Sundarrajan|Vijayakanth, Kushboo, Ranjitha|null|Ravuther Films|/wiki/Karuppu_Nila
2!Engirundho Vandhan|Santhana Bharathi|Sivaji Ganesan, Sathyaraj, Roja, Aamani|null|V. S. R. Pictures|/wiki/Engirundho_Vandhan
3!Baashha|Suresh Krissna|Rajinikanth, Nagma|null|Sathya Movies|/wiki/Baashha
4!Kattumarakaran|P. Vasu|Prabhu, Eva Grover, Sanghavi|null|A. G. S. Movies|/wiki/Kattumarakaran
5!Naan Petha Magane|V. Sekhar|Nizhalgal Ravi, Raadhika, Urvashi, Manorama|null|Manithalayam Productions|/wiki/Naan_Petha_Magane
6!Oru Oorla Oru Rajakumari|K. Bhagyaraj|K. Bhagyaraj, Meena|null|Indrani Movies|/wiki/Oru_Oorla_Oru_Rajakumari
7!Sathi Leelavathi|Balu Mahendra|Kamal Haasan, Ramesh Aravind, Kalpana, Heera, Kovai Sarala|null|Raaj Kamal Films International|/wiki/Sathi_Leelavathi_(1995_film)
8!Veluchami|Arul|Sarath Kumar, Vineetha, Shruthi|null|Lakshmi Movie Makers|/wiki/Veluchami
9!Gangai Karai Paattu|Manivannan|V

82!Ayudha Poojai|C. Sivakumar|Arjun, Urvashi, Roja|null|Bhagyam Cine Combines|/wiki/Ayudha_Poojai
83!Ilaya Raagam|P. R. Devaraj|Jothi Shankar, Kavitha|null|Pournami Film International|
84!Maaman Magal|Guru Dhanapal|Sathyaraj, Meena, Goundamani|null|A. G. S. Movies|/wiki/Maaman_Magal_(1995_film)
85!Jameen Kottai|Ramchandar|Kalaipuli G. Sekaran, Seetha|null|Thiraiparavai|/wiki/Jameen_Kottai
86!Varraar Sandiyar|N. K. Vishwanathan|Prem, Kushboo|null|Sree Rajakaliamman Cine Arts|/wiki/Varraar_Sandiyar
87!Ilaimaikku Oru Etcharikkai|S. Manikandan|Bhaskaran, Poovitha|null|Sanjaykumar Films|
88!Murai Mappillai|Sundar C.|Arunkumar, Kirthika|null|Anbalaya Films|/wiki/Murai_Mappillai
89!Thotta Chinungi|K. S. Adhiyaman|Karthik, Revathi, Raghuvaran, Rohini, Nagendra Prasad, Devayani|null|Span Vision|/wiki/Thotta_Chinungi
90!Mannai Thottu Kumbidanum|R. Umashankar|Selva, Keerthana|null|Sree Renukaambika Movies|/wiki/Mannai_Thottu_Kumbidanum
91!Dear Son Maruthu|M. Solai Rajendran|Rahman, Sivakumar, Sou

In [25]:
df=df.drop(0)

In [26]:
df

year                       title             director  \
1   1995                Karuppu Nila       R. Sundarrajan   
2   1995          Engirundho Vandhan    Santhana Bharathi   
3   1995                     Baashha       Suresh Krissna   
4   1995              Kattumarakaran              P. Vasu   
5   1995           Naan Petha Magane            V. Sekhar   
6   1995    Oru Oorla Oru Rajakumari         K. Bhagyaraj   
7   1995            Sathi Leelavathi        Balu Mahendra   
8   1995                   Veluchami                 Arul   
9   1995         Gangai Karai Paattu           Manivannan   
10  1995              Mudhal Udhayam           Prithviraj   
11  1995               Paattu Padava  B. R. Vijayalakshmi   
12  1995             Valli Vara Pora     S. V. Solai Raja   
13  1995               Raja Muthirai      R. K. Selvamani   
14  1995                        Deva  S. A. Chandrasekhar   
15  1995              Kizhakku Malai        Ganesh Ramana   
16  1995                Muthu Kaalai      Gokula Krishnan   
17  1995          Thottil Kuzhandhai     S. P. Muthuraman   
18  1995                 Chinna Mani      M. Thilakarajan   
19  1995                    Kalyanam         R. Narayanan   
20  1995        Manathile Oru Paattu           Gnanamozhi   
21  1995              Puthiya Aatchi     Velu Prabhakaran   
22  1995                      Bombay          Mani Ratnam   
23  1995      Muthukulikka Vaariyala      K. S. Ravikumar   
24  1995              Raja Enga Raja     T. Vijayasingaam   
25  1995                     Thondan            Karvannan   
26  1995                  Aanazhagan          Thyagarajan   
27  1995     Dharmangal Sirikkinrana        A. Kasilingam   
28  1995            Udhavum Karangal      Shanmuga Priyan   
29  1995                      Coolie              P. Vasu   
30  1995          Ellame En Rasathan             Rajkiran   
..   ...                         ...                  ...   
62  1995                      Vishnu  S. A. Chandrasekhar   
63  1995               Nadodi Mannan          Manivasagam   
64  1995                   Raasaiyya            P. Kannan   
65  1995             Periya Kudumbam      K. S. Ravikumar   
66  1995                       Aasai              Vasanth   
67  1995                  Mr. Madras              P. Vasu   
68  1995                    Kolangal           I. V. Sasi   
69  1995                 Sindhu Bath           Balu Anand   
70  1995         Mannukku Mariyadhai        T. R. Vijayan   
71  1995       Thaikulame Thaikulame          N. Murugesh   
72  1995               Aakaya Pookal           Deenadayal   
73  1995                   Ilavarasi      Selva Vinayagam   
74  1995                Chandralekha           Nambhiraaj   
75  1995                Kuruthipunal        P. C. Sreeram   
76  1995               Makkal Aatchi      R. K. Selvamani   
77  1995                       Muthu      K. S. Ravikumar   
78  1995                 Neela Kuyil      Alleppey Ashraf   
79  1995             Ragasiya Police     R. S. Ilavarasan   
80  1995                 Maamanithan          Mardhananda   
81  1995                   Seethanam       R. Sundarrajan   
82  1995               Ayudha Poojai         C. Sivakumar   
83  1995                Ilaya Raagam        P. R. Devaraj   
84  1995                Maaman Magal        Guru Dhanapal   
85  1995               Jameen Kottai           Ramchandar   
86  1995            Varraar Sandiyar   N. K. Vishwanathan   
87  1995  Ilaimaikku Oru Etcharikkai        S. Manikandan   
88  1995             Murai Mappillai            Sundar C.   
89  1995             Thotta Chinungi      K. S. Adhiyaman   
90  1995    Mannai Thottu Kumbidanum        R. Umashankar   
91  1995            Dear Son Maruthu   M. Solai Rajendran   

                                                 cast genre  \
1                      Vijayakanth, Kushboo, Ranjitha  null   
2             Sivaji Ganesan, Sathyaraj, Roja, Aamani  null   
3                               

In [27]:
df['cast_size']=''
for index,row in df.iterrows():
    mlink=row['cast']
    if mlink is not None:
        arr=str(mlink).split(',')
        df.at[index,'cast_size']=len(arr)

for index,row in df.iterrows():
  mlink=row['link']
  if mlink is not None:
    print(str(mlink).strip()[6:])
    dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
    strdir=tostr(dirs)
    print(str(index)+' | '+strdir)
    df.at[index,'music']=strdir


Karuppu_Nila


en.wikipedia.org (parse) Karuppu_Nila
Karuppu Nila (en) data
{
  infobox: <dict(15)> name, director, producer, writer, screenplay...
  pageid: 45449827
  parsetree: <str(9658)> <root><template><title>Use dmy dates</tit...
  requests: <list(1)> parse
  title: Karuppu Nila
  wikibase: Q19892572
  wikidata_url: https://www.wikidata.org/wiki/Q19892572
  wikitext: <str(5952)> {{Use dmy dates|date=October 2015}}{{Use I...
}
en.wikipedia.org (parse) Engirundho_Vandhan


['Deva', 'Deva']
1 | Deva,Deva
Engirundho_Vandhan


Engirundho Vandhan (en) data
{
  infobox: <dict(15)> name, director, producer, writer, screenplay...
  pageid: 50066737
  parsetree: <str(10364)> <root><template><title>distinguish</titl...
  requests: <list(1)> parse
  title: Engirundho Vandhan
  wikibase: Q24905226
  wikidata_url: https://www.wikidata.org/wiki/Q24905226
  wikitext: <str(6685)> {{distinguish|Engirundho Vandhaal}}{{short...
}
en.wikipedia.org (parse) Baashha


['Viswanathan', 'Ramamoorthy']
2 | Viswanathan,Ramamoorthy
Baashha


...

Thotta Chinungi (en) data
{
  infobox: <dict(15)> name, director, producer, writer, screenplay...
  pageid: 22318172
  parsetree: <str(8412)> <root><template><title>Use dmy dates</tit...
  requests: <list(1)> parse
  title: Thotta Chinungi
  wikibase: Q7796917
  wikidata_url: https://www.wikidata.org/wiki/Q7796917
  wikitext: <str(6013)> {{Use dmy dates|date=October 2015}}{{Use I...
}
en.wikipedia.org (parse) Mannai_Thottu_Kumbidanum


['Philip Jerry']
89 | Philip Jerry
Mannai_Thottu_Kumbidanum


Mannai Thottu Kumbidanum (en) data
{
  infobox: <dict(13)> name, director, producer, writer, starring, ...
  pageid: 46365396
  parsetree: <str(8186)> <root><template><title>Use dmy dates</tit...
  requests: <list(1)> parse
  title: Mannai Thottu Kumbidanum
  wikibase: Q19818520
  wikidata_url: https://www.wikidata.org/wiki/Q19818520
  wikitext: <str(4836)> {{Use dmy dates|date=November 2015}}{{Use ...
}
en.wikipedia.org (parse) Dear_Son_Maruthu


['Deva', 'Deva']
90 | Deva,Deva
Dear_Son_Maruthu
['Deva', 'composer', 'Deva']
91 | Deva,composer,Deva


Dear Son Maruthu (en) data
{
  infobox: <dict(13)> name, director, producer, writer, starring, ...
  pageid: 52267562
  parsetree: <str(9065)> <root><template><title>short description<...
  requests: <list(1)> parse
  title: Dear Son Maruthu
  wikibase: Q27963139
  wikidata_url: https://www.wikidata.org/wiki/Q27963139
  wikitext: <str(5521)> {{short description|1995 film directed by ...
}


In [28]:
for index,row in df.iterrows():
  mname=row['title']
  myear=row['year']
  if mname is not None:
    rating=getrating(mname,myear)
    print(mname+' | '+rating)
    df.at[index,'imdb']=rating.strip()

https://www.imdb.com/name/nm2128968/
Karuppu Nila | 
Engirundho Vandhan | 
https://www.imdb.com/title/tt0139876/
Baashha | 8.2
https://www.imdb.com/title/tt9015066/
Kattumarakaran | 
https://www.imdb.com/title/tt0305871/
Naan Petha Magane | 5.9
https://www.imdb.com/title/tt0318520/
Oru Oorla Oru Rajakumari | 6.0
https://www.imdb.com/title/tt0140534/
Sathi Leelavathi | 8.2
https://www.imdb.com/title/tt9163658/
Veluchami | 
Gangai Karai Paattu | 
https://www.imdb.com/title/tt1445689/
Mudhal Udhayam | 
https://www.imdb.com/title/tt0318533/fullcredits
Paattu Padava | 
https://www.imdb.com/title/tt10147886/
Valli Vara Pora | 
Raja Muthirai | 
https://www.imdb.com/title/tt6823074/
Deva | 7.1
https://www.imdb.com/name/nm0621066/
Kizhakku Malai | 
https://www.imdb.com/title/tt0317935/
Muthu Kaalai | 5.8
Thottil Kuzhandhai | 
https://www.imdb.com/title/tt10218884/
Chinna Mani | 
https://www.imdb.com/india/top-rated-tamil-movies/
Kalyanam | 
https://www.imdb.com/title/tt0320607/
Manathile Oru Pa

In [29]:
df

year                       title             director  \
1   1995                Karuppu Nila       R. Sundarrajan   
2   1995          Engirundho Vandhan    Santhana Bharathi   
3   1995                     Baashha       Suresh Krissna   
4   1995              Kattumarakaran              P. Vasu   
5   1995           Naan Petha Magane            V. Sekhar   
6   1995    Oru Oorla Oru Rajakumari         K. Bhagyaraj   
7   1995            Sathi Leelavathi        Balu Mahendra   
8   1995                   Veluchami                 Arul   
9   1995         Gangai Karai Paattu           Manivannan   
10  1995              Mudhal Udhayam           Prithviraj   
11  1995               Paattu Padava  B. R. Vijayalakshmi   
12  1995             Valli Vara Pora     S. V. Solai Raja   
13  1995               Raja Muthirai      R. K. Selvamani   
14  1995                        Deva  S. A. Chandrasekhar   
15  1995              Kizhakku Malai        Ganesh Ramana   
16  1995                Muthu Kaalai      Gokula Krishnan   
17  1995          Thottil Kuzhandhai     S. P. Muthuraman   
18  1995                 Chinna Mani      M. Thilakarajan   
19  1995                    Kalyanam         R. Narayanan   
20  1995        Manathile Oru Paattu           Gnanamozhi   
21  1995              Puthiya Aatchi     Velu Prabhakaran   
22  1995                      Bombay          Mani Ratnam   
23  1995      Muthukulikka Vaariyala      K. S. Ravikumar   
24  1995              Raja Enga Raja     T. Vijayasingaam   
25  1995                     Thondan            Karvannan   
26  1995                  Aanazhagan          Thyagarajan   
27  1995     Dharmangal Sirikkinrana        A. Kasilingam   
28  1995            Udhavum Karangal      Shanmuga Priyan   
29  1995                      Coolie              P. Vasu   
30  1995          Ellame En Rasathan             Rajkiran   
..   ...                         ...                  ...   
62  1995                      Vishnu  S. A. Chandrasekhar   
63  1995               Nadodi Mannan          Manivasagam   
64  1995                   Raasaiyya            P. Kannan   
65  1995             Periya Kudumbam      K. S. Ravikumar   
66  1995                       Aasai              Vasanth   
67  1995                  Mr. Madras              P. Vasu   
68  1995                    Kolangal           I. V. Sasi   
69  1995                 Sindhu Bath           Balu Anand   
70  1995         Mannukku Mariyadhai        T. R. Vijayan   
71  1995       Thaikulame Thaikulame          N. Murugesh   
72  1995               Aakaya Pookal           Deenadayal   
73  1995                   Ilavarasi      Selva Vinayagam   
74  1995                Chandralekha           Nambhiraaj   
75  1995                Kuruthipunal        P. C. Sreeram   
76  1995               Makkal Aatchi      R. K. Selvamani   
77  1995                       Muthu      K. S. Ravikumar   
78  1995                 Neela Kuyil      Alleppey Ashraf   
79  1995             Ragasiya Police     R. S. Ilavarasan   
80  1995                 Maamanithan          Mardhananda   
81  1995                   Seethanam       R. Sundarrajan   
82  1995               Ayudha Poojai         C. Sivakumar   
83  1995                Ilaya Raagam        P. R. Devaraj   
84  1995                Maaman Magal        Guru Dhanapal   
85  1995               Jameen Kottai           Ramchandar   
86  1995            Varraar Sandiyar   N. K. Vishwanathan   
87  1995  Ilaimaikku Oru Etcharikkai        S. Manikandan   
88  1995             Murai Mappillai            Sundar C.   
89  1995             Thotta Chinungi      K. S. Adhiyaman   
90  1995    Mannai Thottu Kumbidanum        R. Umashankar   
91  1995            Dear Son Maruthu   M. Solai Rajendran   

                                                 cast genre  \
1                      Vijayakanth, Kushboo, Ranjitha  null   
2             Sivaji Ganesan, Sathyaraj, Roja, Aamani  null   
3                               

In [30]:
df.to_csv("1995_movies.csv")

# 1994

In [36]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1994'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1994=soup.find_all('table')[0:1]

In [37]:
standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

In [38]:
df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [39]:
for table in tables_1994:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='1994'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')-1

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')-1

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')-1
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Opening', 'Title', 'Director', 'Cast', 'Production', 'Music Director']
1!Amaidhi Padai|Manivannan|Sathyaraj, Ranjitha, Kasthuri|null|M. R. Films International|/wiki/Amaidhi_Padai
2!Mahanadi|Santhana Bharathi|Kamal Haasan, Sukanya|null|Sree Amman Creations|/wiki/Mahanadi_(film)
3!Rajakumaran|R. V. Udayakumar|Prabhu, Meena, Nadhiya|null|Sivaji Productions|/wiki/Rajakumaran_(film)
4!Ravanan|E. Ramdoss|Mansoor Ali Khan, Ahana|null|Raj Kennedy Films|/wiki/Ravanan_(1994_film)
5!Sethupathi IPS|P. Vasu|Vijayakanth, Meena|null|AVM Productions|/wiki/Sethupathi_IPS
6!Sindhu Nathi Poo|Senthamizhan|Ranjith, Rajakumari|null|A. R. S. Film International|/wiki/Sindhu_Nathi_Poo
7!Siragadikka Aasai|Vijayachandran|Sivakumar, Vijayakumar, Manjula Vijayakumar, Zeenat|null|Yadhav Productions|/wiki/Siragadikka_Aasai
8!Vaanga Partner Vaanga|Rama Narayanan|Visu, Radha Ravi, S. S. Chandran, Venniradai Moorthy|null|Azhagar Films|/wiki/Vaanga_Partner_Vaanga
9!Veetla Visheshanga|K. Bhagyaraj|K. Bhagyaraj, Pragath

90!Manju Virattu|L. S. Valaiyapathi|Murali, Mohana|null|Thennagam Creations|/wiki/Manju_Virattu_(film)
91!Chinna Pulla|K. Baskaran|Chinni Jayanth, Revathi|null|Ashtalakshmi Creators|/wiki/Chinna_Pulla
92!Vanaja Girija|K. R.|Napoleon, Ramki, Kushboo, Mohini|null|P. A. Art Productions|/wiki/Vanaja_Girija


In [40]:
df=df.drop(0)

In [41]:
df

year                               title                  director  \
1   1994                       Amaidhi Padai                Manivannan   
2   1994                            Mahanadi         Santhana Bharathi   
3   1994                         Rajakumaran          R. V. Udayakumar   
4   1994                             Ravanan                E. Ramdoss   
5   1994                      Sethupathi IPS                   P. Vasu   
6   1994                    Sindhu Nathi Poo              Senthamizhan   
7   1994                   Siragadikka Aasai            Vijayachandran   
8   1994               Vaanga Partner Vaanga            Rama Narayanan   
9   1994                  Veetla Visheshanga              K. Bhagyaraj   
10  1994                       Paasamalargal      Suresh Chandra Menon   
11  1994         Veettai Paaru Naattai Paaru                Thulasidas   
12  1994                      Athiradi Padai           R. K. Selvamani   
13  1994                   Subramaniya Swamy               Pandiarajan   
14  1994                             Kaviyam                 M. Ponraj   
15  1994                        Chinna Muthu         Shanmuga Sundaram   
16  1994                  Aranmanai Kaavalan           Selva Vinayagam   
17  1994                             Captain          Kodi Ramakrishna   
18  1994                      Magalir Mattum  Singeetham Srinivasa Rao   
19  1994                   Pondattiye Deivam        N. K. Vishwanathan   
20  1994                             Uzhiyan          C. Durai Pandian   
21  1994                         En Rajangam                     Siraj   
22  1994  Purushanai Kaikulla Poottukanum[3]                   Balanna   
23  1994                          Rasa Magan                Manivannan   
24  1994                            Adharmam           Ramesh Krishnan   
25  1994                          Honest Raj                K. S. Ravi   
26  1994                               Indhu                 Pavithran   
27  1994                           Sakthivel           K. S. Ravikumar   
28  1994        Varavu Ettana Selavu Pathana                 V. Sekhar   
29  1994                               Veera            Suresh Krissna   
30  1994                              Seeman                Raj Kapoor   
..   ...                                 ...                       ...   
63  1994                           Sathyavan                Raj Kapoor   
64  1994                Ungal Anbu Thangachi                Jacobbreez   
65  1994                                Hero            A. Jagannathan   
66  1994                      Mudhal Payanam          A. K. Ravi Varma   
67  1994                         Thai Maaman             Guru Dhanapal   
68  1994                           Veeramani                      Prem   
69  1994                            Kadhalan                   Shankar   
70  1994               Pattukottai Periyappa                      Visu   
71  1994                Pudhusa Pootha Roosa              P. S. Dharan   
72  1994                Thalavanin Arulullam          R. V. Udayakumar   
73  1994                            Ulavaali      Kalaipuli G. Sekaran   
74  1994                        Ilaignar Ani            K. R. Selvaraj   
75  1994                Thendral Varum Theru       N. G. Gowri Manohar   
76  1994                   Jallikattu Kaalai               Manivasagam   
77  1994                        Mani Rathnam              K. Jayabalan   
78  1994                            Nammavar       K. S. Sethumadhavan   
79  1994                            Nattamai           K. S. Ravikumar   
80  1994                            Pavithra                 K. Subash   
81  1994                      Periya Marudhu        N. K. Vishwanathan   
82  1994                        Karuththamma              Bharathiraja   
83  1994                     Veera Padhakkam                Manivannan   
84  1994                                Nila                Nambhiraaj   
85  1994

In [42]:
df['cast_size']=''
for index,row in df.iterrows():
    mlink=row['cast']
    if mlink is not None:
        arr=str(mlink).split(',')
        df.at[index,'cast_size']=len(arr)

for index,row in df.iterrows():
  mlink=row['link']
  if mlink is not None:
    print(str(mlink).strip()[6:])
    dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
    strdir=tostr(dirs)
    print(str(index)+' | '+strdir)
    df.at[index,'music']=strdir


Amaidhi_Padai


en.wikipedia.org (parse) Amaidhi_Padai
en.wikipedia.org (imageinfo) File:Amaidhi Padai.jpg
Amaidhi Padai (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Amaidhi P...
  infobox: <dict(15)> name, image, caption, director, producer, wr...
  pageid: 25095660
  parsetree: <str(13697)> <root><template><title>Use dmy dates</ti...
  requests: <list(2)> parse, imageinfo
  title: Amaidhi Padai
  wikibase: Q3644983
  wikidata_url: https://www.wikidata.org/wiki/Q3644983
  wikitext: <str(10703)> {{Use dmy dates|date=November 2015}}{{Use...
}
en.wikipedia.org (parse) Mahanadi_(film)


['Ilaiyaraaja']
1 | Ilaiyaraaja
Mahanadi_(film)


en.wikipedia.org (imageinfo) File:Mahanadhi poster.jpg
Mahanadhi (film) (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Mahanadhi...
  infobox: <dict(16)> name, image, caption, director, producer, sc...
  pageid: 5379346
  parsetree: <str(43640)> <root><template><title>short description...
  requests: <list(2)> parse, imageinfo
  title: Mahanadhi (film)
  wikibase: Q6733087
  wikidata_url: https://www.wikidata.org/wiki/Q6733087
  wikitext: <str(30153)> {{short description|1994 film by Santhana...
}
en.wikipedia.org (parse) Rajakumaran_(film)


['Ilaiyaraaja']
2 | Ilaiyaraaja
Rajakumaran_(film)


...

Manju Virattu (film) (en) data
{
  infobox: <dict(13)> name, director, producer, writer, starring, ...
  pageid: 45449828
  parsetree: <str(7642)> <root><template><title>Use dmy dates</tit...
  requests: <list(1)> parse
  title: Manju Virattu (film)
  wikibase: Q12987000
  wikidata_url: https://www.wikidata.org/wiki/Q12987000
  wikitext: <str(4200)> {{Use dmy dates|date=November 2015}}{{Use ...
}
en.wikipedia.org (parse) Chinna_Pulla


['Deva', 'Deva']
90 | Deva,Deva
Chinna_Pulla


Chinna Pulla (en) data
{
  infobox: <dict(13)> name, director, producer, writer, starring, ...
  pageid: 52112924
  parsetree: <str(10467)> <root><template><title>Use dmy dates</ti...
  requests: <list(1)> parse
  title: Chinna Pulla
  wikibase: Q27591021
  wikidata_url: https://www.wikidata.org/wiki/Q27591021
  wikitext: <str(6874)> {{Use dmy dates|date=October 2016}}{{Use I...
}
en.wikipedia.org (parse) Vanaja_Girija


['Adithyan']
91 | Adithyan
Vanaja_Girija
['Ilaiyaraaja']
92 | Ilaiyaraaja


Vanaja Girija (en) data
{
  infobox: <dict(13)> name, director, producer, writer, starring, ...
  pageid: 36057886
  parsetree: <str(8856)> <root><template><title>Use dmy dates</tit...
  requests: <list(1)> parse
  title: Vanaja Girija
  wikibase: Q7913939
  wikidata_url: https://www.wikidata.org/wiki/Q7913939
  wikitext: <str(5432)> {{Use dmy dates|date=November 2015}}{{Use ...
}


In [43]:
for index,row in df.iterrows():
  mname=row['title']
  myear=row['year']
  if mname is not None:
    rating=getrating(mname,myear)
    print(mname+' | '+rating)
    df.at[index,'imdb']=rating.strip()

https://www.imdb.com/title/tt0305204/
Amaidhi Padai | 7.6
https://www.imdb.com/title/tt0140348/
Mahanadi | 8.5
https://www.imdb.com/title/tt0318610/
Rajakumaran | 5.2
https://www.imdb.com/title/tt1664806/
Ravanan | 6.9
https://www.imdb.com/title/tt0318680/
Sethupathi IPS | 6.8
Sindhu Nathi Poo | 
Siragadikka Aasai | 
https://www.imdb.com/title/tt7520148/
Vaanga Partner Vaanga | 
https://www.imdb.com/title/tt0318802/
Veetla Visheshanga | 6.6
https://www.imdb.com/title/tt2373616/
Paasamalargal | 5.2
https://www.imdb.com/title/tt0320227/
Veettai Paaru Naattai Paaru | 
https://www.imdb.com/title/tt10168414/
Athiradi Padai | 
https://www.imdb.com/title/tt10157670/
Subramaniya Swamy | 
https://www.imdb.com/title/tt0214505/
Kaviyam | 
https://www.imdb.com/name/nm1099597/
Chinna Muthu | 
https://www.imdb.com/title/tt10919328/
Aranmanai Kaavalan | 6.6
https://www.imdb.com/title/tt9033548/
Captain | 
https://www.imdb.com/title/tt0264820/
Magalir Mattum | 7.3
https://www.imdb.com/title/tt0235662/

In [44]:
df

year                               title                  director  \
1   1994                       Amaidhi Padai                Manivannan   
2   1994                            Mahanadi         Santhana Bharathi   
3   1994                         Rajakumaran          R. V. Udayakumar   
4   1994                             Ravanan                E. Ramdoss   
5   1994                      Sethupathi IPS                   P. Vasu   
6   1994                    Sindhu Nathi Poo              Senthamizhan   
7   1994                   Siragadikka Aasai            Vijayachandran   
8   1994               Vaanga Partner Vaanga            Rama Narayanan   
9   1994                  Veetla Visheshanga              K. Bhagyaraj   
10  1994                       Paasamalargal      Suresh Chandra Menon   
11  1994         Veettai Paaru Naattai Paaru                Thulasidas   
12  1994                      Athiradi Padai           R. K. Selvamani   
13  1994                   Subramaniya Swamy               Pandiarajan   
14  1994                             Kaviyam                 M. Ponraj   
15  1994                        Chinna Muthu         Shanmuga Sundaram   
16  1994                  Aranmanai Kaavalan           Selva Vinayagam   
17  1994                             Captain          Kodi Ramakrishna   
18  1994                      Magalir Mattum  Singeetham Srinivasa Rao   
19  1994                   Pondattiye Deivam        N. K. Vishwanathan   
20  1994                             Uzhiyan          C. Durai Pandian   
21  1994                         En Rajangam                     Siraj   
22  1994  Purushanai Kaikulla Poottukanum[3]                   Balanna   
23  1994                          Rasa Magan                Manivannan   
24  1994                            Adharmam           Ramesh Krishnan   
25  1994                          Honest Raj                K. S. Ravi   
26  1994                               Indhu                 Pavithran   
27  1994                           Sakthivel           K. S. Ravikumar   
28  1994        Varavu Ettana Selavu Pathana                 V. Sekhar   
29  1994                               Veera            Suresh Krissna   
30  1994                              Seeman                Raj Kapoor   
..   ...                                 ...                       ...   
63  1994                           Sathyavan                Raj Kapoor   
64  1994                Ungal Anbu Thangachi                Jacobbreez   
65  1994                                Hero            A. Jagannathan   
66  1994                      Mudhal Payanam          A. K. Ravi Varma   
67  1994                         Thai Maaman             Guru Dhanapal   
68  1994                           Veeramani                      Prem   
69  1994                            Kadhalan                   Shankar   
70  1994               Pattukottai Periyappa                      Visu   
71  1994                Pudhusa Pootha Roosa              P. S. Dharan   
72  1994                Thalavanin Arulullam          R. V. Udayakumar   
73  1994                            Ulavaali      Kalaipuli G. Sekaran   
74  1994                        Ilaignar Ani            K. R. Selvaraj   
75  1994                Thendral Varum Theru       N. G. Gowri Manohar   
76  1994                   Jallikattu Kaalai               Manivasagam   
77  1994                        Mani Rathnam              K. Jayabalan   
78  1994                            Nammavar       K. S. Sethumadhavan   
79  1994                            Nattamai           K. S. Ravikumar   
80  1994                            Pavithra                 K. Subash   
81  1994                      Periya Marudhu        N. K. Vishwanathan   
82  1994                        Karuththamma              Bharathiraja   
83  1994                     Veera Padhakkam                Manivannan   
84  1994                                Nila                Nambhiraaj   
85  1994

In [45]:
df.to_csv('1994_movies.csv')

# 1993

In [77]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1993'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1993=soup.find_all('table')[0:1]

In [78]:
standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

In [79]:
df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [80]:
for table in tables_1993:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='1993'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')-1

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')-1

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')-1
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Opening', 'Title', 'Director', 'Cast', 'Production', 'Music Director', 'Ref']
1!Aadhityan|V. L. Bashkaraaj|R. Sarathkumar, Sukanya|null|Vellore Film Associates|/wiki/Aadhityan
2!Captain Magal|Bharathiraja|Raja, Kushboo|null|Manoj Creations|/wiki/Captain_Magal
3!Chinna Mapillai|Santhana Bharathi|Prabhu, Sukanya, Sivaranjani|null|Amma Creations|/wiki/Chinna_Mapillai
4!Jathi Malli|K. Balachander|Mukesh, Kushboo, Vineeth, Yuvarani|null|Kavithalayaa Productions|/wiki/Jathi_Malli
5!Koyil Kaalai|Gangai Amaran|Vijayakanth, Kanaka|null|Ambaal Pictures|/wiki/Koyil_Kaalai
6!Maamiyar Veedu|S. Ganesaraj|Saravanan, Selva, Sithara, Nandhini|null|Karpaga Jothi Films|/wiki/Maamiyar_Veedu
7!Marupadiyum|Balu Mahendra|Revathi, Arvind Swamy, Nizhalgal Ravi, Rohini|null|Ashwin International|/wiki/Marupadiyum
8!Mutrugai|Manobala|Arun Pandian, Bhanupriya, Ranjitha|null|Sri Varalakshmi Cine Productions|/wiki/Mutrugai
9!Walter Vetrivel|P. Vasu|Sathyaraj, Sukanya, Ranjitha|null|Kamalam Movies|/wiki/Walter_Vetr

In [81]:
df=df.drop(0)
df

year                                  title             director  \
1   1993                              Aadhityan     V. L. Bashkaraaj   
2   1993                          Captain Magal         Bharathiraja   
3   1993                        Chinna Mapillai    Santhana Bharathi   
4   1993                            Jathi Malli       K. Balachander   
5   1993                           Koyil Kaalai        Gangai Amaran   
6   1993                         Maamiyar Veedu         S. Ganesaraj   
7   1993                            Marupadiyum        Balu Mahendra   
8   1993                               Mutrugai             Manobala   
9   1993                        Walter Vetrivel              P. Vasu   
10  1993                        Chinna Kannamma             R. Raghu   
11  1993                             Dasarathan  Raja Krishnamoorthy   
12  1993                         En Idhaya Rani         S. S. Vikram   
13  1993                          Rakkayi Koyil          Manivasagam   
14  1993                              Manikuyil           Rajavarman   
15  1993                           Pudhu Piravi                 Prem   
16  1993                                Ejamaan     R. V. Udayakumar   
17  1993                           Ezhai Jaathi     Liaquat Ali Khan   
18  1993                          Mudhal Paadal        Arul Krishnan   
19  1993                      Madurai Meenakshi          P. Amirtham   
20  1993                      Suriyan Chandiran      K. S. Ravikumar   
21  1993                              Maharasan     G. N. Rangarajan   
22  1993                            Pudhu Vayal       Nanjil Kennedy   
23  1993                  Malare Kurunji Malare         A. Parthiban   
24  1993                              Iniyaraja    V. Ganesh Pandian   
25  1993                             Amma Ponnu                 Arun   
26  1993                            Enga Thambi          S. D. Sabha   
27  1993                            Pon Vilangu       K. S. Rajkumar   
28  1993                     Pettredutha Pillai      Poorna Chandran   
29  1993                              Kalaignan          G. B. Vijay   
30  1993                      Minmini Poochigal            Raghuvyas   
..   ...                                    ...                  ...   
66  1993                 Varam Tharum Vadivelan      Shanmuga Sidhan   
67  1993                            Sivarathiri       Rama Narayanan   
68  1993                      Moondravadhu Kann           Manivannan   
69  1993  Poranthalum Aambalaya Porakka Kudathu   N. K. Vishwanathan   
70  1993                        Dhool Parakuthu         K. Hariharan   
71  1993                         Karuppu Vellai             Manobala   
72  1993                     Akkarai Cheemayile       Radha Bharathi   
73  1993                         Otta Pandhayam  S. Navaraj Chelliah   
74  1993                              Rajadurai  S. A. Chandrasekhar   
75  1993                           I Love India            Pavithran   
76  1993                     Karpagam Vanthachu     R. Krishnamurthy   
77  1993                   Dhuruva Natchathiram                 Raja   
78  1993                  Parvathi Ennai Paradi            V. Sekhar   
79  1993                          Chinna Jameen           Raj Kapoor   
80  1993                         Enga Muthalali     Liaquat Ali Khan   
81  1993                            Kattabomman          Manivasagam   
82  1993                     Kilipetchu Ketkava                Fazil   
83  1993                    Kizhakku Cheemayile         Bharathiraja   
84  1993                             Muthupandi      A. Raja Pandian   
85  1993                          Paarambariyam             Manobala   
86  1993                       Pudhiya Thendral            Prabhakar   
87  1993                            Sabash Babu           Sasi Mohan   
88  1993                        Thiruda Thiruda          Mani Ratnam   
89  1993                                Uzha

In [88]:
df['cast_size']=''
for index,row in df.iterrows():
    mlink=row['cast']
    if mlink is not None:
        arr=str(mlink).split(',')
        df.at[index,'cast_size']=len(arr)

for index,row in df.iterrows():
  mlink=row['link']
  if mlink is not None:
    print(str(mlink).strip()[6:])
    dirs=getmusicdir(str(row['link'][6:]) if len(row['link'])>0 else row['title'],str(row['year']))
    strdir=tostr(dirs)
    print(str(index)+' | '+strdir)
    df.at[index,'music']=strdir


Aadhityan


en.wikipedia.org (parse) Aadhityan
Aadhityan (en) data
{
  infobox: <dict(13)> name, director, producer, writer, starring, ...
  pageid: 42026735
  parsetree: <str(9050)> <root><template><title>About</title><part...
  requests: <list(1)> parse
  title: Aadhityan
  wikibase: Q16057842
  wikidata_url: https://www.wikidata.org/wiki/Q16057842
  wikitext: <str(5240)> {{About|the Tamil film|the Tamil music dir...
}
en.wikipedia.org (parse) Captain_Magal


['Gangai Amaran']
1 | Gangai Amaran
Captain_Magal


Captain Magal (en) data
{
  infobox: <dict(14)> name, director, screenplay, story, starring,...
  pageid: 34155557
  parsetree: <str(4842)> <root><template><title>Use dmy dates</tit...
  requests: <list(1)> parse
  title: Captain Magal
  wikibase: Q5036726
  wikidata_url: https://www.wikidata.org/wiki/Q5036726
  wikitext: <str(2837)> {{Use dmy dates|date=November 2015}}{{Use ...
}
en.wikipedia.org (parse) Chinna_Mapillai


['Hamsalekha']
2 | Hamsalekha
Chinna_Mapillai


...

Kizhakke Varum Paattu (en) data
{
  image: <list(0)> 
  infobox: <dict(16)> name, image, caption, director, producer, wr...
  pageid: 40378616
  parsetree: <str(6071)> <root><template><title>Use dmy dates</tit...
  requests: <list(1)> parse
  title: Kizhakke Varum Paattu
  wikibase: Q16250472
  wikidata_url: https://www.wikidata.org/wiki/Q16250472
  wikitext: <str(3381)> {{Use dmy dates|date=December 2015}}{{Use ...
}
en.wikipedia.org (parse) Sendhoorapandi


['Deva', 'Deva']
94 | Deva,Deva
Sendhoorapandi


en.wikipedia.org (imageinfo) File:Sendhoorapandi poster.jpg


['Deva', 'Deva']
95 | Deva,Deva


Sendhoorapandi (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Sendhoora...
  infobox: <dict(17)> name, image, caption, director, producer, sc...
  pageid: 9689254
  parsetree: <str(10358)> <root><template><title>Use dmy dates</ti...
  requests: <list(2)> parse, imageinfo
  title: Sendhoorapandi
  wikibase: Q12981031
  wikidata_url: https://www.wikidata.org/wiki/Q12981031
  wikitext: <str(5959)> {{Use dmy dates|date=November 2015}}{{Use ...
}


In [83]:
for index,row in df.iterrows():
  mname=row['title']
  myear=row['year']
  if mname is not None:
    rating=getrating(mname,myear)
    print(mname+' | '+rating)
    df.at[index,'imdb']=rating.strip()

https://www.imdb.com/title/tt1445713/
Aadhityan | 4.2
https://www.imdb.com/title/tt0155585/
Captain Magal | 6.7
https://www.imdb.com/title/tt0317239/
Chinna Mapillai | 6.4
https://www.imdb.com/title/tt0154673/
Jathi Malli | 6.4
https://www.imdb.com/title/tt0317845/
Koyil Kaalai | 5.2
https://www.imdb.com/title/tt0318422/
Maamiyar Veedu | 6.2
https://www.imdb.com/title/tt0235574/
Marupadiyum | 7.3
https://www.imdb.com/title/tt10189066/
Mutrugai | 
https://www.imdb.com/title/tt0318822/
Walter Vetrivel | 6.2
https://www.imdb.com/title/tt0317238/
Chinna Kannamma | 6.6
https://www.imdb.com/title/tt12275192/
Dasarathan | 
https://www.imdb.com/title/tt8963128/
En Idhaya Rani | 
https://www.imdb.com/title/tt0318606/
Rakkayi Koyil | 
https://www.imdb.com/title/tt0318424/
Manikuyil | 
https://www.imdb.com/title/tt0095872/
Pudhu Piravi | 7.7
https://www.imdb.com/title/tt0313951/
Ejamaan | 6.6
https://www.imdb.com/title/tt0317464/
Ezhai Jaathi | 5.2
https://www.imdb.com/title/tt8144932/
Mudhal Paa

In [84]:
df

year                                  title             director  \
1   1993                              Aadhityan     V. L. Bashkaraaj   
2   1993                          Captain Magal         Bharathiraja   
3   1993                        Chinna Mapillai    Santhana Bharathi   
4   1993                            Jathi Malli       K. Balachander   
5   1993                           Koyil Kaalai        Gangai Amaran   
6   1993                         Maamiyar Veedu         S. Ganesaraj   
7   1993                            Marupadiyum        Balu Mahendra   
8   1993                               Mutrugai             Manobala   
9   1993                        Walter Vetrivel              P. Vasu   
10  1993                        Chinna Kannamma             R. Raghu   
11  1993                             Dasarathan  Raja Krishnamoorthy   
12  1993                         En Idhaya Rani         S. S. Vikram   
13  1993                          Rakkayi Koyil          Manivasagam   
14  1993                              Manikuyil           Rajavarman   
15  1993                           Pudhu Piravi                 Prem   
16  1993                                Ejamaan     R. V. Udayakumar   
17  1993                           Ezhai Jaathi     Liaquat Ali Khan   
18  1993                          Mudhal Paadal        Arul Krishnan   
19  1993                      Madurai Meenakshi          P. Amirtham   
20  1993                      Suriyan Chandiran      K. S. Ravikumar   
21  1993                              Maharasan     G. N. Rangarajan   
22  1993                            Pudhu Vayal       Nanjil Kennedy   
23  1993                  Malare Kurunji Malare         A. Parthiban   
24  1993                              Iniyaraja    V. Ganesh Pandian   
25  1993                             Amma Ponnu                 Arun   
26  1993                            Enga Thambi          S. D. Sabha   
27  1993                            Pon Vilangu       K. S. Rajkumar   
28  1993                     Pettredutha Pillai      Poorna Chandran   
29  1993                              Kalaignan          G. B. Vijay   
30  1993                      Minmini Poochigal            Raghuvyas   
..   ...                                    ...                  ...   
66  1993                 Varam Tharum Vadivelan      Shanmuga Sidhan   
67  1993                            Sivarathiri       Rama Narayanan   
68  1993                      Moondravadhu Kann           Manivannan   
69  1993  Poranthalum Aambalaya Porakka Kudathu   N. K. Vishwanathan   
70  1993                        Dhool Parakuthu         K. Hariharan   
71  1993                         Karuppu Vellai             Manobala   
72  1993                     Akkarai Cheemayile       Radha Bharathi   
73  1993                         Otta Pandhayam  S. Navaraj Chelliah   
74  1993                              Rajadurai  S. A. Chandrasekhar   
75  1993                           I Love India            Pavithran   
76  1993                     Karpagam Vanthachu     R. Krishnamurthy   
77  1993                   Dhuruva Natchathiram                 Raja   
78  1993                  Parvathi Ennai Paradi            V. Sekhar   
79  1993                          Chinna Jameen           Raj Kapoor   
80  1993                         Enga Muthalali     Liaquat Ali Khan   
81  1993                            Kattabomman          Manivasagam   
82  1993                     Kilipetchu Ketkava                Fazil   
83  1993                    Kizhakku Cheemayile         Bharathiraja   
84  1993                             Muthupandi      A. Raja Pandian   
85  1993                          Paarambariyam             Manobala   
86  1993                       Pudhiya Thendral            Prabhakar   
87  1993                            Sabash Babu           Sasi Mohan   
88  1993                        Thiruda Thiruda          Mani Ratnam   
89  1993                                Uzha

In [89]:
for index,row in df.iterrows():
    mdir=row['music']
    mset=[st for st in mdir.split(',')]
    conv_mset=[st for st in set(mset)]
    mdirs_str=""
    for s in conv_mset:
        mdirs_str=mdirs_str+s+','
    print(mdirs_str[:-1])
    df.at[index,'music']=mdirs_str[:-1]

Gangai Amaran
Hamsalekha
Ilaiyaraaja
M. M. Keeravani,Maragadha Mani
Ilaiyaraaja
Ilaiyaraaja
Ilaiyaraaja
Siddhartha
Ilaiyaraaja
Ilaiyaraaja
L. Vaidyanathan
S. S. Vikram
Ilaiyaraaja


Ilaiyaraaja
Ilaiyaraaja

Deva
Deva
Ilaiyaraaja



Ekandhan,Siva,Film score
Ilaiyaraaja
Ilaiyaraaja
T. Rajendar
Ilaiyaraaja

Ilaiyaraaja
Ilaiyaraaja
1 song,Karthik Raja,Ilaiyaraaja
Maragathamani
Ilaiyaraaja

Deva
M. S. Viswanathan
Ilaiyaraaja
composer,Deva
A. R. Rahman
Bala Bharathi
Ilaiyaraaja
Ilaiyaraaja
Sirpy

Deva
Mansoor Ali Khan
Karthik Raja,Ilaiyaraaja,ubl
Ilaiyaraaja
M. S. Viswanathan
Deva
composer,Deva
composer,Deva
Vidyasagar,composer
Sirpy
Ilaiyaraaja

composer,Deva
Ilaiyaraaja
A. R. Rahman
Deva
Ilaiyaraaja
Ilaiyaraaja
Ilaiyaraaja


Deva

composer,M.S.Viswanathan
Deva


Deva
Ilaiyaraaja
Sankar Ganesh
Ilaiyaraaja
Ilaiyaraaja
Ilaiyaraaja
Ilaiyaraaja
Deva
Ilaiyaraaja
A. R. Rahman

Shankar Ganesh

T. Rajendar

A. R. Rahman
Ilaiyaraaja
Deva
Deva
S. P. Venkatesh
Deva
Deva


In [90]:
df

year                                  title             director  \
1   1993                              Aadhityan     V. L. Bashkaraaj   
2   1993                          Captain Magal         Bharathiraja   
3   1993                        Chinna Mapillai    Santhana Bharathi   
4   1993                            Jathi Malli       K. Balachander   
5   1993                           Koyil Kaalai        Gangai Amaran   
6   1993                         Maamiyar Veedu         S. Ganesaraj   
7   1993                            Marupadiyum        Balu Mahendra   
8   1993                               Mutrugai             Manobala   
9   1993                        Walter Vetrivel              P. Vasu   
10  1993                        Chinna Kannamma             R. Raghu   
11  1993                             Dasarathan  Raja Krishnamoorthy   
12  1993                         En Idhaya Rani         S. S. Vikram   
13  1993                          Rakkayi Koyil          Manivasagam   
14  1993                              Manikuyil           Rajavarman   
15  1993                           Pudhu Piravi                 Prem   
16  1993                                Ejamaan     R. V. Udayakumar   
17  1993                           Ezhai Jaathi     Liaquat Ali Khan   
18  1993                          Mudhal Paadal        Arul Krishnan   
19  1993                      Madurai Meenakshi          P. Amirtham   
20  1993                      Suriyan Chandiran      K. S. Ravikumar   
21  1993                              Maharasan     G. N. Rangarajan   
22  1993                            Pudhu Vayal       Nanjil Kennedy   
23  1993                  Malare Kurunji Malare         A. Parthiban   
24  1993                              Iniyaraja    V. Ganesh Pandian   
25  1993                             Amma Ponnu                 Arun   
26  1993                            Enga Thambi          S. D. Sabha   
27  1993                            Pon Vilangu       K. S. Rajkumar   
28  1993                     Pettredutha Pillai      Poorna Chandran   
29  1993                              Kalaignan          G. B. Vijay   
30  1993                      Minmini Poochigal            Raghuvyas   
..   ...                                    ...                  ...   
66  1993                 Varam Tharum Vadivelan      Shanmuga Sidhan   
67  1993                            Sivarathiri       Rama Narayanan   
68  1993                      Moondravadhu Kann           Manivannan   
69  1993  Poranthalum Aambalaya Porakka Kudathu   N. K. Vishwanathan   
70  1993                        Dhool Parakuthu         K. Hariharan   
71  1993                         Karuppu Vellai             Manobala   
72  1993                     Akkarai Cheemayile       Radha Bharathi   
73  1993                         Otta Pandhayam  S. Navaraj Chelliah   
74  1993                              Rajadurai  S. A. Chandrasekhar   
75  1993                           I Love India            Pavithran   
76  1993                     Karpagam Vanthachu     R. Krishnamurthy   
77  1993                   Dhuruva Natchathiram                 Raja   
78  1993                  Parvathi Ennai Paradi            V. Sekhar   
79  1993                          Chinna Jameen           Raj Kapoor   
80  1993                         Enga Muthalali     Liaquat Ali Khan   
81  1993                            Kattabomman          Manivasagam   
82  1993                     Kilipetchu Ketkava                Fazil   
83  1993                    Kizhakku Cheemayile         Bharathiraja   
84  1993                             Muthupandi      A. Raja Pandian   
85  1993                          Paarambariyam             Manobala   
86  1993                       Pudhiya Thendral            Prabhakar   
87  1993                            Sabash Babu           Sasi Mohan   
88  1993                        Thiruda Thiruda          Mani Ratnam   
89  1993                                Uzha

In [91]:
df.to_csv('1993_movies.csv')

# 1992

In [9]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1992'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1992=soup.find_all('table')[1:2]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [10]:
df

year title director  cast genre producer  link
0  2017  test     test  test  test     test  test

In [11]:
for table in tables_1992:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='1992'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')-1

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')-1

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')-1
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Opening', 'Title', 'Director', 'Cast', 'Production', 'Music Director']
1!Amaran|K. Rajeshwar|Karthik, Bhanupriya|null|Annalakshmi Films|/wiki/Amaran_(film)
2!Aval Oru Vasantham|Jayadevan|Prabhuraj, Jayanthi|null|Sharmila Creations|
3!Brahmachari|Muktha Srinivasan|Nizhalgal Ravi, Gautami|null|Muktha Films|/wiki/Brahmachari_(1992_film)
4!Chinna Gounder|R. V. Udayakumar|Vijayakanth, Sukanya, Goundamani|null|Ananthi Films|/wiki/Chinna_Gounder
5!Ilavarasan|Senthilnathan|R. Sarathkumar, Sukanya|null|Sathya Movies|/wiki/Ilavarasan
6!Mannan|P. Vasu|Rajinikanth, Vijayashanti, Kushboo|null|Sivaji Productions|/wiki/Mannan_(film)
7!Pandithurai|Manoj Kumar|Prabhu, Kushboo|null|K. B. Films|/wiki/Pandithurai
8!Rickshaw Mama|P. Vasu|Sathyaraj, Kushboo, Gautami, 'Baby' Sridevi|null|Shri Mishri Productions|/wiki/Rickshaw_Mama
9!Sundara Kandam|K. Bhagyaraj|K. Bhagyaraj, Bhanupriya, Sindhuja|null|Saranya Cine Combines|/wiki/Sundara_Kandam
10!Vanna Vanna Pookkal|Balu Mahendra|Prashanth, Mounika, Vinodhin

In [12]:
df=df.drop(0)
df

year                        title               director  \
1    1992                       Amaran           K. Rajeshwar   
2    1992           Aval Oru Vasantham              Jayadevan   
3    1992                  Brahmachari      Muktha Srinivasan   
4    1992               Chinna Gounder       R. V. Udayakumar   
5    1992                   Ilavarasan          Senthilnathan   
6    1992                       Mannan                P. Vasu   
7    1992                  Pandithurai            Manoj Kumar   
8    1992                Rickshaw Mama                P. Vasu   
9    1992               Sundara Kandam           K. Bhagyaraj   
10   1992          Vanna Vanna Pookkal          Balu Mahendra   
11   1992                Chinna Thayee           S. Ganesaraj   
12   1992                 Agni Paarvai            P. Madhavan   
13   1992  Rendu Pondatti Kaavalkaaran  Relangi Narasimha Rao   
14   1992              Thambi Pondatti     Panchu Arunachalam   
15   1992               Kaaval Geetham       S. P. Muthuraman   
16   1992                 Moondrampadi              Karvannan   
17   1992    Unnai Vaazhthi Paadugiren                 Sridev   
18   1992            Sugamana Sumaigal              Parthiban   
19   1992                Naane Varuven               Sripriya   
20   1992               Jodi Sendhachu                Lakshmi   
21   1992         Periya Gounder Ponnu            Manivasagam   
22   1992         Government Mappillai             Manivannan   
23   1992              Kasu Thangakasu            Mohamed Raj   
24   1992               Sivantha Malar             Sasi Mohan   
25   1992                      Naangal                 Haasan   
26   1992              Kasthuri Manjal         J. B. Rajaravi   
27   1992          Thanga Manasukkaran             Rajavarman   
28   1992             Enga Veetu Velan            T. Rajendar   
29   1992                   Thangarasu           K. Alexander   
30   1992               Innisai Mazhai    Shoba Chandrasekhar   
..    ...                          ...                    ...   
91   1992                  Annai Vayal              Ponvannan   
92   1992   Annan Ennada Thambi Ennada       Vijay Krishnaraj   
93   1992                Chinna Chittu              Manivarma   
94   1992                  David Uncle                   Guna   
95   1992              Annayin Madiyil            S. Rajaguru   
96   1992         Sagalakala Vaandugal          Senthil Kumar   
97   1992             Kaviya Thalaivan   K. S. Gopalakrishnan   
98   1992              Mangala Nayagan           K. Raghunath   
99   1992                     Pandiyan       S. P. Muthuraman   
100  1992                   Raasukutti           K. Bhagyaraj   
101  1992       Sathiyam Adhu Nichayam            T. Durairaj   
102  1992            Senthamizh Paattu                P. Vasu   
103  1992                 Thevar Magan              Bharathan   
104  1992        Thirumathi Palanisamy         R. Sundarrajan   
105  1992       Onna Irukka Kathukanum              V. Sekhar   
106  1992                     Abhirami            Dilip Kumar   
107  1992       Neenga Nalla Irukkanum                   Visu   
108  1992             Villu Pattukaran          Gangai Amaran   
109  1992             Deiva Kuzhanthai          Senthilnathan   
110  1992            Harihara Puthiran             M. N. Balu   
111  1992             Naalaiya Theerpu    S. A. Chandrasekhar   
112  1992          Natchathira Nayagan          Senthilnathan   
113  1992               Pudhu Varusham               Rajkumar   
114  1992       Purushan Enakku Arasan         Rama Narayanan   
115  1992                   Solaiyamma          Kasthuri Raja   
116  1992         Kaavalukku Kannillai           Ashwin Kumar   
117  1992                        Meera          P. C. Sreeram   
118  1992            Thoorathu Sontham        K. S. Adhiyaman   
119  1992           Devar Veettu Ponnu         Rama Narayanan   
120  1992                   Mr. Prasad  

In [13]:
getAllMusic()

Amaran_(film)


en.wikipedia.org (parse) Amaran_(film)
en.wikipedia.org (imageinfo) File:Amaran DVD cover.jpg
Amaran (film) (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Amaran DV...
  infobox: <dict(15)> name, image, caption, director, producer, wr...
  pageid: 34307355
  parsetree: <str(8852)> <root><template><title>short description<...
  requests: <list(2)> parse, imageinfo
  title: Amaran (film)
  wikibase: Q4740141
  wikidata_url: https://www.wikidata.org/wiki/Q4740141
  wikitext: <str(5425)> {{short description|1992 film directed by ...
}
en.wikipedia.org (parse) Aval Oru Vasantham


['Adithyan']
1 | Adithyan



API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Brahmachari_(1992_film)


Exception
2 | 
Brahmachari_(1992_film)


...

API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Devar Veettu Ponnu


Exception
118 | 



API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Mr. Prasad


Exception
119 | 

Exception
120 | 


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}


In [14]:
getAllrating()

https://www.imdb.com/title/tt10713436/
Amaran | 7.6
https://www.imdb.com/title/tt8919526/
Aval Oru Vasantham | 
https://www.imdb.com/title/tt8929710/
Brahmachari | 
https://www.imdb.com/title/tt0306644/
Chinna Gounder | 6.9
https://www.imdb.com/title/tt8963640/plotsummary
Ilavarasan | 
https://www.imdb.com/title/tt0319813/
Mannan | 7.2
https://www.imdb.com/title/tt10133754/
Pandithurai | 4.2
https://www.imdb.com/title/tt0318629/
Rickshaw Mama | 5.3
https://www.imdb.com/title/tt0156114/
Sundara Kandam | 7.3
https://www.imdb.com/title/tt0235855/
Vanna Vanna Pookkal | 6.1
https://www.imdb.com/title/tt0317242/
Chinna Thayee | 5.9
https://www.imdb.com/title/tt0317061/
Agni Paarvai | 
Rendu Pondatti Kaavalkaaran | 
https://www.imdb.com/title/tt0318747/
Thambi Pondatti | 5.2
https://www.imdb.com/title/tt0317790/
Kaaval Geetham | 7.8
https://www.imdb.com/search/title/?languages=ta&sort=release_date_us&title_type=feature&year=1992,1992
Moondrampadi | 
https://www.imdb.com/title/tt0318791/
Unnai

In [15]:
df

year                        title               director  \
1    1992                       Amaran           K. Rajeshwar   
2    1992           Aval Oru Vasantham              Jayadevan   
3    1992                  Brahmachari      Muktha Srinivasan   
4    1992               Chinna Gounder       R. V. Udayakumar   
5    1992                   Ilavarasan          Senthilnathan   
6    1992                       Mannan                P. Vasu   
7    1992                  Pandithurai            Manoj Kumar   
8    1992                Rickshaw Mama                P. Vasu   
9    1992               Sundara Kandam           K. Bhagyaraj   
10   1992          Vanna Vanna Pookkal          Balu Mahendra   
11   1992                Chinna Thayee           S. Ganesaraj   
12   1992                 Agni Paarvai            P. Madhavan   
13   1992  Rendu Pondatti Kaavalkaaran  Relangi Narasimha Rao   
14   1992              Thambi Pondatti     Panchu Arunachalam   
15   1992               Kaaval Geetham       S. P. Muthuraman   
16   1992                 Moondrampadi              Karvannan   
17   1992    Unnai Vaazhthi Paadugiren                 Sridev   
18   1992            Sugamana Sumaigal              Parthiban   
19   1992                Naane Varuven               Sripriya   
20   1992               Jodi Sendhachu                Lakshmi   
21   1992         Periya Gounder Ponnu            Manivasagam   
22   1992         Government Mappillai             Manivannan   
23   1992              Kasu Thangakasu            Mohamed Raj   
24   1992               Sivantha Malar             Sasi Mohan   
25   1992                      Naangal                 Haasan   
26   1992              Kasthuri Manjal         J. B. Rajaravi   
27   1992          Thanga Manasukkaran             Rajavarman   
28   1992             Enga Veetu Velan            T. Rajendar   
29   1992                   Thangarasu           K. Alexander   
30   1992               Innisai Mazhai    Shoba Chandrasekhar   
..    ...                          ...                    ...   
91   1992                  Annai Vayal              Ponvannan   
92   1992   Annan Ennada Thambi Ennada       Vijay Krishnaraj   
93   1992                Chinna Chittu              Manivarma   
94   1992                  David Uncle                   Guna   
95   1992              Annayin Madiyil            S. Rajaguru   
96   1992         Sagalakala Vaandugal          Senthil Kumar   
97   1992             Kaviya Thalaivan   K. S. Gopalakrishnan   
98   1992              Mangala Nayagan           K. Raghunath   
99   1992                     Pandiyan       S. P. Muthuraman   
100  1992                   Raasukutti           K. Bhagyaraj   
101  1992       Sathiyam Adhu Nichayam            T. Durairaj   
102  1992            Senthamizh Paattu                P. Vasu   
103  1992                 Thevar Magan              Bharathan   
104  1992        Thirumathi Palanisamy         R. Sundarrajan   
105  1992       Onna Irukka Kathukanum              V. Sekhar   
106  1992                     Abhirami            Dilip Kumar   
107  1992       Neenga Nalla Irukkanum                   Visu   
108  1992             Villu Pattukaran          Gangai Amaran   
109  1992             Deiva Kuzhanthai          Senthilnathan   
110  1992            Harihara Puthiran             M. N. Balu   
111  1992             Naalaiya Theerpu    S. A. Chandrasekhar   
112  1992          Natchathira Nayagan          Senthilnathan   
113  1992               Pudhu Varusham               Rajkumar   
114  1992       Purushan Enakku Arasan         Rama Narayanan   
115  1992                   Solaiyamma          Kasthuri Raja   
116  1992         Kaavalukku Kannillai           Ashwin Kumar   
117  1992                        Meera          P. C. Sreeram   
118  1992            Thoorathu Sontham        K. S. Adhiyaman   
119  1992           Devar Veettu Ponnu         Rama Narayanan   
120  1992                   Mr. Prasad  

In [17]:
for index,row in df.iterrows():
    mdir=row['music']
    mset=[st for st in mdir.split(',')]
    conv_mset=[st for st in set(mset)]
    mdirs_str=""
    for s in conv_mset:
        mdirs_str=mdirs_str+s+','
    print(mdirs_str[:-1])
    df.at[index,'music']=mdirs_str[:-1]

Adithyan

Deva
Ilaiyaraaja
Deva
Ilaiyaraaja
Ilaiyaraaja
Ilaiyaraaja
Deepak
Ilaiyaraaja
Ilaiyaraaja
Ilaiyaraaja
Raj,Koti
Ilaiyaraaja
Ilaiyaraaja

Ilaiyaraaja
Chandrabose
Shankar,Ganesh

Deva
Deva

Maragadha Mani,M. M. Keeravani


Ilaiyaraaja

Deva
Ilaiyaraaja
Ilaiyaraaja
Ilaiyaraaja
Deva
Ilaiyaraaja
Ilaiyaraaja

Ilaiyaraaja

Gangai Amaran
Ilaiyaraaja
Deva

Ilaiyaraaja
Deva
Deva
Maragadha Mani,M. M. Keeravani

Sriraj M S
Sangeetha Rajan,S. P. Venkatesh
S. P. Balasubrahmanyam


Ilaiyaraaja
Ilaiyaraaja

Maragadha Mani,M. M. Keeravani


Deva
Deva
Ilaiyaraaja



S. A. Rajkumar
Ilaiyaraaja
Ilaiyaraaja
Sivaji Raja

Ilaiyaraaja
Chandrabose
Soundaryan
Adithyan
Deva
Deva
Sankar Ganesh
Deva
Deva
A. R. Rahman

Ilaiyaraaja
Bala Bharathi


Ilaiyaraaja
S. A. Rajkumar
Ilaiyaraaja


Deva

Aabavanan,Manoj Gyan,Gyan Varma

Adithyan


Aravind Siddhartha

Karthik Raja,Ilaiyaraaja
Ilaiyaraaja

M. S. Viswanathan,Ilaiyaraaja,Viswanathan
Ilaiyaraaja
Ilaiyaraaja
Ilaiyaraaja
Deva
M. S. Viswanathan
Ilaiyaraaja


M

In [18]:
df

year                        title               director  \
1    1992                       Amaran           K. Rajeshwar   
2    1992           Aval Oru Vasantham              Jayadevan   
3    1992                  Brahmachari      Muktha Srinivasan   
4    1992               Chinna Gounder       R. V. Udayakumar   
5    1992                   Ilavarasan          Senthilnathan   
6    1992                       Mannan                P. Vasu   
7    1992                  Pandithurai            Manoj Kumar   
8    1992                Rickshaw Mama                P. Vasu   
9    1992               Sundara Kandam           K. Bhagyaraj   
10   1992          Vanna Vanna Pookkal          Balu Mahendra   
11   1992                Chinna Thayee           S. Ganesaraj   
12   1992                 Agni Paarvai            P. Madhavan   
13   1992  Rendu Pondatti Kaavalkaaran  Relangi Narasimha Rao   
14   1992              Thambi Pondatti     Panchu Arunachalam   
15   1992               Kaaval Geetham       S. P. Muthuraman   
16   1992                 Moondrampadi              Karvannan   
17   1992    Unnai Vaazhthi Paadugiren                 Sridev   
18   1992            Sugamana Sumaigal              Parthiban   
19   1992                Naane Varuven               Sripriya   
20   1992               Jodi Sendhachu                Lakshmi   
21   1992         Periya Gounder Ponnu            Manivasagam   
22   1992         Government Mappillai             Manivannan   
23   1992              Kasu Thangakasu            Mohamed Raj   
24   1992               Sivantha Malar             Sasi Mohan   
25   1992                      Naangal                 Haasan   
26   1992              Kasthuri Manjal         J. B. Rajaravi   
27   1992          Thanga Manasukkaran             Rajavarman   
28   1992             Enga Veetu Velan            T. Rajendar   
29   1992                   Thangarasu           K. Alexander   
30   1992               Innisai Mazhai    Shoba Chandrasekhar   
..    ...                          ...                    ...   
91   1992                  Annai Vayal              Ponvannan   
92   1992   Annan Ennada Thambi Ennada       Vijay Krishnaraj   
93   1992                Chinna Chittu              Manivarma   
94   1992                  David Uncle                   Guna   
95   1992              Annayin Madiyil            S. Rajaguru   
96   1992         Sagalakala Vaandugal          Senthil Kumar   
97   1992             Kaviya Thalaivan   K. S. Gopalakrishnan   
98   1992              Mangala Nayagan           K. Raghunath   
99   1992                     Pandiyan       S. P. Muthuraman   
100  1992                   Raasukutti           K. Bhagyaraj   
101  1992       Sathiyam Adhu Nichayam            T. Durairaj   
102  1992            Senthamizh Paattu                P. Vasu   
103  1992                 Thevar Magan              Bharathan   
104  1992        Thirumathi Palanisamy         R. Sundarrajan   
105  1992       Onna Irukka Kathukanum              V. Sekhar   
106  1992                     Abhirami            Dilip Kumar   
107  1992       Neenga Nalla Irukkanum                   Visu   
108  1992             Villu Pattukaran          Gangai Amaran   
109  1992             Deiva Kuzhanthai          Senthilnathan   
110  1992            Harihara Puthiran             M. N. Balu   
111  1992             Naalaiya Theerpu    S. A. Chandrasekhar   
112  1992          Natchathira Nayagan          Senthilnathan   
113  1992               Pudhu Varusham               Rajkumar   
114  1992       Purushan Enakku Arasan         Rama Narayanan   
115  1992                   Solaiyamma          Kasthuri Raja   
116  1992         Kaavalukku Kannillai           Ashwin Kumar   
117  1992                        Meera          P. C. Sreeram   
118  1992            Thoorathu Sontham        K. S. Adhiyaman   
119  1992           Devar Veettu Ponnu         Rama Narayanan   
120  1992                   Mr. Prasad  

In [19]:
df.to_csv('1992_movies.csv')

# 1991

In [9]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1991'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1991=soup.find_all('table')[1:2]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [10]:
df

year title director  cast genre producer  link
0  2017  test     test  test  test     test  test

In [11]:
for table in tables_1991:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='1991'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')-1

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')-1

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')-1
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Opening', 'Title', 'Director', 'Cast', 'Production', 'Music Director']
1!Gnana Paravai|Vietnam Veedu Sundaram|Sivaji Ganesan, Harish Kumar, Sasikala, Manorama|null|Yagava Productions|/wiki/Gnana_Paravai
2!Sami Potta Mudichu|R. Sundarrajan|Murali, Sindhu|null|Amma Creations|/wiki/Sami_Potta_Mudichu
3!Sigaram|Ananthu|S. P. Balasubrahmanyam, Rekha, Radha, Anand Babu, Ramya Krishnan|null|Kavithalayaa Productions|/wiki/Sigaram
4!Vaa Arugil Vaa|Kalaivanan Kannadasan|Raja, Vaishnavi, Ramya Krishnan|null|Chozha Cine Arts|/wiki/Vaa_Arugil_Vaa
5!Eeramana Rojave|K. R.|Shiva, Mohini|null|K. R Enterprises|/wiki/Eeramana_Rojave
6!Dharma Dorai|Rajasekhar|Rajinikanth, Gautami, Nizhalgal Ravi|null|Raasikala Mandhir International|/wiki/Dharma_Dorai
7!Kumbakarai Thangaiah|Gangai Amaran|Prabhu, Kanaka|null|Murugan Cine Arts|/wiki/Kumbakarai_Thangaiah
8!Naadu Adhai Naadu|Ramathilaga Rajan|Ramarajan, Rupini|null|Meena Movies|/wiki/Naadu_Adhai_Naadu
9!Perum Pulli|Vikraman|Babu, Suman Ranganathan|null|Super

In [12]:
getAllMusic()

en.wikipedia.org (random) 🍩
William M. Fairbank (en) data
{
  pageid: 27978147
  requests: <list(1)> random
  title: William M. Fairbank
}
en.wikipedia.org (parse) 27978147
William M. Fairbank (en) data
{
  infobox: <dict(5)> name, birth_place, education, occupation, org...
  pageid: 27978147
  parsetree: <str(7716)> <root><template><title>Infobox person</ti...
  requests: <list(2)> random, parse
  title: William M. Fairbank
  wikibase: Q765203
  wikidata_url: https://www.wikidata.org/wiki/Q765203
  wikitext: <str(6479)> {{Infobox person| name               = Wil...
}
en.wikipedia.org (parse) Gnana_Paravai


0 | 
Gnana_Paravai


en.wikipedia.org (imageinfo) File:Gnana Paravai poster.jpg
Gnana Paravai (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Gnana Par...
  infobox: <dict(15)> name, image, caption, director, producer, wr...
  pageid: 43317904
  parsetree: <str(8771)> <root><template><title>Use dmy dates</tit...
  requests: <list(2)> parse, imageinfo
  title: Gnana Paravai
  wikibase: Q18125139
  wikidata_url: https://www.wikidata.org/wiki/Q18125139
  wikitext: <str(5075)> {{Use dmy dates|date=November 2015}}{{Use ...
}
en.wikipedia.org (parse) Sami_Potta_Mudichu


['M. S. Viswanathan']
1 | M. S. Viswanathan
Sami_Potta_Mudichu


Sami Potta Mudichu (en) data
{
  infobox: <dict(13)> name, director, producer, writer, starring, ...
  pageid: 34858799
  parsetree: <str(3999)> <root><template><title>Use dmy dates</tit...
  requests: <list(1)> parse
  title: Sami Potta Mudichu
  wikibase: Q7409390
  wikidata_url: https://www.wikidata.org/wiki/Q7409390
  wikitext: <str(2070)> {{Use dmy dates|date=October 2015}}{{Use I...
}
en.wikipedia.org (parse) Sigaram


...

API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Aval


Exception
109 | 



Aval (en) data
{
  pageid: 12054481
  parsetree: <str(415)> <root>'''''Aval''''' (<template><title>lit...
  requests: <list(1)> parse
  title: Aval
  wikibase: Q29458273
  wikidata_url: https://www.wikidata.org/wiki/Q29458273
  wikitext: <str(280)> '''''Aval''''' ({{lit|She}}) may refer to:*...
}
en.wikipedia.org (parse) Thayamma


110 | 
Thayamma
['Ilaiyaraaja']
111 | Ilaiyaraaja


Thayamma (en) data
{
  infobox: <dict(15)> name, director, producer, writer, screenplay...
  pageid: 43101202
  parsetree: <str(8492)> <root><template><title>Use dmy dates</tit...
  requests: <list(1)> parse
  title: Thayamma
  wikibase: Q17300120
  wikidata_url: https://www.wikidata.org/wiki/Q17300120
  wikitext: <str(4835)> {{Use dmy dates|date=October 2015}}{{Use I...
}


In [13]:
getAllrating()

https://www.imdb.com/search/title/?year=2017,2017&title_type=feature&explore=languages&languages=ta&ref_=adv_explore_rhs
test | 
https://www.imdb.com/title/tt1440179/
Gnana Paravai | 
https://www.imdb.com/title/tt0320395/
Sami Potta Mudichu | 
https://www.imdb.com/title/tt10122538/
Sigaram | 
https://www.imdb.com/title/tt0232883/
Vaa Arugil Vaa | 6.0
https://www.imdb.com/title/tt0317411/
Eeramana Rojave | 6.1
https://www.imdb.com/title/tt0317352/
Dharma Dorai | 6.5
https://www.imdb.com/title/tt0319711/
Kumbakarai Thangaiah | 3.2
Naadu Adhai Naadu | 
Perum Pulli | 
https://www.imdb.com/title/tt10189582/
Thaiyalkaran | 
https://www.imdb.com/title/tt0155334/
Theechati Govindhan | 6.8
https://www.imdb.com/title/tt6312788/
Thai Poosam | 
https://www.imdb.com/title/tt3269138/
Vanakkam Vathiyare | 
https://www.imdb.com/title/tt6843516/
Namma Ooru Mariamma | 
https://www.imdb.com/title/tt10189406/
Nanbargal | 
https://www.imdb.com/title/tt0320471/
Sir...I Love You | 5.0
https://www.imdb.com/ti

In [15]:
df=df.drop(0)
df

year                           title                director  \
1    1991                   Gnana Paravai  Vietnam Veedu Sundaram   
2    1991              Sami Potta Mudichu          R. Sundarrajan   
3    1991                         Sigaram                 Ananthu   
4    1991                  Vaa Arugil Vaa   Kalaivanan Kannadasan   
5    1991                 Eeramana Rojave                   K. R.   
6    1991                    Dharma Dorai              Rajasekhar   
7    1991            Kumbakarai Thangaiah           Gangai Amaran   
8    1991               Naadu Adhai Naadu       Ramathilaga Rajan   
9    1991                     Perum Pulli                Vikraman   
10   1991                    Thaiyalkaran        S. P. Muthuraman   
11   1991             Theechati Govindhan              Sasi Mohan   
12   1991                     Thai Poosam          Rama Narayanan   
13   1991              Vanakkam Vathiyare                Ameerjan   
14   1991             Namma Ooru Mariamma                K. Rajan   
15   1991                       Nanbargal     Shoba Chandrasekhar   
16   1991                Sir...I Love You        G. N. Rangarajan   
17   1991             Aasai Kiliye Kobama            T. K. Prasad   
18   1991                  Irumbu Pookkal             G. M. Kumar   
19   1991          Naan Pudicha Mappillai               V. Sekhar   
20   1991               Chithirai Pookkal           Kanmani Subbu   
21   1991            Annan Kaattiya Vazhi         K. S. Madhangan   
22   1991               Mookuthi Poomeley                Sangaman   
23   1991        Pudhu Nellu Pudhu Naathu            Bharathiraja   
24   1991                          Uruvam             G. M. Kumar   
25   1991                  Vetri Karangal       R. Krishnamoorthy   
26   1991                   Vetri Padigal                Manobala   
27   1991                  Apoorva Naagam             Chozharajan   
28   1991                 Gopura Vasalile            Priyadarshan   
29   1991                  Pudhu Manithan              Manivannan   
30   1991              Thanga Thamaraigal           V. Azhagappan   
..    ...                             ...                     ...   
82   1991                       Oyilattam          R. Sundarrajan   
83   1991                      Marupakkam     K. S. Sethumadhavan   
84   1991                        Eeshwari          Rama Narayanan   
85   1991             Oru Oomaiyin Raagam          C. Gurushankar   
86   1991            Vaasalil Oru Vennila          Cochin Haneefa   
87   1991        Iyumbathilum Aasai Varum            P. Kalaimani   
88   1991                          Bramma               K. Subash   
89   1991        En Pottukku Sonthakkaran              Sekar Raja   
90   1991                           Gunaa       Santhana Bharathi   
91   1991  Moondrezhuthil En Moochirukkum                Manobala   
92   1991           Nenjamundu Nermaiundu              E. Ramdoss   
93   1991                   Pillai Paasam          Cochin Haneefa   
94   1991                          Rudhra              Sasi Mohan   
95   1991                      Thalapathi             Mani Ratnam   
96   1991            Thalattu Ketkuthamma              Raj Kapoor   
97   1991            Putham Pudhu Payanam         K. S. Ravikumar   
98   1991            Thangamana Thangachi           Senthilnathan   
99   1991               Vaidehi Vandhachu          Radha Bharathi   
100  1991            Anbulla Thangachikku             Chandranath   
101  1991       Pondatti Sonna Kettukanum               V. Sekhar   
102  1991               Paattondru Ketten        V. C. Guhanathan   
103  1991                   Aadi Viratham          Rama Narayanan   
104  1991        Thoodhu Poo Chellakiliye           Kasthuri Raja   
105  1991                   Kurumbukkaran                Ameerjan   
106  1991              Rasathi Varum Naal                    Rafi   
107  1991          Padhai Maariya Payanam         A. Balakrishnan   
108

In [16]:
df.to_csv("1991_movies.csv")

# 1990

In [10]:
wiki_url_test='https://en.wikipedia.org/wiki/List_of_Tamil_films_of_1990'
webdata=requests.get(wiki_url_test).text
soup=BeautifulSoup(webdata,'lxml')
tables_1990=soup.find_all('table')[1:2]

standard_columns=['Title','Director','Cast','Genre','Producer','Studio','Production Studio']
calender=['JAN','FEB','MAR','APR','MAY','JUN','JUL','AUG','SEP','OCT','NOV','DEC'] 

df = pd.DataFrame(columns=['year','title','director','cast','genre','producer'])
df
data_2019={'year':'2017',
           'title':'test',
           'director':'test',
           'cast':'test',
           'genre':'test',
           'producer':'test',
              'link':'test'}
df=df.append(data_2019,ignore_index=True)

In [11]:
df

year title director  cast genre producer  link
0  2017  test     test  test  test     test  test

In [12]:
for table in tables_1990:
    t_body=table.find('tbody')
    rows=t_body.find_all('tr')
    title_index=0
    director_index=1
    cast_index=2
    genre_index=3
    producer_index=4
    columns=table.find_all('th')
    column_names=[]
    genre_member=False
    director_member=False
    producer_member=False
    cast_member=False
    yr='1990'
    for col in columns:
      column_names.append(col.text.strip())
    print(column_names)

    if 'Genre' in column_names:
      genre_member=True
      genre_index=column_names.index('Genre')-1

    if 'Director' in column_names:
      director_member=True
      director_index=column_names.index('Director')-1

    if 'Cast' in column_names:
      cast_member=True
      cast_index=column_names.index('Cast')-1

    if 'Studio' in column_names:
      producer_index=column_names.index('Studio')-1
      producer_member=True

    if 'Producer' in column_names:
      producer_index=column_names.index('Producer')-1
      producer_member=True

    if 'Production' in column_names:
      producer_index=column_names.index('Production')-1
      producer_member=True
        
    if 'Production Studio' in column_names:
      producer_index=column_names.index('Production Studio')-1
      producer_member=True
    i=0
    for datarow in rows:
      if i==0:
        i=i+1
        continue
      all_data=datarow.find_all('td')
      if(len(all_data)>0 and all_data is not None):
        if(all_data[title_index].text is not None and all_data[title_index].text.strip().isnumeric()):
          if(all_data[title_index].text.strip() in calender):
            print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          if(len(all_data)>4):
            print(str(len(df))+'!'+all_data[title_index+1].text.strip()+'|'+(all_data[director_index+1].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+1].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+1].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+1].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index+1].text.strip(),(all_data[director_index+1].text.strip() if director_member else 'null'),(all_data[cast_index+1].text.strip() if cast_member else 'null'),(all_data[genre_index+1].text.strip() if genre_member else 'null'),(all_data[producer_index+1].text.strip() if producer_member else 'null'),(all_data[title_index+1].find_all('a', href=True)[0]['href'] if len(all_data[title_index+1].find_all('a', href=True))>0 else '')]
        else:
          if(all_data[0].text.strip() in calender):
            if(len(all_data)>2):
              print(str(len(df))+'!'+all_data[title_index+2].text.strip()+'|'+(all_data[director_index+2].text.strip() if director_member else 'null')+'|'+(all_data[cast_index+2].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index+2].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index+2].text.strip() if producer_member else 'null')+'|'+(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else ''))
              df.loc[df.index.max()+1]=[yr,all_data[title_index+2].text.strip(),(all_data[director_index+2].text.strip() if director_member else 'null'),(all_data[cast_index+2].text.strip() if cast_member else 'null'),(all_data[genre_index+2].text.strip() if genre_member else 'null'),(all_data[producer_index+2].text.strip() if producer_member else 'null'),(all_data[title_index+2].find_all('a', href=True)[0]['href'] if len(all_data[title_index+2].find_all('a', href=True))>0 else '')]
          else:
            print(str(len(df))+'!'+all_data[title_index].text.strip() +'|'+(all_data[director_index].text.strip() if director_member else 'null')+'|'+(all_data[cast_index].text.strip() if cast_member else 'null')+'|'+(all_data[genre_index].text.strip() if genre_member else 'null')+'|'+(all_data[producer_index].text.strip() if producer_member else 'null')+'|'+(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else ''))
            df.loc[df.index.max()+1]=[yr,all_data[title_index].text.strip(),(all_data[director_index].text.strip() if director_member else 'null'),(all_data[cast_index].text.strip() if cast_member else 'null'),(all_data[genre_index].text.strip() if genre_member else 'null'),(all_data[producer_index].text.strip() if producer_member else 'null'),(all_data[title_index].find_all('a', href=True)[0]['href'] if len(all_data[title_index].find_all('a', href=True))>0 else '')]


['Opening', 'Title', 'Director', 'Cast', 'Production', 'Music Director']
1!Enakkoru Neethi|K. S. Gopalakrishnan|Siraj, Chithra, Pallavi|null|Anil Creations|/w/index.php?title=Enakkoru_Neethi&action=edit&redlink=1
2!Idhaya Thamarai|K. Rajeshwar|Karthik, Revathi|null|Amartheeb Productions|/wiki/Idhaya_Thamarai
3!Kavalukku Kettikaran|Santhana Bharathi|Prabhu, Nirosha|null|Thirai Koodam|/wiki/Kavalukku_Kettikaran
4!Nalla Kaalam Porandaachu|T. P. Gajendran|Prabhu, Ramya Krishnan|null|Mather Cine Productions|/wiki/Nalla_Kaalam_Porandaachu
5!Pachai Kodi|Manoj Kumar|Pandiarajan, Nirosha|null|Guru Sri Shanthi Productions|/wiki/Pachai_Kodi
6!Panakkaran|P. Vasu|Rajinikanth, Gautami|null|Sathya Movies|/wiki/Panakkaran
7!Pulan Visaranai|R. K. Selvamani|Vijayakanth, Rupini|null|I. V. Cine Productions|/wiki/Pulan_Visaranai
8!Aadi Velli|Rama Narayanan|Nizhalgal Ravi, Seetha|null|Sri Thenaandal Films|/wiki/Aadi_Velli
9!Manasukketha Maapillai|G. Ravi Raja|Pandiarajan, Rohini|null|Amudhasurabi Pictures|/

78!Aalay Pathu Malai Mathu|M. Venkat Vasu|Nizhalgal Ravi, Chandrasekhar, Madhuri, Bhagya, Radha Ravi|null|Kunjapanai Mariamman Films|/wiki/Aalay_Pathu_Malai_Mathu
79!Thalattu Padava|R. Sundarrajan|Parthiban, Kushboo, Rupini|null|Raavuthar Films|/wiki/Thalattu_Padava
80!Oru Veedu Iru Vasal|K. Balachander|Kumaresh, Ganesh, Yamini, Vaishnavi|null|Kavithalayaa Productions|/wiki/Oru_Veedu_Iru_Vasal
81!Puriyaadha Pudhir|K. S. Ravikumar|Rahman, Rekha, Anand Babu, Raghuvaran|null|Super Good Films|/wiki/Puriyaadha_Pudhir
82!Sathyam Sivam Sundaram|Rama Narayanan|Raja, Radha|null|Sree Durga Films|
83!Andhi Varum Neram|R. Ramesh Kumar|Nizhalgal Ravi, Chandrakanth, Latha, Rama|null|G. S. Productions|/w/index.php?title=Andhi_Varum_Neram&action=edit&redlink=1
84!Enkitta Mothathe|R. Sundarrajan|Vijayakanth, Shobana, Kushboo|null|Rajeshwari Productions|/wiki/Enkitta_Mothathe
85!Namathu Deivam|Kasi Arasu|Mohana Muthu, Manorama, 'Baby' Sridevi|null|Apple Creations|/w/index.php?title=Namathu_Deivam&action

In [13]:
getAllMusic()

en.wikipedia.org (random) 🍩
Zankyo Reference (en) data
{
  pageid: 33263484
  requests: <list(1)> random
  title: Zankyo Reference
}
en.wikipedia.org (parse) 33263484
en.wikipedia.org (imageinfo) File:Zankyo Reference Ok One Rock Al...
Zankyo Reference (en) data
{
  image: <list(1)> {'kind': 'parse-cover', 'file': 'File:Zankyo Re...
  infobox: <dict(14)> name, type, artist, cover, released, genre, ...
  pageid: 33263484
  parsetree: <str(16853)> <root><template><title>Infobox album</ti...
  requests: <list(3)> random, parse, imageinfo
  title: Zankyo Reference
  wikibase: Q926808
  wikidata_url: https://www.wikidata.org/wiki/Q926808
  wikitext: <str(7749)> {{Infobox album| name       = Zankyo Refer...
}
en.wikipedia.org (parse) ex.php?title=Enakkoru_Neethi&action=edit...


0 | 
ex.php?title=Enakkoru_Neethi&action=edit&redlink=1


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Idhaya_Thamarai


Exception
1 | 
Idhaya_Thamarai


en.wikipedia.org (imageinfo) File:Idhaya Thamarai DVD cover.jpg


...

API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/mailman/listinfo/mediawiki-api-announce&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Engal_Swamy_Ayyappan


Exception
116 | 
Engal_Swamy_Ayyappan


en.wikipedia.org (imageinfo) File:Engal Swamy Ayyappan DVD cover.jpg


['Dasarathan']
117 | Dasarathan


Engal Swamy Ayyappan (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Engal Swa...
  infobox: <dict(15)> name, image, caption, director, producer, wr...
  pageid: 39151909
  parsetree: <str(8723)> <root><template><title>Use dmy dates</tit...
  requests: <list(2)> parse, imageinfo
  title: Engal Swamy Ayyappan
  wikibase: Q16248276
  wikidata_url: https://www.wikidata.org/wiki/Q16248276
  wikitext: <str(5459)> {{Use dmy dates|date=December 2015}}{{Use ...
}


In [14]:
getAllrating()

https://www.imdb.com/search/title/?year=2017,2017&title_type=feature&explore=languages&languages=ta&ref_=adv_explore_rhs
test | 
https://www.imdb.com/title/tt8947996/
Enakkoru Neethi | 4.8
https://www.imdb.com/title/tt6160582/
Idhaya Thamarai | 5.0
https://www.imdb.com/title/tt0319624/
Kavalukku Kettikaran | 4.8
https://www.imdb.com/title/tt12399124/
Nalla Kaalam Porandaachu | 
https://www.imdb.com/title/tt12399176/
Pachai Kodi | 
https://www.imdb.com/title/tt1286164/
Panakkaran | 6.8
https://www.imdb.com/title/tt0320321/
Pulan Visaranai | 6.9
https://www.imdb.com/title/tt12405532/
Aadi Velli | 
https://www.imdb.com/title/tt12405730/
Manasukketha Maapillai | 
https://www.imdb.com/title/tt12405780/
Pattikattan | 
https://www.imdb.com/title/tt10189646/
Vaazhkai Chakkaram | 
https://www.imdb.com/title/tt0155693/
En Uyir Thozhan | 6.9
https://www.imdb.com/title/tt12409884/
Mr. Karthik | 
https://www.imdb.com/name/nm0035018/
Paattali Magan | 
https://www.imdb.com/title/tt1863366/
Salem Vish

In [15]:
df

year                       title                 director  \
0    2017                        test                     test   
1    1990             Enakkoru Neethi     K. S. Gopalakrishnan   
2    1990             Idhaya Thamarai             K. Rajeshwar   
3    1990        Kavalukku Kettikaran        Santhana Bharathi   
4    1990    Nalla Kaalam Porandaachu          T. P. Gajendran   
5    1990                 Pachai Kodi              Manoj Kumar   
6    1990                  Panakkaran                  P. Vasu   
7    1990             Pulan Visaranai          R. K. Selvamani   
8    1990                  Aadi Velli           Rama Narayanan   
9    1990      Manasukketha Maapillai             G. Ravi Raja   
10   1990                 Pattikattan      T. S. Krishna Kumar   
11   1990          Vaazhkai Chakkaram               Manivannan   
12   1990             En Uyir Thozhan             Bharathiraja   
13   1990                 Mr. Karthik    Kalaivanan Kannadasan   
14   1990              Paattali Magan            Senthilnathan   
15   1990                Salem Vishnu              Thyagarajan   
16   1990             Arangetra Velai                    Fazil   
17   1990       Paattukku Naan Adimai           Shanmuga Rajan   
18   1990             Rajavin Paarvai               A. V. Babu   
19   1990           Valiba Vizhayattu                   Sornam   
20   1990                      Paalam                Karvannan   
21   1990                   Parigaram            Bhama Manalan   
22   1990      Pengal Veettin Kanngal          T. P. Gajendran   
23   1990     Kavithai Paadum Alaigal               T. K. Bose   
24   1990                 Vetri Malai          V. S. Shanmugam   
25   1990          Jagathalaprathapan                   Sankar   
26   1990        Manaivi Oru Manickam              Chozharajan   
27   1990     Unnai Solli Kutramillai                 Ameerjan   
28   1990         En Veedu En Kanavar            Senbaga Raman   
29   1990                Kalyana Rasi            K. Sivaprasad   
..    ...                         ...                      ...   
88   1990                      Thyagu         S. P. Muthuraman   
89   1990          60 Naal 60 Nimidam            K. Rajavarman   
90   1990      Amman Kovil Thiruvizha         P. R. Somasunder   
91   1990          Avasara Police 100             K. Bhagyaraj   
92   1990                   Chatriyan               K. Subaash   
93   1990           En Kadhal Kanmani                T. C. Joy   
94   1990           Mallu Vetti Minor                 Manobala   
95   1990   Michael Madana Kama Rajan  Singeetam Srinivasa Rao   
96   1990              Pudhiya Kaatru                Karvannan   
97   1990                 Puthu Paatu       Panchu Arunachalam   
98   1990      Sirayil Sila Raagangal           Rajendra Kumar   
99   1990             Vellaiya Thevan              Manoj Kumar   
100  1990                  Uchi Veyil             Jaya Sarathi   
101  1990        Pudhu Pudhu Ragangal                  Adhavan   
102  1990         Vaigasi Poranthachu           Radha Bharathi   
103  1990    Aatha Naan Pass Ayittaen          M. K. Sai Mohan   
104  1990        Aerikarai Poongaatre         M. N. Jai Sundar   
105  1990         Namma Ooru Poovatha              Manivasagam   
106  1990               Sathya Vaakku                Arvindraj   
107  1990     Thangaikku Oru Thalattu          K. S. Madhangan   
108  1990       Thai Maasam Poovaasam                 Ameerjan   
109  1990                     Nadigan                  P. Vasu   
110  1990                Urudhi Mozhi         R. V. Udayakumar   
111  1990                Ethir Kaatru         Muktha S. Sundar   
112  1990            Raja Kaiya Vacha           Suresh Krissna   
113  1990     Nee Sirithaal Deepavali       Malaysia Vasudevan   
114  1990          Pudhiya Sarithiram       K. N. Muruganandam   
115  1990         Vaazhnthu Kaattuvom        R. Krishnamoorthy   
116  1990  Avanga Namma Ooru Ponnunga        

In [16]:
df=df.drop(0)
df

year                       title                 director  \
1    1990             Enakkoru Neethi     K. S. Gopalakrishnan   
2    1990             Idhaya Thamarai             K. Rajeshwar   
3    1990        Kavalukku Kettikaran        Santhana Bharathi   
4    1990    Nalla Kaalam Porandaachu          T. P. Gajendran   
5    1990                 Pachai Kodi              Manoj Kumar   
6    1990                  Panakkaran                  P. Vasu   
7    1990             Pulan Visaranai          R. K. Selvamani   
8    1990                  Aadi Velli           Rama Narayanan   
9    1990      Manasukketha Maapillai             G. Ravi Raja   
10   1990                 Pattikattan      T. S. Krishna Kumar   
11   1990          Vaazhkai Chakkaram               Manivannan   
12   1990             En Uyir Thozhan             Bharathiraja   
13   1990                 Mr. Karthik    Kalaivanan Kannadasan   
14   1990              Paattali Magan            Senthilnathan   
15   1990                Salem Vishnu              Thyagarajan   
16   1990             Arangetra Velai                    Fazil   
17   1990       Paattukku Naan Adimai           Shanmuga Rajan   
18   1990             Rajavin Paarvai               A. V. Babu   
19   1990           Valiba Vizhayattu                   Sornam   
20   1990                      Paalam                Karvannan   
21   1990                   Parigaram            Bhama Manalan   
22   1990      Pengal Veettin Kanngal          T. P. Gajendran   
23   1990     Kavithai Paadum Alaigal               T. K. Bose   
24   1990                 Vetri Malai          V. S. Shanmugam   
25   1990          Jagathalaprathapan                   Sankar   
26   1990        Manaivi Oru Manickam              Chozharajan   
27   1990     Unnai Solli Kutramillai                 Ameerjan   
28   1990         En Veedu En Kanavar            Senbaga Raman   
29   1990                Kalyana Rasi            K. Sivaprasad   
30   1990              Muthalali Amma         V. C. Guhanathan   
..    ...                         ...                      ...   
88   1990                      Thyagu         S. P. Muthuraman   
89   1990          60 Naal 60 Nimidam            K. Rajavarman   
90   1990      Amman Kovil Thiruvizha         P. R. Somasunder   
91   1990          Avasara Police 100             K. Bhagyaraj   
92   1990                   Chatriyan               K. Subaash   
93   1990           En Kadhal Kanmani                T. C. Joy   
94   1990           Mallu Vetti Minor                 Manobala   
95   1990   Michael Madana Kama Rajan  Singeetam Srinivasa Rao   
96   1990              Pudhiya Kaatru                Karvannan   
97   1990                 Puthu Paatu       Panchu Arunachalam   
98   1990      Sirayil Sila Raagangal           Rajendra Kumar   
99   1990             Vellaiya Thevan              Manoj Kumar   
100  1990                  Uchi Veyil             Jaya Sarathi   
101  1990        Pudhu Pudhu Ragangal                  Adhavan   
102  1990         Vaigasi Poranthachu           Radha Bharathi   
103  1990    Aatha Naan Pass Ayittaen          M. K. Sai Mohan   
104  1990        Aerikarai Poongaatre         M. N. Jai Sundar   
105  1990         Namma Ooru Poovatha              Manivasagam   
106  1990               Sathya Vaakku                Arvindraj   
107  1990     Thangaikku Oru Thalattu          K. S. Madhangan   
108  1990       Thai Maasam Poovaasam                 Ameerjan   
109  1990                     Nadigan                  P. Vasu   
110  1990                Urudhi Mozhi         R. V. Udayakumar   
111  1990                Ethir Kaatru         Muktha S. Sundar   
112  1990            Raja Kaiya Vacha           Suresh Krissna   
113  1990     Nee Sirithaal Deepavali       Malaysia Vasudevan   
114  1990          Pudhiya Sarithiram       K. N. Muruganandam   
115  1990         Vaazhnthu Kaattuvom        R. Krishnamoorthy   
116  1990  Avanga Namma Ooru Ponnunga        

In [17]:
df.to_csv('1990_movies.csv')